In [81]:
#Import libs 
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
# import optuna
# import os
# import statistics as st
from scipy.ndimage import gaussian_filter
from scipy.interpolate import interp1d
from scipy.stats import gmean
# from scipy import stats
# from scipy.stats import randint
# from scipy.stats import boxcox
# from scipy.special import inv_boxcox
# from scipy.spatial import Voronoi, voronoi_plot_2d
# from IPython.display import display, HTML
import math
# from catboost import CatBoostRegressor
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Circle
# import matplotlib.lines as mlines
import plotly.graph_objects as go
# import plotly.offline as go_offline
# import plotly.io as pio
from plotly.subplots import make_subplots
# import plotly.express as px
from tqdm import tqdm
# import textwrap
import seaborn as sns
from statistics import mean
import geopandas as gpd
from shapely.geometry import Point, Polygon, mapping
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.svm import SVR
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.svm import SVR
# from sklearn.tree import DecisionTreeRegressor 
# from sklearn.linear_model import BayesianRidge
# from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import SGDRegressor
# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.ensemble import AdaBoostRegressor
# from sklearn.gaussian_process import GaussianProcessRegressor
# from sklearn.neural_network import MLPRegressor
# from sklearn.metrics import r2_score as r2 
# from sklearn.metrics import mean_absolute_error as mae 
# from sklearn.metrics import mean_squared_error as mse
# from sklearn.metrics import make_scorer
# from sklearn.metrics.pairwise import euclidean_distances
# from sklearn.inspection import permutation_importance
# from sklearn.pipeline import Pipeline
# from yellowbrick.regressor import PredictionError
# from lightgbm import LGBMRegressor
# from xgboost import XGBRegressor
# from datetime import datetime
# import random

pd.set_option("display.precision", 3)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 15)

## Data upload & interpolation

In [82]:
def cube_data_loading():
       df_cube = pd.read_csv('C:\jupyter\SPP\input\cubeDataBal8_10.csv').drop(columns=['Unnamed: 0'])[['wellName','DEPT', 'DEN_cube_orig_mean_extracted',
              'FORMATION', 'GR_cube_orig_mean_extracted', 'NET', 'PHIT_cube_orig_mean_extracted', 'TST','Vsh_cl_cube_orig_mean_extracted']]
       df_cube = df_cube.rename(columns={'wellName':'well',
                                   'DEPT':'MD',
                                   'DEN_cube_orig_mean_extracted':'RHOB_cube', 
                                   'GR_cube_orig_mean_extracted':'GR_cube', 
                                   'PHIT_cube_orig_mean_extracted':'PHIT_cube', 
                                   'Vsh_cl_cube_orig_mean_extracted':'VSH_cube'})
       df_cube.loc[df_cube.FORMATION.str.contains('Balakhany VIII'), 'FORMATION_up'] = 'Balakhany VIII'
       df_cube.loc[df_cube.FORMATION.str.contains('Balakhany X'), 'FORMATION_up'] = 'Balakhany X'
       return df_cube
df_cube = cube_data_loading()

In [83]:
def well_gr_phit_cube(dataset, formation, var, well):
    well_sel = dataset[(dataset.well == well) & (dataset.FORMATION_up == formation)]
    depth = well_sel['TST']
    vsh = well_sel['VSH_cube']
    phit = well_sel['PHIT_cube']
    net = well_sel[var]
    phit_cutoff = [0.13 for i in range(len(depth))]
    fig = go.Figure()
    fig = make_subplots(rows=1, cols=2, subplot_titles=('VSH', 'PHIT'))
    fig.add_trace(go.Scatter(x=vsh, y=depth, mode='lines', name='VSH', 
                             line=dict(color='gray')), row=1, col=1)
    
    fig.update_xaxes(range = [0,1])
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  
                             line=dict(color='rgba(245, 190, 20, 0.75)'), 
                             fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=1)
    fig.update_xaxes(range = [0, 1], row=1, col=1)

    fig.add_trace(go.Scatter(x=phit, y=depth, mode='lines', name='PHIT', 
                            line = dict(color='firebrick', width=2)), row=1, col=2)
    fig.add_trace(go.Scatter(x=phit_cutoff, y=depth, mode='lines', name='PHIT', 
                            line = dict(color='black', width=2, dash='dash')), row=1, col=2)
    fig.update_xaxes(range = [0.35, 0], row=1, col=2)
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  
                             marker=dict(color='rgba(245, 190, 20, 0.75)'), 
                             fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=2)
    
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(height=700, width=400,
                      title=dict(text=well, font=dict(size=24)),
                      showlegend=False,
                      margin=dict(l=10,r=10,b=10,t=50))
    fig.show()
# well_gr_phit_cube(df_cube, 'Balakhany X','NET', 'A01')

In [84]:
#Loading metadata, distribution wells per Platforms and all the that.
def metadata_parquet_loading():
    path = 'C:\\jupyter\\SPP\\input\\'
    metadata_init = pd.read_csv(path + 'ACG_wells_metadata.csv', sep=',')
    metadata = metadata_init.copy()
    metadata = metadata.rename(columns={'X':'X_wellhead', 'Y':'Y_wellhead'})
    metadata.Status = metadata.Status.str.strip()
    metadata.Status = metadata.Status.str.lower()
    metadata.loc[metadata.Status == 'oil', 'Status' ] = 'production oil'
    metadata.loc[metadata.Status == 'oil producer', 'Status' ] = 'production oil'
    metadata.loc[metadata.Status == 'production', 'Status' ] = 'production oil'
    metadata.loc[metadata.Status == 'produiction oil', 'Status' ] = 'production oil'
    metadata.loc[metadata.Status == 'production_oil', 'Status' ] = 'production oil'
    metadata.loc[metadata.Status == 'abandoned production oil', 'Status' ] = 'abandoned oil'
    metadata.loc[metadata.Status == 'abandoned  oil', 'Status' ] = 'abandoned oil'
    metadata.loc[metadata.Status == 'abandoned oi', 'Status' ] = 'abandoned oil'
    metadata.loc[metadata.Status == 'injector  - water', 'Status' ] = 'injector - water'
    metadata.loc[metadata.Status == 'injector water', 'Status' ] = 'injector - water'
    metadata.loc[metadata.Status == 'injetor  - water', 'Status' ] = 'injector - water'
    metadata.loc[metadata.Status == 'abandoned injector - water per b', 'Status' ] = 'abandoned injector - water'
    metadata.loc[metadata.Status == 'plugged and abandoned', 'Status' ] = 'p&a'
    metadata.loc[metadata.X_wellhead==118.270, 'X_wellhead'] = 526258.84
    metadata.loc[metadata.Y_wellhead==526261.510, 'Y_wellhead'] = 4435802.01
    metadata.loc[metadata.well=='C39', 'X_wellhead'] = 526258.840
    metadata.loc[metadata.well=='C39', 'Y_wellhead'] = 4435802.010
    metadata.loc[metadata.field=='West Azeri', 'field'] = 'WEST AZERI'
    metadata.loc[metadata.field=='COP', 'field'] = 'WEST CHIRAG'
    metadata.loc[metadata.well=='AZERI2', 'field'] = 'WEST AZERI'
    metadata.loc[metadata.well=='AZERI3', 'field'] = 'WEST AZERI'
    metadata.loc[metadata.well=='B31', 'field'] = 'CENTRAL AZERI'
    metadata.loc[metadata.well=='J28_bpQIP', 'field'] = 'WEST CHIRAG'

    #Read data from parquet
    path = 'C:\\jupyter\\SPP\\input\\'
    df_prq = pd.read_parquet(path + 'ACG_wells_JOINT_BEST_v10.parquet.gzip')
    df_prq.rename(columns={'wellName':'well'}, inplace=True)
    df_prq = df_prq.set_index('well').join(metadata.set_index('well')).reset_index()
    # Filter data with bad_well_list 
    bad_well_list = ['E10Z','Predrill_J01Z', 'Predrill_J08', 'J28_bpQIP', 'A01W_2']
    df_prq = df_prq[~df_prq.well.isin(bad_well_list)]
    #Assign any Fluidcode_mod number by variable gross_pay=1 and gross_pay=0 if Fluidcode_mod as NaN
    df_prq.loc[df_prq.FLUIDS>0, 'FLUIDS_int'] = 1
    df_prq.loc[df_prq.FLUIDS<=0, 'FLUIDS_int'] = 0
    df_prq.FLUIDS_int = df_prq.FLUIDS_int.astype('int')
    # Unite of FU for each formation
    df_bal = df_prq[df_prq.FORMATION.str.contains('Balakhany')]
    df_bal.loc[df_bal.FORMATION.str.contains('Balakhany VIII'), 'FORMATION_up'] = 'Balakhany VIII'
    df_bal.loc[df_bal.FORMATION.str.contains('Balakhany X'), 'FORMATION_up'] = 'Balakhany X'
    df_bal = df_bal[df_bal.FORMATION_up.notna()]
    #Getting XY mean coords of Balakhany formation
    xy_coord_mean = df_bal[['well', 'FORMATION_up', 'X', 'Y']]
    xy_coord_mean = xy_coord_mean.groupby(['well', 'FORMATION_up']).agg({'X': 'mean', 'Y':'mean'}).reset_index()
    xy_coord_mean = xy_coord_mean.rename(columns={'X':'X_mean', 'Y':'Y_mean'})
    xy_coord_mean = xy_coord_mean[xy_coord_mean.FORMATION_up.str.contains('Balakhany') & (xy_coord_mean.X_mean>0) & (xy_coord_mean.Y_mean>0)]
    df_bal.rename(columns={'X':'X_traj', 'Y':'Y_traj'}, inplace=True)
    df_bal = df_bal.set_index(['well', 'FORMATION_up']).join(xy_coord_mean.set_index(['well', 'FORMATION_up'])).reset_index()
    return df_bal
df_bal = metadata_parquet_loading()

In [ ]:
def interpolate_by_depth_fm_run(df, step):
    df_tst = df[df.TST.notna()].round({'MD':1})
    
    def interpolate_by_depth_fm(one_well, step):
        one_well = one_well.sort_values(by='TST')
        well_name = one_well["well"].iloc[0]
        data_range = np.floor((one_well["TST"].max() - one_well["TST"].min())/step)
        starting_tst = one_well["TST"].iloc[0]
        new_TST_values = [starting_tst + i*0.1 for i in range(1,int(data_range))]
        interp_MD = interp1d(one_well['TST'], one_well['MD'], kind='linear', fill_value="extrapolate")
        interp_DEVI = interp1d(one_well['TST'], one_well['DEVI'], kind='linear', fill_value="extrapolate")
        interp_HAZI = interp1d(one_well['TST'], one_well['HAZI'], kind='linear', fill_value="extrapolate")
        interp_NET = interp1d(one_well['TST'], one_well['NET'], kind='linear', fill_value="extrapolate")
        interp_NET_VSH = interp1d(one_well['TST'], one_well['NET_VSH'], kind='linear', fill_value="extrapolate")
        # interp_FLUIDS = interp1d(one_well['TST'], one_well['FLUIDS'], kind='linear', fill_value="extrapolate")
        interp_LPERM = interp1d(one_well['TST'], one_well['LPERM'], kind='linear', fill_value="extrapolate")
        interp_PHIT = interp1d(one_well['TST'], one_well['PHIT'], kind='linear', fill_value="extrapolate")
        interp_GR = interp1d(one_well['TST'], one_well['GR_N'], kind='linear', fill_value="extrapolate")
        interp_VSH = interp1d(one_well['TST'], one_well['VSH'], kind='linear', fill_value="extrapolate")
        interp_NPSS = interp1d(one_well['TST'], one_well['NPSS'], kind='linear', fill_value="extrapolate")
        interp_RHOB = interp1d(one_well['TST'], one_well['RHOB'], kind='linear', fill_value="extrapolate")
        interp_RDEEP = interp1d(one_well['TST'], one_well['RDEEP'], kind='linear', fill_value="extrapolate")
        interp_SON = interp1d(one_well['TST'], one_well['SON'], kind='linear', fill_value="extrapolate")
        interp_SONSH = interp1d(one_well['TST'], one_well['SONSH'], kind='linear', fill_value="extrapolate")
        interp_TVDSCS = interp1d(one_well['TST'], one_well['TVD_SCS'], kind='linear', fill_value="extrapolate")
        interp_Xtraj = interp1d(one_well['TST'], one_well['X_traj'], kind='linear', fill_value="extrapolate")
        interp_Ytraj = interp1d(one_well['TST'], one_well['Y_traj'], kind='linear', fill_value="extrapolate")
        interp_Xmean = interp1d(one_well['TST'], one_well['X_mean'], kind='linear', fill_value="extrapolate")
        interp_Ymean = interp1d(one_well['TST'], one_well['Y_mean'], kind='linear', fill_value="extrapolate")    
        # Create a new DataFrame with the interpolated values for new TVD_SCS
        new_data = {
            'well': [well_name for _ in range(len(new_TST_values))],
            # 'FORMATION_up': [formation_name for _ in range(len(new_TST_values))],
            'tst_index': [_ for _ in range(len(new_TST_values))],
            'MD': interp_MD(new_TST_values),
            'TST': new_TST_values,
            'DEVI': interp_DEVI(new_TST_values),
            'HAZI': interp_HAZI(new_TST_values),
            'NET': interp_NET(new_TST_values),
            'NET_VSH': interp_NET_VSH(new_TST_values),
            # 'FLUIDS': interp_FLUIDS(new_TST_values),
            'LPERM': interp_LPERM(new_TST_values),
            'PHIT': interp_PHIT(new_TST_values),
            'GR_N': interp_GR(new_TST_values),
            'VSH': interp_VSH(new_TST_values),
            'NPSS': interp_NPSS(new_TST_values),
            'RHOB': interp_RHOB(new_TST_values),
            'RDEEP': interp_RDEEP(new_TST_values),
            'SON': interp_SON(new_TST_values),
            'SONSH': interp_SONSH(new_TST_values),
            'TVD_SCS': interp_TVDSCS(new_TST_values),
            'X_traj': interp_Xtraj(new_TST_values),
            'Y_traj': interp_Ytraj(new_TST_values),
            'Xmean': interp_Xmean(new_TST_values),
            'Ymean':interp_Ymean(new_TST_values)
        }
        new_df = pd.DataFrame(new_data)
        return new_df
    df_lst = []
    print('Start interpolation')
    for wellnames in tqdm(df_tst.well.unique()):
        well_sel = df_tst[df_tst.well == wellnames]
        well_interp = interpolate_by_depth_fm(well_sel, step)
        df_lst.append(well_interp)
    df_interp = pd.concat(df_lst)
    df_interp = df_interp.round({'MD':1, 'TVD_SCS':1, 'TST':1})

    print('Start joining')
    def well_bal_interp_join(dataset):
        df_tst = df[(df.TST.notna()) & (df.FORMATION_up.notna())].round({'MD':1})
        data_fu = df_tst[['well','MD','FORMATION_up', 'FORMATION', 'field']]
        well_join = dataset.set_index(['well','MD']).join(data_fu.set_index(['well','MD'])).reset_index()
        well_join.insert(3, 'FORMATION_up', well_join.pop('FORMATION_up'))
        well_join.insert(4, 'FORMATION', well_join.pop('FORMATION'))
        well_join.insert(5, 'tst_index', well_join.pop('tst_index'))
        return well_join
    well_interp_v2 = well_bal_interp_join(df_interp)
    well_interp_v2.loc[well_interp_v2.NET_VSH > 0, 'NET_VSH'] = 1
    well_interp_v2.loc[well_interp_v2.NET > 0, 'NET'] = 1
    return well_interp_v2
df_interp_old = interpolate_by_depth_fm_run(df_bal, 0.1)
df_interp_old = df_interp_old[df_interp_old.well != 'B28']
df_interp_old = df_interp_old[df_interp_old.well != 'B27']

In [ ]:
# 'C27X', 'C27Y','J09Z','J13X','C19Z' - wells with absence of X,Y trajectories
def interpolate_by_depth_new_wells():
    df_new = pd.read_parquet('C:\jupyter\SPP\input\JOIN_BEST_new130324.parquet')
    df_new = df_new.replace(-9999.000, 0)

    xy_coord_mean = df_new.groupby(['well', 'FORMATION_up']).agg({'X_traj': 'mean', 'Y_traj':'mean'}).reset_index()
    xy_coord_mean = xy_coord_mean.rename(columns={'X_traj':'X_mean', 'Y_traj':'Y_mean'})
    xy_coord_mean = xy_coord_mean[xy_coord_mean.FORMATION_up.str.contains('Balakhany') & (xy_coord_mean.X_mean>0) & (xy_coord_mean.Y_mean>0)]
    df_new = df_new.set_index(['well', 'FORMATION_up']).join(xy_coord_mean.set_index(['well', 'FORMATION_up'])).reset_index()
    df_new['field'] = 'new_wells'

    def new_well_detection(df_new, df_old):
        new_well = []
        for well_new in df_new.well.unique():
            if well_new in df_old.well.unique():
                pass
            else:
                new_well.append(well_new)
        return new_well
    new_well = new_well_detection(df_new, df_interp_old)
    new_well.append('B28')

    gr8_well_list = ['E20Y', 'E10Z', 'D19Y', 'C27X', 'C19Z', 'B28', 'A01']
    df_new.loc[(df_new.well.isin(gr8_well_list))  & (df_new.FORMATION_up == 'Balakhany VIII') 
                                                & (df_new.GR_N <= 60), 'NET'] = 1
    gr10_well_list = ['A01', 'B28', 'D19Y', 'E20Y', 'J09Z']
    df_new.loc[(df_new.well.isin(gr10_well_list))  & (df_new.FORMATION_up == 'Balakhany X') 
                                                & (df_new.GR_N <= 60), 'NET'] = 1

    def interpolate_by_depth_fm_run(df, step):
        df_tst = df[df.TST.notna()].round({'MD':1})
        
        def interpolate_by_depth_fm(one_well, step):
            one_well = one_well.sort_values(by='TST')
            well_name = one_well["well"].iloc[0]
            data_range = np.floor((one_well["TST"].max() - one_well["TST"].min())/step)
            starting_tst = one_well["TST"].iloc[0]
            new_TST_values = [starting_tst + i*0.1 for i in range(1,int(data_range))]
            interp_MD = interp1d(one_well['TST'], one_well['MD'], kind='linear', fill_value="extrapolate")
            interp_DEVI = interp1d(one_well['TST'], one_well['DEVI'], kind='linear', fill_value="extrapolate")
            interp_HAZI = interp1d(one_well['TST'], one_well['HAZI'], kind='linear', fill_value="extrapolate")
            interp_NET = interp1d(one_well['TST'], one_well['NET'], kind='linear', fill_value="extrapolate")
            interp_NET_VSH = interp1d(one_well['TST'], one_well['NET_VSH'], kind='linear', fill_value="extrapolate")
            # interp_FLUIDS = interp1d(one_well['TST'], one_well['FLUIDS'], kind='linear', fill_value="extrapolate")
            interp_LPERM = interp1d(one_well['TST'], one_well['LPERM'], kind='linear', fill_value="extrapolate")
            interp_PHIT = interp1d(one_well['TST'], one_well['PHIT'], kind='linear', fill_value="extrapolate")
            interp_GR = interp1d(one_well['TST'], one_well['GR_N'], kind='linear', fill_value="extrapolate")
            interp_VSH = interp1d(one_well['TST'], one_well['VSH'], kind='linear', fill_value="extrapolate")
            interp_NPSS = interp1d(one_well['TST'], one_well['NPSS'], kind='linear', fill_value="extrapolate")
            interp_RHOB = interp1d(one_well['TST'], one_well['RHOB'], kind='linear', fill_value="extrapolate")
            interp_RDEEP = interp1d(one_well['TST'], one_well['RDEEP'], kind='linear', fill_value="extrapolate")
            interp_SON = interp1d(one_well['TST'], one_well['SON'], kind='linear', fill_value="extrapolate")
            interp_SONSH = interp1d(one_well['TST'], one_well['SONSH'], kind='linear', fill_value="extrapolate")
            interp_TVDSCS = interp1d(one_well['TST'], one_well['TVD_SCS'], kind='linear', fill_value="extrapolate")
            interp_Xtraj = interp1d(one_well['TST'], one_well['X_traj'], kind='linear', fill_value="extrapolate")
            interp_Ytraj = interp1d(one_well['TST'], one_well['Y_traj'], kind='linear', fill_value="extrapolate")
            interp_Xmean = interp1d(one_well['TST'], one_well['X_mean'], kind='linear', fill_value="extrapolate")
            interp_Ymean = interp1d(one_well['TST'], one_well['Y_mean'], kind='linear', fill_value="extrapolate")    
            # Create a new DataFrame with the interpolated values for new TVD_SCS
            new_data = {
                'well': [well_name for _ in range(len(new_TST_values))],
                # 'FORMATION_up': [formation_name for _ in range(len(new_TST_values))],
                'tst_index': [_ for _ in range(len(new_TST_values))],
                'MD': interp_MD(new_TST_values),
                'TST': new_TST_values,
                'DEVI': interp_DEVI(new_TST_values),
                'HAZI': interp_HAZI(new_TST_values),
                'NET': interp_NET(new_TST_values),
                'NET_VSH': interp_NET_VSH(new_TST_values),
                # 'FLUIDS': interp_FLUIDS(new_TST_values),
                'LPERM': interp_LPERM(new_TST_values),
                'PHIT': interp_PHIT(new_TST_values),
                'GR_N': interp_GR(new_TST_values),
                'VSH': interp_VSH(new_TST_values),
                'NPSS': interp_NPSS(new_TST_values),
                'RHOB': interp_RHOB(new_TST_values),
                'RDEEP': interp_RDEEP(new_TST_values),
                'SON': interp_SON(new_TST_values),
                'SONSH': interp_SONSH(new_TST_values),
                'TVD_SCS': interp_TVDSCS(new_TST_values),
                'X_traj': interp_Xtraj(new_TST_values),
                'Y_traj': interp_Ytraj(new_TST_values),
                'Xmean': interp_Xmean(new_TST_values),
                'Ymean':interp_Ymean(new_TST_values)
            }
            new_df = pd.DataFrame(new_data)
            return new_df
        df_lst = []
        print('Start interpolation')
        for wellnames in tqdm(df_tst.well.unique()):
            well_sel = df_tst[df_tst.well == wellnames]
            well_interp = interpolate_by_depth_fm(well_sel, step)
            df_lst.append(well_interp)
        df_interp = pd.concat(df_lst)
        df_interp = df_interp.round({'MD':1, 'TVD_SCS':1, 'TST':1})

        print('Start joining')
        def well_bal_interp_join(dataset):
            df_tst = df[(df.TST.notna()) & (df.FORMATION_up.notna())].round({'MD':1})
            data_fu = df_tst[['well','MD','FORMATION_up', 'FORMATION', 'field']]
            well_join = dataset.set_index(['well','MD']).join(data_fu.set_index(['well','MD'])).reset_index()
            well_join.insert(3, 'FORMATION_up', well_join.pop('FORMATION_up'))
            well_join.insert(4, 'FORMATION', well_join.pop('FORMATION'))
            well_join.insert(5, 'tst_index', well_join.pop('tst_index'))
            return well_join
        well_interp_v2 = well_bal_interp_join(df_interp)
        well_interp_v2.loc[well_interp_v2.NET_VSH > 0, 'NET_VSH'] = 1
        well_interp_v2.loc[well_interp_v2.NET > 0, 'NET'] = 1
        return well_interp_v2
    df_new_interp = interpolate_by_depth_fm_run(df_new, 0.1)

    def perm_ds_us_calc(dataset):
        def perm_ds_1(x):
            return 10**(39139.46 * pow(x, 5) - 11140.04 * pow(x, 4) + 855.2176 * pow(x, 3) + 7.04505 * pow(x, 2) + 5.750233 * x - 1.997085)
        def perm_us_1(x):
            return 10**(1638286 * pow(x, 5) - 1396883 * pow(x, 4) + 468324.8 * pow(x, 3) - 76974.79 * pow(x, 2) + 6217.262 * x - 198.5042)
        def perm_ds_2(x):
            return 10**(+ 5675.143 * pow(x, 5) - 11106.91 * pow(x, 4) + 8608.366 * pow(x, 3) - 3318.893 * pow(x, 2) + 644.7713 * x - 48.16968)
        def perm_us_2(x):
            return 10**(+ 81.59968 * pow(x, 5) - 275.5442 * pow(x, 4) + 364.9522 * pow(x, 3) - 238.8838 * pow(x, 2) + 79.97139 * x - 7.15)
        df = dataset
        try:
            df.loc[(df.PHIT >= 0.13) & (df.PHIT <=0.2), 'PERM_DS'] = df.PHIT.apply(perm_ds_1)
            df.loc[(df.PHIT >= 0.13) & (df.PHIT <=0.2), 'PERM_US'] = df.PHIT.apply(perm_us_1)
            df.loc[(df.PHIT >= 0.2) & (df.PHIT <=0.4), 'PERM_DS'] = df.PHIT.apply(perm_ds_2)
            df.loc[(df.PHIT >= 0.2) & (df.PHIT <=0.4), 'PERM_US'] = df.PHIT.apply(perm_us_2)
        except:
            df.loc[(df.PHIT >= 0.13) & (df.PHIT <=0.2), 'PERM_DS'] = 0
            df.loc[(df.PHIT >= 0.13) & (df.PHIT <=0.2), 'PERM_US'] = 0
            df.loc[(df.PHIT >= 0.2) & (df.PHIT <=0.4), 'PERM_DS'] = 0
            df.loc[(df.PHIT >= 0.2) & (df.PHIT <=0.4), 'PERM_US'] = 0
        df['PERM_DS'] = df['PERM_DS'].fillna(0)
        df['PERM_US'] = df['PERM_US'].fillna(0)
        return df

    df_new_well8 = df_new_interp[df_new_interp.well.isin(new_well) & (df_new_interp.FORMATION_up == 'Balakhany VIII') & (~df_new_interp.well.isin(['E12Z']))]
    # df_new_well8 = perm_ds_us_calc(df_new_well8)
    df_new_well10 = df_new_interp[df_new_interp.well.isin(new_well) & (df_new_interp.FORMATION_up == 'Balakhany X') & (~df_new_interp.well.isin(['A01','C27X']))]
    # df_new_well10 = perm_ds_us_calc(df_new_well10)
    df_new_interp2 = pd.concat([df_new_well8, df_new_well10])
    return df_new_interp2
df_interp_new = interpolate_by_depth_new_wells()
df_interp_new['NET'] = df_interp_new['NET'].fillna(0)
df_interp_new['V'] = ((df_interp_new.GR_N - 39) / (87-39)).clip(0,1)
df_interp_new['VSH'] = ((1.7 - (3.38 - (df_interp_new.V + 0.7)**2)**0.5)).clip(0,1)

def xy_uploading_corr_run(dataset_base):
    def xy_new_uploading():
        xy_new = pd.read_csv('C:\\jupyter\\SPP\\input\\slb_xy_JOIN_BEST.csv')[1:]
        xy_new = xy_new.rename(columns={'wellName':'well'})
        xy_new = xy_new.replace(-9999.0, np.nan)
        xy_new = xy_new.replace('-9999', np.nan)
        xy_new = xy_new[['well', 'X', 'Y', 'FORMATION']].dropna(subset=['FORMATION','X','Y'])
        xy_new.loc[xy_new.FORMATION.str.contains('Balakhany VIII'), 'FORMATION_up'] = 'Balakhany VIII'
        xy_new.loc[xy_new.FORMATION.str.contains('Balakhany X'), 'FORMATION_up'] = 'Balakhany X'
        xy_new = xy_new[['well', 'X', 'Y', 'FORMATION', 'FORMATION_up']].dropna(subset=['FORMATION_up'])
        xy_new_gb = xy_new.groupby(['well','FORMATION_up']).agg({'X':'mean', 'Y':'mean'}).reset_index().rename(columns={'X':'Xmean', 'Y':'Ymean'})
        return xy_new_gb
    xy_new_gb = xy_new_uploading()

    def xy_new_correction(dataset_base, dataset_add, fm, wellname):
        dataset_base.loc[(dataset_base.well == wellname) & 
                        (dataset_base.FORMATION_up == fm), 
                        'Xmean'] = dataset_add[(dataset_add.well == wellname) & 
                                            (dataset_add.FORMATION_up == fm)]['Xmean'].values[0]
        dataset_base.loc[(dataset_base.well == wellname) & 
                        (dataset_base.FORMATION_up == fm), 
                        'Ymean'] = dataset_add[(dataset_add.well == wellname) & 
                                            (dataset_add.FORMATION_up == fm)]['Ymean'].values[0]
        return dataset_base
    df_interp_new = xy_new_correction(dataset_base, xy_new_gb, 'Balakhany VIII', 'C19Z')
    df_interp_new = xy_new_correction(dataset_base, xy_new_gb, 'Balakhany VIII', 'C27Y')
    df_interp_new = xy_new_correction(dataset_base, xy_new_gb, 'Balakhany X', 'C27Y')
    df_interp_new = xy_new_correction(dataset_base, xy_new_gb, 'Balakhany X', 'J09Z')
    df_interp_new = xy_new_correction(dataset_base, xy_new_gb, 'Balakhany VIII', 'J13X')
    df_interp_new = xy_new_correction(dataset_base, xy_new_gb, 'Balakhany X', 'J13X')
    return df_interp_new
df_interp_new = xy_uploading_corr_run(df_interp_new)

df_interp = pd.concat([df_interp_old, df_interp_new])

In [87]:
def check_presence_gr_cube_techlog():
    df_cube8 = df_cube[df_cube.FORMATION_up == 'Balakhany VIII']
    df_cube10 = df_cube[df_cube.FORMATION_up == 'Balakhany X']
    df_interp8 = df_interp[df_interp.FORMATION_up == 'Balakhany VIII']
    def check_presence_well(dataset, dataset_cube):
        well_list = []
        for well in dataset.well.unique():
            if well in dataset_cube.well.unique():
                pass
            else:
                well_list.append(well)  
        return well_list
    well_list8 = check_presence_well(df_interp8, df_cube8)
    well_list10 = check_presence_well(df_interp, df_cube10)
    well_list8_r = check_presence_well(df_cube8, df_interp8)
    well_list10_r = check_presence_well(df_cube10, df_interp)
    print('well_list8',np.array(well_list8))
    print('well_list10', np.array(well_list10))
    print('well_list8_r', np.array(well_list8_r))
    print('well_list10_r', np.array(well_list10_r))
# check_presence_gr_cube_techlog()

In [88]:
def vsh_phit_cleaning(df, cutoff):
    md_total = df.groupby(['well','FORMATION_up'])['MD'].count().reset_index()
    df_phit = df[df.PHIT > 0]
    phit_total = df_phit.groupby(['well','FORMATION_up'])['PHIT'].count().reset_index()
    df_vsh = df[df.VSH > 0]
    vsh_total = df_vsh.groupby(['well','FORMATION_up'])['VSH'].count().reset_index()
    
    full_df = md_total.set_index(['well','FORMATION_up']).join(phit_total.set_index(['well','FORMATION_up'])).reset_index()
    full_df_v2 = full_df.set_index(['well','FORMATION_up']).join(vsh_total.set_index(['well','FORMATION_up'])).reset_index()
    full_df_v2['phit_md'] = full_df_v2['PHIT'] / full_df_v2['MD']
    full_df_v2['vsh_md'] = full_df_v2['VSH'] / full_df_v2['MD']
    full_df_v2 = full_df_v2.round({'phit_md':2, 'vsh_md':2})

    phit_clean_bal8 = full_df_v2[(full_df_v2['phit_md'] >= cutoff) & (full_df_v2.FORMATION_up == 'Balakhany VIII')].reset_index(drop=True)
    wells_porostiy_back = ['B04', 'B29', 'B31', 'B40', 'C01AY', 'C08Z', 'C38', 'D04Z', 'D29']
    phit_well_back_bal8 = full_df_v2[full_df_v2.well.isin(wells_porostiy_back)]
    phit_clean_bal8 = pd.concat([phit_clean_bal8, phit_well_back_bal8])
    phit_clean_bal10 = full_df_v2[(full_df_v2['phit_md'] >= cutoff) & (full_df_v2.FORMATION_up == 'Balakhany X')].reset_index(drop=True)
    vsh_clean_bal8 = full_df_v2[(full_df_v2['vsh_md'] >= cutoff) & (full_df_v2.FORMATION_up == 'Balakhany VIII')].reset_index(drop=True)
    vsh_clean_bal10 = full_df_v2[(full_df_v2['vsh_md'] >= cutoff) & (full_df_v2.FORMATION_up == 'Balakhany X')].reset_index(drop=True)
    result = {'phit_bal8': phit_clean_bal8, 'phit_bal10':phit_clean_bal10, 'vsh_bal8': vsh_clean_bal8, 'vsh_bal10':vsh_clean_bal10}
    return result
vsh_clean_bal8 = vsh_phit_cleaning(df_interp, 0.9)['vsh_bal8']
phit_clean_bal8 = vsh_phit_cleaning(df_interp, 0.9)['phit_bal8']
vsh_clean_bal10 = vsh_phit_cleaning(df_interp, 0.9)['vsh_bal10']
phit_clean_bal10 = vsh_phit_cleaning(df_interp, 0.9)['phit_bal10']

## NET cleaning for Bal8 Azeri

### netvsh_clp2 bal8

In [89]:
def well_display_logs(dataset, wellname, fmname, net_var, comments, ref_depth, depth_step, print_flag, path):
    """
    dataset = df_bal or something else
    net_var = NET or FLUIDS_int
    comments = put what you want
    ref_depth = MD or TST
    depth_step = step for ticks on the diagramm
    print = 1 if we want to print the plot
    well_display_logs(df_bal8_azr_vshclp2_flag, 'C01', 'Balakhany VIII', 'NET_clp2', 'df_interp', 'TST')
    """
    data = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname)]
    depth = data[ref_depth]
    grn = data['GR_N']
    vsh = data['VSH']
    rhob = data['RHOB'] 
    npss = data['NPSS']
    rdeep = data['RDEEP']
    phit = data['PHIT']
    net_init = data['NET'] 
    net = data[net_var]
    perm = data['LPERM']
    perm1 = data['PERM_DS']
    perm2 = data['PERM_US']
    fig, ax = plt.subplots(1,4, figsize=(7,7), sharey=True)
    well_bal_tops = dataset[(dataset.well == wellname)].groupby('FORMATION')[ref_depth].apply(lambda x: x.iloc[0]).reset_index()
    ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[0].plot(grn, depth, color='lightgreen', lw=3, zorder=10)
    ax[0].invert_yaxis() 
    ax[0].set_xlim(0, 150) 
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
                    xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    twin0 = ax[0].twiny()
    twin0.plot(vsh, depth, color='black', alpha=0.33, zorder=5)
    twin0.vlines(0.5, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    twin0.xaxis.set_ticks(np.arange(0, 1.5, 0.5))
    twin0.set_xlim(0, 1.5)
    twin0.grid(True)
    ax[1].plot(rhob, depth, color='red', alpha=0.75) 
    ax[1].invert_yaxis() 
    ax[1].xaxis.set_ticks(np.arange(1.65, 2.65, 0.3))
    ax[1].grid(True)
    ax[1].set_xlim(1.65, 2.65)
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[1].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
        xmin=0, xmax=150, linewidth=2, color='black', lw=2, alpha=0.33)
        ax[1].text(1.67, well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0]+0.5*depth_step, i, fontsize = 8, color ="black")
    twin1 = ax[1].twiny()
    twin1.plot(npss, depth, color='blue', alpha=0.75)
    twin1.set_xlim(0.6, 0)
    twin1.grid(False)
    twin1.plot(net_init, depth, color='orange', linewidth=0.1)
    twin1.fill_betweenx(depth,net_init, color='orange', alpha=0.25)
    ax[2].plot(phit, depth, color='green', linestyle='dashed')
    ax[2].set_xlim(0.3, 0)
    ax[2].xaxis.set_ticks(np.arange(0, 0.3, 0.1))
    ax[2].tick_params(labelsize=10)
    ax[2].grid(True)
    ax[2].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[2].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    ax[2].vlines(0.13, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    twin2 = ax[2].twiny()
    twin2.plot(net, depth, color='orange', linewidth=0.5, alpha=1)
    twin2.fill_betweenx(depth,net, color='orange', alpha=0.33)
    twin2.set_xlim(0, 1)
    twin2.set_ylim(min(depth), max(depth))
    twin2.xaxis.set_ticks([])
    twin2.grid(False)
    twin3 = ax[2].twiny()
    twin3.plot(rdeep, depth, color='black', linewidth=1, alpha=1)
    twin3.set_xscale('log')
    twin3.set_xlim(1, 200)
    twin3.grid(False)
    ax[3].plot(perm, depth, color='purple', alpha=0.75)
    ax[3].plot(perm1, depth, color='purple', alpha=0.5)
    ax[3].plot(perm2, depth, color='purple', alpha=0.5)
    ax[3].set_xscale('log')
    ax[3].set_xlim(0.1, 1000)
    ax[3].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[3].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    # fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(round(max(kh.dropna()),0)) + ' ' + str(comments), fontsize=14)
    fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(comments), fontsize=14)
    fig.tight_layout()
    if print_flag == 'print':
        fig.savefig(path + wellname + '.png')
    else:
        pass
    plt.show()
# well_display_logs(df_bal8_azr_vshclp2, 'B16Z', 'Balakhany VIII', 'NET_clp2', 'df_interp', 'TST', 10, 'dontprint', 'C:\\jupyter\\SPP\\output\\')

In [90]:
def netvsh_calc_by_gr8(df_interp):
    df_interp.loc[(df_interp.well == 'B17')  & (df_interp.FORMATION_up == 'Balakhany VIII') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'B30')  & (df_interp.FORMATION_up == 'Balakhany VIII') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'B35')  & (df_interp.FORMATION_up == 'Balakhany VIII') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'B36')  & (df_interp.FORMATION_up == 'Balakhany VIII') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'B38Y')  & (df_interp.FORMATION_up == 'Balakhany VIII') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'C27Z')  & (df_interp.FORMATION_up == 'Balakhany VIII') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    return df_interp
df_interp = netvsh_calc_by_gr8(df_interp)

def vsh_calc_by_gr8(df_interp): 
    gr8_well_list = ['B04Z', 'B12Z', 'B18X', 'B18Y', 'B22Z', 'B25', 'B27', 'B28', 'B33',
                    'C03', 'C05', 'C07Z', 'C08', 'C11Z', 'C13', 'C15Z', 'C19', 'C22',
                    'C23', 'C24', 'C25Z', 'C29', 'C30', 'C34Z', 'D01Y', 'D05Y', 'D10',
                    'D14Z', 'D16', 'D17', 'D17Z', 'D18', 'D19', 'D20X', 'D20Z', 'D22',
                    'D23Y', 'D23Z', 'D28']
    df_interp.loc[(df_interp.well.isin(gr8_well_list))  & (df_interp.FORMATION_up == 'Balakhany VIII') 
                                                        & (df_interp.VSH <= 0.5), 'NET'] = 1
    return df_interp
df_interp = vsh_calc_by_gr8(df_interp)

def well_gr_corr_display(dataset, formation, well):
    well_sel = dataset[(dataset.well == well) & (dataset.FORMATION_up == formation)]
    depth = well_sel['TST']
    grn = well_sel['GR_N']
    net = well_sel['NET']*100
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=grn, y=depth, mode='lines', name='GR', marker=dict(color='green')))
    fig.update_xaxes(range=[0, 150])
    fig.update_yaxes(autorange="reversed")
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  marker=dict(color='rgba(245, 190, 20, 0.75)'), fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'))
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(height=700, width=250, title_text=well, showlegend=False,
                    margin=dict(l=10,r=10,b=10,t=40))
    fig.show()
# well_gr_corr_display(df_interp, 'Balakhany VIII', 'B12Z')

In [ ]:
def net_flag_cleaning(dataset_main, fm):
    df_net_bal = dataset_main

    print('Calculation dataframe with h_tst from MD to TST')
    def ntd_calculation_brief(dataset,well, desired_fm, net_var):
        data = dataset[(dataset.well==well)]
        data['TST'].iloc[0] = 0
        data['TST'].iloc[-1] = 0
        tst_top = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i-1][net_var]==0)]
        tst_bot = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i+1][net_var]==0)]
        tops = zip(tst_top, tst_bot)
        df_htst = pd.DataFrame(tops, columns=['tst_top', 'tst_bot'])
        df_htst['FORMATION_up'] = desired_fm
        df_htst['well'] = well
        df_htst['h_tst'] = df_htst.tst_bot - df_htst.tst_top
        df_htst = df_htst[['well','FORMATION_up','tst_top','tst_bot','h_tst']]
        return df_htst
    df_list = []
    for well in tqdm(df_net_bal.well.unique()):
        df = ntd_calculation_brief(df_net_bal, well, fm, 'NET')
        df_list.append(df)
    ntd_net = pd.concat(df_list)
    # ntd_net

    # Cleaning NET variable and making up NET_clp with clipped data, join NET_clp to main dataframe
    print('Cleaning NET variable and making up NET_clp with clipped data')
    def ntd_htst_cleaning(dataset_main, dataset_ntd, cutoff):
        ntd_well = dataset_ntd[(dataset_ntd.well ==well)]
        ntd_well_cutoff = ntd_well[ntd_well.h_tst >= cutoff]
        well_short = dataset_main[['well', 'FORMATION', 'FORMATION_up', 'MD', 'TST', 'GR_N', 'NET', ]]
        net_well = well_short[(well_short.well==well)]
        net_well['NET_clp'] = 0
        for j in range(len(ntd_well_cutoff.well)):
            ntd_top = ntd_well_cutoff.iloc[j, 2].round(3)
            ntd_bot = ntd_well_cutoff.iloc[j, 3].round(3)
            for i in range(len(net_well.TST)):
                well_tst = net_well['TST'].iloc[i].round(3)
                if well_tst >= ntd_top and well_tst <= ntd_bot:
                    net_well['NET_clp'].iloc[i] = 1
        return net_well
    df_list_ntd = []
    for well in tqdm(ntd_net.well.unique()):
        net_well = ntd_htst_cleaning(df_net_bal, ntd_net, 1)
        df_list_ntd.append(net_well)
    net_clp = pd.concat(df_list_ntd)
    # net_clp

    # Cleaning NET_clp from 1-point zero
    print('Cleaning NET_clp from 1-point zero')
    for i in tqdm(range(len(net_clp.NET_clp)-1)):
        if (net_clp.NET_clp.iloc[i] == 0 and  
            net_clp.NET_clp.iloc[i-1] == 1 and 
            net_clp.NET_clp.iloc[i+1] == 1):
            net_clp.NET_clp.iloc[i] = 1        
    df_zero_bal = net_clp[['well', 'MD', 'TST', 'NET_clp', 'FORMATION_up']]
    # df_zero_bal

    print('NET-zero layers searching')
    def ntd_calculation_zero(dataset,well,formation, net_var):
        data = dataset[(dataset.well==well)]
        data.iloc[0, 3] = 1
        data.iloc[-1, 3] = 1
        tst_zero_top = [data.iloc[i]['TST'].round(3) for i in range(len(data)-1)[1:]
                    if (data.iloc[i][net_var] == 0 and data.iloc[i-1][net_var] == 1)]
        tst_zero_bot = [data.iloc[i]['TST'].round(3) for i in range(len(data)-1)[1:] 
                    if (data.iloc[i][net_var] == 0 and data.iloc[i+1][net_var] == 1)]
        tops_zero = zip(tst_zero_top, tst_zero_bot)
        df_zero_htst = pd.DataFrame(tops_zero, columns=['tst_zero_top', 'tst_zero_bot'])
        df_zero_htst['FORMATION_up'] = formation
        df_zero_htst['well'] = well
        df_zero_htst['h_tst_zero'] = df_zero_htst.tst_zero_bot - df_zero_htst.tst_zero_top
        df_zero_htst = df_zero_htst[['well','FORMATION_up','tst_zero_top','tst_zero_bot','h_tst_zero']]
        return df_zero_htst
    df_zero_list = []
    for well in tqdm(df_zero_bal.well.unique()):
        df = ntd_calculation_zero(df_zero_bal, well, fm, 'NET_clp')
        df_zero_list.append(df)
    ntd_zero = pd.concat(df_zero_list)
    # ntd_zero

    print('NET-zero layers removing')
    def ntd_htst_zero_cleaning(dataset_main, dataset_ntd, cutoff, net_var1, net_var2):
        ntd_well_zero = dataset_ntd[(dataset_ntd.well ==well)]
        ntd_well_zero_sel = ntd_well_zero[ntd_well_zero.h_tst_zero <= cutoff]
        well_zero_short = dataset_main[['well','FORMATION_up','MD','TST', net_var1, 'GR_N', 'NET', 'FORMATION']]
        well_zero_short[net_var2] = well_zero_short[net_var1]
        well_zero_sel = well_zero_short[(well_zero_short.well==well)]
        for j in range(len(ntd_well_zero_sel.well)):
            ntd_zero_top = ntd_well_zero_sel.iloc[j, 2].round(3)
            ntd_zero_bot = ntd_well_zero_sel.iloc[j, 3].round(3)
            for i in range(len(well_zero_sel.TST)):
                well_zero_tst = well_zero_sel['TST'].iloc[i].round(3)
                if well_zero_tst >= ntd_zero_top and well_zero_tst <= ntd_zero_bot:
                    well_zero_sel[net_var2].iloc[i] = 1
        return well_zero_sel
    df_list_ntd_zero = []
    for well in tqdm(ntd_zero.well.unique()):
        well_zero_sel = ntd_htst_zero_cleaning(net_clp, ntd_zero, 0.75, 'NET_clp', 'NET_clp2')
        df_list_ntd_zero.append(well_zero_sel)
    net_clp2 = pd.concat(df_list_ntd_zero)
    return net_clp2
black_list_wells8 = ['AZERI4', 'AZERI5', 'CHIRAG1', 'CHIRAG4', 'CHIRAG5', 'CHIRAG6', 
                    'GCA1','GCA2', 'GCA4', 'GCA4Z', 'GCA5', 'GCA5Z', 'GCA6', 'GCA6Y', 'GCA6Z','GCA7',
                    'B04','D02']
df_bal8_azr_vsh = df_interp[(df_interp.well.isin(vsh_clean_bal8.well.unique()))
                            & (~df_interp.well.isin(black_list_wells8)) 
                            & (df_interp.FORMATION_up == 'Balakhany VIII')
                            & (df_interp.field.isin(['WEST AZERI', 'CENTRAL AZERI', 'EAST AZERI','new_wells']))]
netvsh_clp2_bal8 = net_flag_cleaning(df_bal8_azr_vsh, 'Balakhany VIII')

def vsh_bal8_net_corr(test_net_clp2):
    test_net_clp2.loc[(test_net_clp2.well == 'B22')  & (test_net_clp2.TST >= 1982.3) 
                                                & (test_net_clp2.TST <= 1986.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B32')  & (test_net_clp2.TST >= 2346) 
                                                    & (test_net_clp2.TST <= 2349), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B27Z')  & (test_net_clp2.TST >= 147) 
                                                & (test_net_clp2.TST <= 190), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B27Z')  & (test_net_clp2.TST >= 148.9) 
                                                    & (test_net_clp2.TST <= 156.7), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B27Z')  & (test_net_clp2.TST >= 159.1) 
                                                    & (test_net_clp2.TST <= 162.2), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B27Z')  & (test_net_clp2.TST >= 165.8) 
                                                    & (test_net_clp2.TST <= 174), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B27Z')  & (test_net_clp2.TST >= 180.1) 
                                                    & (test_net_clp2.TST <= 189), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C10')  & (test_net_clp2.TST >= 1986) 
                                                    & (test_net_clp2.TST <= 1988), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C10')  & (test_net_clp2.TST >= 1988) 
                                                    & (test_net_clp2.TST <= 1990), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C10')  & (test_net_clp2.TST >= 1990) 
                                                    & (test_net_clp2.TST <= 1995), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C10')  & (test_net_clp2.TST >= 1965) 
                                                    & (test_net_clp2.TST <= 1970), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C13Z')  & (test_net_clp2.TST >= 280) 
                                                    & (test_net_clp2.TST <= 284), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C14Z')  & (test_net_clp2.TST >= 123) 
                                                    & (test_net_clp2.TST <= 128), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C14Z')  & (test_net_clp2.TST >= 133) 
                                                    & (test_net_clp2.TST <= 135), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C14Z')  & (test_net_clp2.TST >= 177) 
                                                    & (test_net_clp2.TST <= 180.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C17')  & (test_net_clp2.TST >= 1363) 
                                                    & (test_net_clp2.TST <= 1366), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C17')  & (test_net_clp2.TST >= 1437) 
                                                    & (test_net_clp2.TST <= 1452), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C17')  & (test_net_clp2.TST >= 1397) 
                                                    & (test_net_clp2.TST <= 1399), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C18')  & (test_net_clp2.TST >= 1543) 
                                                    & (test_net_clp2.TST <= 1545), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C18')  & (test_net_clp2.TST >= 1620) 
                                                    & (test_net_clp2.TST <= 1625), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C18')  & (test_net_clp2.TST >= 1555) 
                                                    & (test_net_clp2.TST <= 1557), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C18')  & (test_net_clp2.TST >= 1593.5) 
                                                    & (test_net_clp2.TST <= 1597.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C21')  & (test_net_clp2.TST >= 1440) 
                                                    & (test_net_clp2.TST <= 1450), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C21')  & (test_net_clp2.TST >= 1525.5) 
                                                    & (test_net_clp2.TST <= 1528), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C21')  & (test_net_clp2.TST >= 1528) 
                                                    & (test_net_clp2.TST <= 1531.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C31')  & (test_net_clp2.TST >= 1937) 
                                                    & (test_net_clp2.TST <= 1939), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C31')  & (test_net_clp2.TST >= 2023) 
                                                    & (test_net_clp2.TST <= 2031.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C42')  & (test_net_clp2.TST >= 2050) 
                                                    & (test_net_clp2.TST <= 2056), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C42')  & (test_net_clp2.TST >= 2109) 
                                                    & (test_net_clp2.TST <= 2111), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C42')  & (test_net_clp2.TST >= 2136) 
                                                    & (test_net_clp2.TST <= 2141), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D05ST1') & (test_net_clp2.TST >= 2025) 
                                                    & (test_net_clp2.TST <= 2031), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D09Z') & (test_net_clp2.TST >= 849) 
                                                    & (test_net_clp2.TST <= 855), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D09Z') & (test_net_clp2.TST >= 837.5) 
                                                    & (test_net_clp2.TST <= 843), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D13Y') & (test_net_clp2.TST >= 295.5) 
                                                    & (test_net_clp2.TST <= 298), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D13Y') & (test_net_clp2.TST >= 246.5) 
                                                    & (test_net_clp2.TST <= 247.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D15') & (test_net_clp2.TST >= 3110) 
                                                    & (test_net_clp2.TST <= 3113), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D15')  & (test_net_clp2.TST >= 3151.4) 
                                                & (test_net_clp2.TST <= 3156.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D15')  & (test_net_clp2.TST >= 3147) 
                                                    & (test_net_clp2.TST <= 3153), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D15')  & (test_net_clp2.TST >= 3160) 
                                                & (test_net_clp2.TST <= 3165), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D23X') & (test_net_clp2.TST >= 456) 
                                                    & (test_net_clp2.TST <= 458), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D24')  & (test_net_clp2.TST >= 2900) 
                                                    & (test_net_clp2.TST <= 2902), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D24')  & (test_net_clp2.TST >= 2918) 
                                                    & (test_net_clp2.TST <= 2922), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D24')  & (test_net_clp2.TST >= 2945) 
                                                    & (test_net_clp2.TST <= 2947), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D24')  & (test_net_clp2.TST >= 2948) 
                                                    & (test_net_clp2.TST <= 2951), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B06')  & (test_net_clp2.TST >= 1891) 
                                                & (test_net_clp2.TST <= 1897), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B11')  & (test_net_clp2.TST >= 1910) 
                                                    & (test_net_clp2.TST <= 1920), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B14Z')  & (test_net_clp2.TST >= 1688) 
                                                    & (test_net_clp2.TST <= 1690), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B17')  & (test_net_clp2.TST >= 1860) 
                                                    & (test_net_clp2.TST <= 1865), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B22')  & (test_net_clp2.TST >= 1951) 
                                                    & (test_net_clp2.TST <= 1960), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B22')  & (test_net_clp2.TST >= 1994.5) 
                                                    & (test_net_clp2.TST <= 1996.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B22')  & (test_net_clp2.TST >= 2053.5) 
                                                    & (test_net_clp2.TST <= 2056.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B29')  & (test_net_clp2.TST >= 2141.5) 
                                                    & (test_net_clp2.TST <= 2146.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B29')  & (test_net_clp2.TST >= 2171) 
                                                    & (test_net_clp2.TST <= 2178), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B31')  & (test_net_clp2.TST >= 1850) 
                                                    & (test_net_clp2.TST <= 1865), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B31')  & (test_net_clp2.TST >= 1981) 
                                                    & (test_net_clp2.TST <= 1986), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B35')  & (test_net_clp2.TST >= 2329) 
                                                    & (test_net_clp2.TST <= 2337), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B36')  & (test_net_clp2.TST >= -23) 
                                                    & (test_net_clp2.TST <= -19.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B39')  & (test_net_clp2.TST >= 680) 
                                                    & (test_net_clp2.TST <= 690), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B40')  & (test_net_clp2.TST >= 106) 
                                                    & (test_net_clp2.TST <= 116), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B41')  & (test_net_clp2.TST >= 310) 
                                                    & (test_net_clp2.TST <= 314), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B41')  & (test_net_clp2.TST >= 318.5) 
                                                    & (test_net_clp2.TST <= 322.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C03Z')  & (test_net_clp2.TST >= 680) 
                                                    & (test_net_clp2.TST <= 690), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C07')  & (test_net_clp2.TST >= 2036.5) 
                                                    & (test_net_clp2.TST <= 2040), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C14')  & (test_net_clp2.TST >= 1960) 
                                                    & (test_net_clp2.TST <= 1970), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C14')  & (test_net_clp2.TST >= 2067) 
                                                    & (test_net_clp2.TST <= 2072), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C14')  & (test_net_clp2.TST >= 2033) 
                                                    & (test_net_clp2.TST <= 2043), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C28')  & (test_net_clp2.TST >= 1755) 
                                                    & (test_net_clp2.TST <= 1760), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C28')  & (test_net_clp2.TST >= 1822) 
                                                    & (test_net_clp2.TST <= 1833), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C29Y')  & (test_net_clp2.TST >= 440) 
                                                    & (test_net_clp2.TST <= 450), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C33')  & (test_net_clp2.TST >= 1604) 
                                                    & (test_net_clp2.TST <= 1614.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C33')  & (test_net_clp2.TST >= 1608) 
                                                    & (test_net_clp2.TST <= 1610), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C33')  & (test_net_clp2.TST >= 1614.5) 
                                                    & (test_net_clp2.TST <= 1617), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C33')  & (test_net_clp2.TST >= 1620) 
                                                    & (test_net_clp2.TST <= 1622), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C33')  & (test_net_clp2.TST >= 1644) 
                                                    & (test_net_clp2.TST <= 1650), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C33Z')  & (test_net_clp2.TST >= 160) 
                                                & (test_net_clp2.TST <= 170), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C33Z')  & (test_net_clp2.TST >= 190) 
                                                    & (test_net_clp2.TST <= 200), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C33Z')  & (test_net_clp2.TST >= 207) 
                                                    & (test_net_clp2.TST <= 210), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C33Z')  & (test_net_clp2.TST >= 211) 
                                                    & (test_net_clp2.TST <= 218.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C36')  & (test_net_clp2.TST >= 2029) 
                                                    & (test_net_clp2.TST <= 2035), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C36')  & (test_net_clp2.TST >= 2150.5) 
                                                    & (test_net_clp2.TST <= 2160.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C37Z')  & (test_net_clp2.TST >= 813.5) 
                                                    & (test_net_clp2.TST <= 818.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C37Z')  & (test_net_clp2.TST >= 820) 
                                                    & (test_net_clp2.TST <= 824.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C38')  & (test_net_clp2.TST >= 2099.5) 
                                                    & (test_net_clp2.TST <= 2104), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C38')  & (test_net_clp2.TST >= 2106) 
                                                    & (test_net_clp2.TST <= 2109.8), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C38')  & (test_net_clp2.TST >= 2113.8) 
                                                    & (test_net_clp2.TST <= 2119.2), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D02')  & (test_net_clp2.TST >= 2931.5) 
                                                    & (test_net_clp2.TST <= 2934), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D02Y')  & (test_net_clp2.TST >= 424.7) 
                                                    & (test_net_clp2.TST <= 426.4), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D03')  & (test_net_clp2.TST >= 2913) 
                                                    & (test_net_clp2.TST <= 2915), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D13Y')  & (test_net_clp2.TST >= 231) 
                                                    & (test_net_clp2.TST <= 233), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D21')  & (test_net_clp2.TST >= 2922.5) 
                                                    & (test_net_clp2.TST <= 2946.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D21')  & (test_net_clp2.TST >= 2995.7) 
                                                    & (test_net_clp2.TST <= 2998.4), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D32')  & (test_net_clp2.TST >= 2976.5) 
                                                    & (test_net_clp2.TST <= 2979.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D35')  & (test_net_clp2.TST >= 2991.3) 
                                                    & (test_net_clp2.TST <= 3001.1), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D35')  & (test_net_clp2.TST >= 3019.5) 
                                                    & (test_net_clp2.TST <= 3023), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D36')  & (test_net_clp2.TST >= 2877.9) 
                                                    & (test_net_clp2.TST <= 2882.4), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D36')  & (test_net_clp2.TST >= 2896.5) 
                                                    & (test_net_clp2.TST <= 2905.1), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D38')  & (test_net_clp2.TST >= 2366.2) 
                                                    & (test_net_clp2.TST <= 2372.3), 'NET_clp2'] = 1
    return test_net_clp2
netvsh_clp2_bal8 = vsh_bal8_net_corr(netvsh_clp2_bal8)

def phit_bal8_net_corr(test_net_clp2): 
    test_net_clp2.loc[(test_net_clp2.well == 'B02')  & (test_net_clp2.TST >= 1920) 
                                                    & (test_net_clp2.TST <= 1930), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B02')  & (test_net_clp2.TST >= 1960) 
                                                    & (test_net_clp2.TST <= 1965), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B02Z')  & (test_net_clp2.TST >= 357.5) 
                                                    & (test_net_clp2.TST <= 363.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B02Z')  & (test_net_clp2.TST >= 372.5) 
                                                    & (test_net_clp2.TST <= 375.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B02Z')  & (test_net_clp2.TST >= 441.8) 
                                                    & (test_net_clp2.TST <= 445.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B05')  & (test_net_clp2.TST >= 1930) 
                                                    & (test_net_clp2.TST <= 1932.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B07')  & (test_net_clp2.TST >= 1850.5) 
                                                    & (test_net_clp2.TST <= 1852.3), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B07')  & (test_net_clp2.TST >= 1893) 
                                                    & (test_net_clp2.TST <= 1898), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B08')  & (test_net_clp2.TST >= 1950) 
                                                    & (test_net_clp2.TST <= 1975), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B08')  & (test_net_clp2.TST >= 1959.4) 
                                                    & (test_net_clp2.TST <= 1963.7), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B08Z')  & (test_net_clp2.TST >= 2754.8) 
                                                    & (test_net_clp2.TST <= 2758.6), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B10')  & (test_net_clp2.TST >= 1816.5) 
                                                    & (test_net_clp2.TST <= 1823), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B10')  & (test_net_clp2.TST >= 1827.5) 
                                                    & (test_net_clp2.TST <= 1831), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B10')  & (test_net_clp2.TST >= 1892.7) 
                                                    & (test_net_clp2.TST <= 1900), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B10')  & (test_net_clp2.TST >= 1910.8) 
                                                    & (test_net_clp2.TST <= 1913.2), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B11')  & (test_net_clp2.TST >= 2015.8) 
                                                    & (test_net_clp2.TST <= 2022.2), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B16Z')  & (test_net_clp2.TST >= 1920.5) 
                                                    & (test_net_clp2.TST <= 1925), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B20')  & (test_net_clp2.TST >= 1963) 
                                                    & (test_net_clp2.TST <= 1964.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B22')  & (test_net_clp2.TST >= 2045) 
                                                    & (test_net_clp2.TST <= 2057), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B22')  & (test_net_clp2.TST >= 2038.6) 
                                                    & (test_net_clp2.TST <= 2042.8), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B22')  & (test_net_clp2.TST >= 2047.2) 
                                                    & (test_net_clp2.TST <= 2050.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B23Z')  & (test_net_clp2.TST >= 1961) 
                                                    & (test_net_clp2.TST <= 1969), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B26')  & (test_net_clp2.TST >= 2110.2) 
                                                    & (test_net_clp2.TST <= 2111.2), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B26')  & (test_net_clp2.TST >= 2113.5) 
                                                    & (test_net_clp2.TST <= 2115.7), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B27Z')  & (test_net_clp2.TST >= 118.5) 
                                                    & (test_net_clp2.TST <= 123.7), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B27Z')  & (test_net_clp2.TST >= 172.5) 
                                                    & (test_net_clp2.TST <= 187.7), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B29')  & (test_net_clp2.TST >= 2231) 
                                                    & (test_net_clp2.TST <= 2235.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B30')  & (test_net_clp2.TST >= 1836.7) 
                                                    & (test_net_clp2.TST <= 1843.1), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B31')  & (test_net_clp2.TST >= 1924) 
                                                    & (test_net_clp2.TST <= 1934), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B31')  & (test_net_clp2.TST >= 1961.5) 
                                                    & (test_net_clp2.TST <= 1966.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B34')  & (test_net_clp2.TST >= 2285.5) 
                                                & (test_net_clp2.TST <= 2294.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B34Z')  & (test_net_clp2.TST >= 675.9) 
                                                    & (test_net_clp2.TST <= 683.2), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B35')  & (test_net_clp2.TST >= 2377) 
                                                    & (test_net_clp2.TST <= 2382), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B37')  & (test_net_clp2.TST >= 118.3) 
                                                    & (test_net_clp2.TST <= 121), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B37')  & (test_net_clp2.TST >= 155.5) 
                                                    & (test_net_clp2.TST <= 157), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B39')  & (test_net_clp2.TST >= 803.7) 
                                                    & (test_net_clp2.TST <= 810), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B40')  & (test_net_clp2.TST >= 164.4) 
                                                    & (test_net_clp2.TST <= 167.3), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C01')  & (test_net_clp2.TST >= 2016.1) 
                                                    & (test_net_clp2.TST <= 2017.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C02')  & (test_net_clp2.TST >= 1914.3) 
                                                    & (test_net_clp2.TST <= 1918.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C02')  & (test_net_clp2.TST >= 1914.3) 
                                                    & (test_net_clp2.TST <= 1918.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C02')  & (test_net_clp2.TST >= 1919.4) 
                                                    & (test_net_clp2.TST <= 1920.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C06')  & (test_net_clp2.TST >= 2049) 
                                                    & (test_net_clp2.TST <= 2056), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C07')  & (test_net_clp2.TST >= 2035.3) 
                                                    & (test_net_clp2.TST <= 2041.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C10')  & (test_net_clp2.TST >= 2021) 
                                                    & (test_net_clp2.TST <= 2022), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C10')  & (test_net_clp2.TST >= 2078.4) 
                                                    & (test_net_clp2.TST <= 2080), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C11')  & (test_net_clp2.TST >= 2081) 
                                                    & (test_net_clp2.TST <= 2097), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C11')  & (test_net_clp2.TST >= 2090) 
                                                    & (test_net_clp2.TST <= 2091), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C12')  & (test_net_clp2.TST >= 1876.8) 
                                                    & (test_net_clp2.TST <= 1877.8), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C12')  & (test_net_clp2.TST >= 1890) 
                                                    & (test_net_clp2.TST <= 1891.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C12')  & (test_net_clp2.TST >= 1899.3) 
                                                    & (test_net_clp2.TST <= 1901.7), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C13Z')  & (test_net_clp2.TST >= 278.3) 
                                                    & (test_net_clp2.TST <= 285.7), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C17')  & (test_net_clp2.TST >= 1369.3) 
                                                    & (test_net_clp2.TST <= 1372.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C17')  & (test_net_clp2.TST >= 1385.5) 
                                                    & (test_net_clp2.TST <= 1405), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C17')  & (test_net_clp2.TST >= 1435) 
                                                    & (test_net_clp2.TST <= 1440.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C20Z')  & (test_net_clp2.TST >= 1578) 
                                                    & (test_net_clp2.TST <= 1582.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C21')  & (test_net_clp2.TST >= 1529) 
                                                    & (test_net_clp2.TST <= 1535), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C21Z')  & (test_net_clp2.TST >= 312) 
                                                    & (test_net_clp2.TST <= 323), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C21Z')  & (test_net_clp2.TST >= 367) 
                                                    & (test_net_clp2.TST <= 371.1), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C29Y')  & (test_net_clp2.TST >= 441.7) 
                                                    & (test_net_clp2.TST <= 442.7), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C29Y')  & (test_net_clp2.TST >= 456.3) 
                                                    & (test_net_clp2.TST <= 460), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C31')  & (test_net_clp2.TST >= 1991) 
                                                    & (test_net_clp2.TST <= 1997), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C33Z')  & (test_net_clp2.TST >= 189.5) 
                                                    & (test_net_clp2.TST <= 192.9), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C34')  & (test_net_clp2.TST >= 2053.1) 
                                                    & (test_net_clp2.TST <= 2058.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C40')  & (test_net_clp2.TST >= 1891.2) 
                                                    & (test_net_clp2.TST <= 1894.7), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C40')  & (test_net_clp2.TST >= 1935.2) 
                                                    & (test_net_clp2.TST <= 1938.7), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C41')  & (test_net_clp2.TST >= 2218.2) 
                                                    & (test_net_clp2.TST <= 2222.7), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C41')  & (test_net_clp2.TST >= 2252.1) 
                                                    & (test_net_clp2.TST <= 2253.4), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C41')  & (test_net_clp2.TST >= 2288) 
                                                    & (test_net_clp2.TST <= 2292), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C42')  & (test_net_clp2.TST >= 2133) 
                                                    & (test_net_clp2.TST <= 2144), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C42')  & (test_net_clp2.TST >= 2180) 
                                                    & (test_net_clp2.TST <= 2190), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D02Z')  & (test_net_clp2.TST >= 274.4) 
                                                    & (test_net_clp2.TST <= 277), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D03Z')  & (test_net_clp2.TST >= 295) 
                                                    & (test_net_clp2.TST <= 301), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D03Z')  & (test_net_clp2.TST >= 343) 
                                                    & (test_net_clp2.TST <= 352), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D04Z')  & (test_net_clp2.TST >= 755) 
                                                    & (test_net_clp2.TST <= 760.7), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D04Z')  & (test_net_clp2.TST >= 768.5) 
                                                    & (test_net_clp2.TST <= 771.7), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D07')  & (test_net_clp2.TST >= 2962) 
                                                    & (test_net_clp2.TST <= 2966), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D12Z')  & (test_net_clp2.TST >= 2120) 
                                                    & (test_net_clp2.TST <= 2130), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D12Z')  & (test_net_clp2.TST >= 2219.4) 
                                                    & (test_net_clp2.TST <= 2224.7), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D13')  & (test_net_clp2.TST >= 3063) 
                                                    & (test_net_clp2.TST <= 3070), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D13Y')  & (test_net_clp2.TST >= 231) 
                                                    & (test_net_clp2.TST <= 250), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D13Y')  & (test_net_clp2.TST >= 290) 
                                                    & (test_net_clp2.TST <= 301.8), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D13Y')  & (test_net_clp2.TST >= 303.7) 
                                                    & (test_net_clp2.TST <= 311.8), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D15')  & (test_net_clp2.TST >= 3118) 
                                                    & (test_net_clp2.TST <= 3127.8), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D24')  & (test_net_clp2.TST >= 2942.5) 
                                                    & (test_net_clp2.TST <= 2953.8), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D25')  & (test_net_clp2.TST >= 2942.5) 
                                                    & (test_net_clp2.TST <= 2953.8), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D27')  & (test_net_clp2.TST >= 2158) 
                                                    & (test_net_clp2.TST <= 2163), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D27')  & (test_net_clp2.TST >= 2209) 
                                                    & (test_net_clp2.TST <= 2213.3), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D33')  & (test_net_clp2.TST >= 2986.6) 
                                                    & (test_net_clp2.TST <= 2989.9), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D36')  & (test_net_clp2.TST >= 2882.4) 
                                                    & (test_net_clp2.TST <= 2884.7), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D36')  & (test_net_clp2.TST >= 2894.7) 
                                                    & (test_net_clp2.TST <= 2901.1), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D37')  & (test_net_clp2.TST >= 2915) 
                                                    & (test_net_clp2.TST <= 2920), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D38')  & (test_net_clp2.TST >= 2441) 
                                                    & (test_net_clp2.TST <= 2454), 'NET_clp2'] = 1
    return test_net_clp2
netvsh_clp2_bal8 = phit_bal8_net_corr(netvsh_clp2_bal8)

df_bal8_azr_vshclp2 = df_bal8_azr_vsh.set_index(['well','TST']).join(
                      netvsh_clp2_bal8.drop(['FORMATION_up','MD','GR_N','NET','FORMATION'], axis=1).set_index(
                                            ['well','TST'])).reset_index()
df_bal8_azr_vshclp2.loc[df_bal8_azr_vshclp2.well.isin(phit_clean_bal8.well.unique()), 'phit_flag'] = 1
df_bal8_azr_vshclp2['phit_flag'] = df_bal8_azr_vshclp2['phit_flag'].fillna(0)

def perm_ds_us_calc(dataset):
    def perm_ds_1(x):
        return 10**(39139.46 * pow(x, 5) - 11140.04 * pow(x, 4) + 855.2176 * pow(x, 3) + 7.04505 * pow(x, 2) + 5.750233 * x - 1.997085)
    def perm_us_1(x):
        return 10**(1638286 * pow(x, 5) - 1396883 * pow(x, 4) + 468324.8 * pow(x, 3) - 76974.79 * pow(x, 2) + 6217.262 * x - 198.5042)
    def perm_ds_2(x):
        return 10**(+ 5675.143 * pow(x, 5) - 11106.91 * pow(x, 4) + 8608.366 * pow(x, 3) - 3318.893 * pow(x, 2) + 644.7713 * x - 48.16968)
    def perm_us_2(x):
        return 10**(+ 81.59968 * pow(x, 5) - 275.5442 * pow(x, 4) + 364.9522 * pow(x, 3) - 238.8838 * pow(x, 2) + 79.97139 * x - 7.15)
    df = dataset.copy()
    df.loc[(df.PHIT >= 0.13) & (df.PHIT <=0.2), 'PERM_DS'] = df.PHIT.apply(perm_ds_1)
    df.loc[(df.PHIT >= 0.13) & (df.PHIT <=0.2), 'PERM_US'] = df.PHIT.apply(perm_us_1)
    df.loc[(df.PHIT >= 0.2) & (df.PHIT <=1), 'PERM_DS'] = df.PHIT.apply(perm_ds_2)
    df.loc[(df.PHIT >= 0.2) & (df.PHIT <=1), 'PERM_US'] = df.PHIT.apply(perm_us_2)
    df['PERM_DS'] = df['PERM_DS'].fillna(0)
    df['PERM_US'] = df['PERM_US'].fillna(0)
    return df
df_bal8_azr_vshclp2 = perm_ds_us_calc(df_bal8_azr_vshclp2)

df_bal8_azr_vshclp2.to_csv('C:\jupyter\SPP\inputoutput\df_bal8_azr_vshclp2.csv')

# B14Z, B16Z, B19, B20, C03Z, C10, C11, C21, C32, D12Z, D20 - корректирока NET по VSH после совещания с ВР
def well_gr_smooth_display(dataset, formation, var, well):
    well_sel = dataset[(dataset.well == well) & (dataset.FORMATION_up == formation)]
    depth = well_sel['TST']
    grn = well_sel['VSH_smooth']
    # grn = well_sel['VSH']
    phit = well_sel['PHIT']
    net = well_sel[var]
    net_smooth = well_sel['NET_smooth']*100
    # net_smooth = net
    phit_cutoff = [0.13 for i in range(len(depth))]
    fig = go.Figure()
    fig = make_subplots(rows=1, cols=2, subplot_titles=('VSH', 'PHIT'))
    fig.add_trace(go.Scatter(x=grn, y=depth, mode='lines', name='VSH', 
                             line=dict(color='gray')), row=1, col=1)
    
    fig.update_xaxes(range = [0,1])
    fig.add_trace(go.Scatter(x=net_smooth, y=depth, mode='lines', name='NET',  
                             line=dict(color='rgba(245, 190, 20, 0.75)'), 
                             fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=1)
    fig.update_xaxes(range = [0, 1], row=1, col=1)

    fig.add_trace(go.Scatter(x=phit, y=depth, mode='lines', name='PHIT', 
                            line = dict(color='firebrick', width=2)), row=1, col=2)
    fig.add_trace(go.Scatter(x=phit_cutoff, y=depth, mode='lines', name='PHIT', 
                            line = dict(color='black', width=2, dash='dash')), row=1, col=2)
    fig.update_xaxes(range = [0.35, 0], row=1, col=2)
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  
                             marker=dict(color='rgba(245, 190, 20, 0.75)'), 
                             fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=2)
    
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(height=900, width=400,
                      title=dict(text=well, font=dict(size=24)),
                      showlegend=False,
                      margin=dict(l=10,r=10,b=10,t=50))
    fig.show()
def well_gaussian_filter_run(percentage):
    well_smooth_list = ['B14Z', 'B16Z', 'B19', 'B20', 'B22Z','C03Z', 'C10', 'C11', 'C21', 'C32', 'D12Z', 'D20']
    smooth_gr = df_bal8_azr_vshclp2[df_bal8_azr_vshclp2.well.isin(well_smooth_list)]
    def well_gaussian_filter(dataset, wellname, fmname, var, percentage):
        data_full = dataset[dataset.well == wellname]  
        data = dataset[dataset.well == wellname][var]
        coeff = percentage/100
        sigma = int(round(len(data)*coeff, 0))
        data = data.reset_index().drop('index', axis=1)
        smoothed_data = gaussian_filter(data, sigma=sigma)
        data_full[var + '_smooth'] = smoothed_data
        return data_full
    df_lst = []
    for wellname in smooth_gr.well.unique():
        smooth_data = well_gaussian_filter(smooth_gr, wellname, 'Balakhany VIII', 'VSH', percentage)
        df_lst.append(smooth_data)
    vsh_smooth = pd.concat(df_lst)
    vsh_smooth['NET_smooth'] = 0
    vsh_smooth.loc[vsh_smooth.VSH_smooth <= 0.5, 'NET_smooth'] = 1
    return vsh_smooth
vsh_smooth = well_gaussian_filter_run(percentage = 0.2)
drop_smooth_wells = df_bal8_azr_vshclp2[~df_bal8_azr_vshclp2.well.isin(['B14Z', 'B16Z', 'B19', 'B20', 'B22Z', 'C03Z', 'C10', 'C11', 'C21', 'C32', 'D12Z', 'D20'])]
add_smooth_wells = pd.concat([drop_smooth_wells, vsh_smooth]).sort_values(by=['well', 'MD'])
df_bal8_azr_v2 = add_smooth_wells.copy()
df_bal8_azr_v2.loc[df_bal8_azr_v2['NET_smooth'].notna(), 'NET_orig'] = df_bal8_azr_v2['NET']
df_bal8_azr_v2.loc[df_bal8_azr_v2['NET_smooth'].notna(), 'NET'] = df_bal8_azr_v2['NET_smooth']

def net_correction(df_bal8_azr_v2):
    vsh_cuttoff = 0.35
    df_bal8_azr_v2.loc[ (df_bal8_azr_v2.well=='C34Z'), 'NET'] = 0
    df_bal8_azr_v2.loc[ (df_bal8_azr_v2.well=='C34Z') &
                        (df_bal8_azr_v2.VSH <= vsh_cuttoff), 'NET'] = 1
    df_bal8_azr_v2.loc[ (df_bal8_azr_v2.well=='C34Z'), 'NET_VSH'] = 0
    df_bal8_azr_v2.loc[ (df_bal8_azr_v2.well=='C34Z') &
                        (df_bal8_azr_v2.VSH <= vsh_cuttoff), 'NET_VSH'] = 1
    return df_bal8_azr_v2
df_bal8_azr_v2 = net_correction(df_bal8_azr_v2)

df_bal8_azr_v2.to_csv('C:\jupyter\SPP\inputoutput\df_bal8_azr_v2.csv')

In [ ]:
def well_display_logs(dataset, wellname, fmname, net_var, comments, ref_depth, depth_step, print_flag, path):
    """
    dataset = df_bal or something else
    net_var = NET or FLUIDS_int
    comments = put what you want
    ref_depth = MD or TST
    depth_step = step for ticks on the diagramm
    print = 1 if we want to print the plot
    well_display_logs(df_bal8_azr_vshclp2_flag, 'C01', 'Balakhany VIII', 'NET_clp2', 'df_interp', 'TST')
    """
    data = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname)]
    depth = data[ref_depth]
    grn = data['GR_N']
    vsh = data['VSH']
    rhob = data['RHOB'] 
    npss = data['NPSS']
    rdeep = data['RDEEP']
    phit = data['PHIT']
    net_init = data['NET_VSH']*100 
    net = data[net_var]*100
    perm = data['LPERM']
    perm1 = data['PERM_DS']
    perm2 = data['PERM_US']

    net_sum = (net.sum()*0.001).round(1)

    fig, ax = plt.subplots(1,4, figsize=(7,11), sharey=True)
    well_bal_tops = dataset[(dataset.well == wellname)].groupby('FORMATION')[ref_depth].apply(lambda x: x.iloc[0]).reset_index()
    ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[0].plot(grn, depth, color='lightgreen', lw=3, zorder=10)
    ax[0].invert_yaxis() 
    ax[0].set_xlim(0, 150) 
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
                    xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    ax[0].set_title('net_vsh')
    twin0 = ax[0].twiny()
    twin0.plot(vsh, depth, color='black', alpha=0.33, zorder=5)
    twin0.vlines(0.5, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    twin0.xaxis.set_ticks(np.arange(0, 1.5, 0.5))
    twin0.set_xlim(0, 1.5)
    twin0.grid(True)
    twin0.plot(net, depth, color='orange', linewidth=0.5, alpha=1)
    twin0.fill_betweenx(depth,net, color='orange', alpha=0.25)
    ax[1].plot(rhob, depth, color='red', alpha=0.75) 
    ax[1].invert_yaxis() 
    ax[1].xaxis.set_ticks(np.arange(1.65, 2.65, 0.3))
    ax[1].grid(True)
    ax[1].set_xlim(1.65, 2.65)
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[1].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
        xmin=0, xmax=150, linewidth=2, color='black', lw=2, alpha=0.33)
        ax[1].text(1.67, well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0]+0.5*depth_step, i, fontsize = 8, color ="black")
    twin1 = ax[1].twiny()
    twin1.plot(npss, depth, color='blue', alpha=0.75)
    twin1.set_xlim(0.6, 0)
    twin1.grid(False)
    ax[2].plot(phit, depth, color='green', linestyle='dashed')
    ax[2].set_xlim(0.3, 0)
    ax[2].xaxis.set_ticks(np.arange(0, 0.3, 0.1))
    ax[2].tick_params(labelsize=10)
    ax[2].grid(True)
    ax[2].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[2].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    ax[2].vlines(0.13, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    ax[2].set_title('net')
    twin2 = ax[2].twiny()
    twin2.plot(net, depth, color='orange', linewidth=0.5, alpha=1)
    twin2.fill_betweenx(depth,net, color='orange', alpha=0.33)
    twin2.set_xlim(0, 1)
    twin2.set_ylim(min(depth), max(depth))
    twin2.xaxis.set_ticks([])
    twin2.grid(False)
    twin3 = ax[2].twiny()
    twin3.plot(rdeep, depth, color='black', linewidth=1, alpha=1)
    twin3.set_xscale('log')
    twin3.set_xlim(1, 200)
    twin3.grid(False)
    ax[3].plot(perm, depth, color='purple', alpha=0.75)
    ax[3].plot(perm1, depth, color='purple', alpha=0.5)
    ax[3].plot(perm2, depth, color='purple', alpha=0.5)
    ax[3].set_xscale('log')
    ax[3].set_xlim(0.1, 1000)
    ax[3].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[3].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    # fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(round(max(kh.dropna()),0)) + ' ' + str(comments), fontsize=14)
    fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(comments) + str(net_sum) + 'm', fontsize=14)
    fig.tight_layout()
    if print_flag == 'print':
        fig.savefig(path + wellname + '.png', dpi=300)
    else:
        pass
    plt.show()

# well_display_logs(df_bal8_azr_v2, 'C43', 'Balakhany VIII', 'NET_VSH', 'df_interp', 'TST', 10, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
for wellname in df_bal8_azr_v2.well.unique():
    well_display_logs(df_bal8_azr_v2, wellname, 'Balakhany VIII', 'NET', 'net:', 'TST', 5, 'print', 'C:\\jupyter\\SPP\\plots\\logs_vsh_bal8\\')

In [ ]:
# ['B04Z', 'B12Z', 'B18X', 'B18Y', 'B22Z', 'B25', 'B27', 'B28', 'B33', 'C03', 'C05', 'C07Z', 'C08', 'C11Z', 'C13', 'C15Z', 'C19', 'C22',
#  'C23', 'C24', 'C25Z', 'C29', 'C30', 'C34Z', 'D01Y', 'D05Y', 'D10', 'D14Z', 'D16', 'D17', 'D17Z', 'D18', 'D19', 'D20X', 'D20Z', 'D22',
#  'D23Y', 'D23Z', 'D28']

def well_gr_phit_corr_display(dataset, formation, var, well):
    well_sel = dataset[(dataset.well == well) & (dataset.FORMATION_up == formation)]
    depth = well_sel['TST']
    grn = well_sel['GR_N']
    phit = well_sel['PHIT']
    rdeep = well_sel['RDEEP']
    net = well_sel[var]*100
    phit_cutoff = [0.13 for i in range(len(depth))]
    fig = go.Figure()
    fig = make_subplots(rows=1, cols=3, subplot_titles=('GR', 'PHIT', 'RDEEP'))
    fig.add_trace(go.Scatter(x=grn, y=depth, mode='lines', name='GR', 
                             line=dict(color='green')), row=1, col=1)
    
    fig.update_xaxes(range = [0,150])
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  
                             line=dict(color='rgba(245, 190, 20, 0.75)'), 
                             fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=1)
    fig.update_xaxes(range = [0, 150], row=1, col=1)

    fig.add_trace(go.Scatter(x=phit, y=depth, mode='lines', name='PHIT', 
                            line = dict(color='firebrick', width=2)), row=1, col=2)
    fig.add_trace(go.Scatter(x=phit_cutoff, y=depth, mode='lines', name='PHIT', 
                            line = dict(color='black', width=2, dash='dash')), row=1, col=2)
    fig.update_xaxes(range = [0.35, 0], row=1, col=2)
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  
                             marker=dict(color='rgba(245, 190, 20, 0.75)'), 
                             fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=2)
    fig.add_trace(go.Scatter(x=rdeep, y=depth, mode='lines', name='RDEEP', 
                            line = dict(color='black', width=2, dash='dash')), row=1, col=3)
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  
                             marker=dict(color='rgba(245, 190, 20, 0.75)'), 
                             fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=3)
    fig.update_xaxes(range = [0, 2], type='log', row=1, col=3)
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(height=850, width=500,
                      title=dict(text=well, font=dict(size=24)),
                      showlegend=False,
                      margin=dict(l=10,r=10,b=10,t=50),
                      xaxis2=dict(  title='xaxis2 title',
                                    overlaying='x2',
                                    side='right'))
    fig.show()
# well_gr_phit_corr_display(df_bal8_azr_v2, 'Balakhany VIII','NET', 'B21')

In [ ]:
def well_display_net(dataset, well, formation, net1, net2, depth_step, print_flag, path):
    well_sel = dataset[(dataset.well == well) & (dataset.FORMATION_up == formation)]
    depth = well_sel['TST']
    grn = well_sel['GR_N']
    net = well_sel['NET']
    net_clp = well_sel[net1]
    net_clp2 = well_sel[net2]
    
    fig, ax = plt.subplots(1,3, figsize=(6,8), sharey=True)
    ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[0].plot(net, depth, color='orange', lw=0.25)
    ax[0].fill_betweenx(depth,net, color='orange', alpha=0.33)
    ax[0].invert_yaxis()
    ax[0].set_xlim(0, 1) 
    ax[0].set_title('NET')
    well_bal_tops = well_sel.groupby('FORMATION')['TST'].apply(lambda x: x.iloc[0]).reset_index()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains('Balakhany VIII')].FORMATION:
        ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i]['TST'].iloc[0], xmin=0, xmax=150, color='black', lw=2, alpha=0.66)
        ax[0].text(0.1, well_bal_tops[well_bal_tops.FORMATION==i]['TST'].iloc[0]+3, i, fontsize = 7, color ="black")
    twin0 = ax[0].twiny()
    twin0.plot(grn, depth, color='green')
    twin0.set_xlim(0, 150)
    twin0.set_xticks([])
    # ax[1].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[1].plot(net_clp, depth, color='orange', lw=0.25)
    ax[1].set_title('NET_clp')
    ax[1].set_xlim(0, 1)
    ax[1].grid(axis='y')
    ax[1].fill_betweenx(depth, net_clp, color='orange', alpha=0.33)
    twin1 = ax[1].twiny()
    twin1.plot(grn, depth, color='green')
    twin1.set_xlim(0, 150)
    twin1.set_xticks([])
    # ax[2].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[2].plot(net_clp2, depth, color='orange', lw=0.25)
    ax[2].set_title('NET_clp2')
    ax[2].set_xlim(0, 1)
    ax[2].grid(axis='y')
    ax[2].fill_betweenx(depth,net_clp2, color='orange', alpha=0.33)
    twin2 = ax[2].twiny()
    twin2.plot(grn, depth, color='green')
    twin2.set_xlim(0, 150)
    twin2.set_xticks([])
    fig.suptitle(well_sel.well.unique()[0], fontsize=14)
    fig.tight_layout()
    if print_flag == 'print':
        fig.savefig(path + well + '.png')
    else:
        pass
    return fig.show()
# for well in netvsh_clp2_bal8.well.unique():
#     well_display_net(netvsh_clp2_bal8, well, 'Balakhany VIII', 'NET_clp','NET_clp2', 10, 'print', 'C:\\jupyter\\SPP\plots\\netvsh_clp2_bal8\\')

### net_clp2 bal8

In [ ]:
def net_flag_cleaning(dataset_main, well_list_clean, fm):
    df_net_bal = dataset_main[dataset_main.well.isin(well_list_clean) & (dataset_main.FORMATION_up==fm)]

    print('Calculation dataframe with h_tst from MD to TST')
    def ntd_calculation_brief(dataset,well,desired_fm, net_var):
        data = dataset[(dataset.well==well)]
        data['TST'].iloc[0] = 0
        data['TST'].iloc[-1] = 0
        tst_top = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i-1][net_var]==0)]
        tst_bot = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i+1][net_var]==0)]
        tops = zip(tst_top, tst_bot)
        df_htst = pd.DataFrame(tops, columns=['tst_top', 'tst_bot'])
        df_htst['FORMATION_up'] = desired_fm
        df_htst['well'] = well
        df_htst['h_tst'] = df_htst.tst_bot - df_htst.tst_top
        df_htst = df_htst[['well','FORMATION_up','tst_top','tst_bot','h_tst']]
        return df_htst
    df_list = []
    for well in tqdm(df_net_bal.well.unique()):
        df = ntd_calculation_brief(df_net_bal, well, fm, 'NET')
        df_list.append(df)
    ntd_net = pd.concat(df_list)
    # ntd_net

    # Cleaning NET variable and making up NET_clp with clipped data, join NET_clp to main dataframe
    print('Cleaning NET variable and making up NET_clp with clipped data')
    def ntd_htst_cleaning(dataset_main, dataset_ntd, cutoff):
        ntd_well = dataset_ntd[(dataset_ntd.well ==well)]
        ntd_well_cutoff = ntd_well[ntd_well.h_tst >= cutoff]
        well_short = dataset_main[['well', 'FORMATION', 'FORMATION_up', 'MD', 'TST', 'GR_N', 'NET', ]]
        net_well = well_short[(well_short.well==well)]
        net_well['NET_clp'] = 0
        for j in range(len(ntd_well_cutoff.well)):
            ntd_top = ntd_well_cutoff.iloc[j, 2].round(3)
            ntd_bot = ntd_well_cutoff.iloc[j, 3].round(3)
            for i in range(len(net_well.TST)):
                well_tst = net_well['TST'].iloc[i].round(3)
                if well_tst >= ntd_top and well_tst <= ntd_bot:
                    net_well['NET_clp'].iloc[i] = 1
        return net_well
    df_list_ntd = []
    for well in tqdm(ntd_net.well.unique()):
        net_well = ntd_htst_cleaning(df_net_bal, ntd_net, 1)
        df_list_ntd.append(net_well)
    net_clp = pd.concat(df_list_ntd)
    # net_clp

    # Cleaning NET_clp from 1-point zero
    print('Cleaning NET_clp from 1-point zero')
    for i in tqdm(range(len(net_clp.NET_clp)-1)):
        if (net_clp.NET_clp.iloc[i] == 0 and  
            net_clp.NET_clp.iloc[i-1] == 1 and 
            net_clp.NET_clp.iloc[i+1] == 1):
            net_clp.NET_clp.iloc[i] = 1        
    df_zero_bal = net_clp[['well', 'MD', 'TST', 'NET_clp', 'FORMATION_up']]
    # df_zero_bal

    print('NET-zero layers searching')
    def ntd_calculation_zero(dataset,well,formation, net_var):
        data = dataset[(dataset.well==well)]
        data.iloc[0, 3] = 1
        data.iloc[-1, 3] = 1
        tst_zero_top = [data.iloc[i]['TST'].round(3) for i in range(len(data)-1)[1:]
                    if (data.iloc[i][net_var] == 0 and data.iloc[i-1][net_var] == 1)]
        tst_zero_bot = [data.iloc[i]['TST'].round(3) for i in range(len(data)-1)[1:] 
                    if (data.iloc[i][net_var] == 0 and data.iloc[i+1][net_var] == 1)]
        tops_zero = zip(tst_zero_top, tst_zero_bot)
        df_zero_htst = pd.DataFrame(tops_zero, columns=['tst_zero_top', 'tst_zero_bot'])
        df_zero_htst['FORMATION_up'] = formation
        df_zero_htst['well'] = well
        df_zero_htst['h_tst_zero'] = df_zero_htst.tst_zero_bot - df_zero_htst.tst_zero_top
        df_zero_htst = df_zero_htst[['well','FORMATION_up','tst_zero_top','tst_zero_bot','h_tst_zero']]
        return df_zero_htst
    df_zero_list = []
    for well in tqdm(df_zero_bal.well.unique()):
        df = ntd_calculation_zero(df_zero_bal, well, fm, 'NET_clp')
        df_zero_list.append(df)
    ntd_zero = pd.concat(df_zero_list)
    # ntd_zero

    print('NET-zero layers removing')
    def ntd_htst_zero_cleaning(dataset_main, dataset_ntd, cutoff, net_var1, net_var2):
        ntd_well_zero = dataset_ntd[(dataset_ntd.well ==well)]
        ntd_well_zero_sel = ntd_well_zero[ntd_well_zero.h_tst_zero <= cutoff]
        well_zero_short = dataset_main[['well','FORMATION_up','MD','TST', net_var1, 'GR_N', 'NET', 'FORMATION']]
        well_zero_short[net_var2] = well_zero_short[net_var1]
        well_zero_sel = well_zero_short[(well_zero_short.well==well)]
        for j in range(len(ntd_well_zero_sel.well)):
            ntd_zero_top = ntd_well_zero_sel.iloc[j, 2].round(3)
            ntd_zero_bot = ntd_well_zero_sel.iloc[j, 3].round(3)
            for i in range(len(well_zero_sel.TST)):
                well_zero_tst = well_zero_sel['TST'].iloc[i].round(3)
                if well_zero_tst >= ntd_zero_top and well_zero_tst <= ntd_zero_bot:
                    well_zero_sel[net_var2].iloc[i] = 1
        return well_zero_sel
    df_list_ntd_zero = []
    for well in tqdm(ntd_zero.well.unique()):
        well_zero_sel = ntd_htst_zero_cleaning(net_clp, ntd_zero, 1, 'NET_clp', 'NET_clp2')
        df_list_ntd_zero.append(well_zero_sel)
    net_clp2 = pd.concat(df_list_ntd_zero)
    return net_clp2
df_interp_bal8_azr = df_interp[df_interp.field.isin(['WEST AZERI', 'CENTRAL AZERI', 'EAST AZERI'])]
net_clp2 = net_flag_cleaning(df_interp_bal8_azr, phit_clean_bal8.well.unique(), 'Balakhany VIII')

In [17]:
def well_display_net(dataset, well, formation, net1, net2, depth_step, print_flag, path):
    well_sel = dataset[(dataset.well == well) & (dataset.FORMATION_up == formation)]
    depth = well_sel['TST']
    grn = well_sel['GR_N']
    net = well_sel['NET']
    net_clp = well_sel[net1]
    net_clp2 = well_sel[net2]
    
    fig, ax = plt.subplots(1,3, figsize=(6,8), sharey=True)
    ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[0].plot(net, depth, color='orange', lw=0.25)
    ax[0].fill_betweenx(depth,net, color='orange', alpha=0.33)
    ax[0].invert_yaxis()
    ax[0].set_xlim(0, 1) 
    ax[0].set_title('NET')
    well_bal_tops = well_sel.groupby('FORMATION')['TST'].apply(lambda x: x.iloc[0]).reset_index()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains('Balakhany VIII')].FORMATION:
        ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i]['TST'].iloc[0], xmin=0, xmax=150, color='black', lw=2, alpha=0.66)
        ax[0].text(0.1, well_bal_tops[well_bal_tops.FORMATION==i]['TST'].iloc[0]+3, i, fontsize = 7, color ="black")
    twin0 = ax[0].twiny()
    twin0.plot(grn, depth, color='green')
    twin0.set_xlim(0, 150)
    twin0.set_xticks([])
    # ax[1].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[1].plot(net_clp, depth, color='orange', lw=0.25)
    ax[1].set_title('NET_clp')
    ax[1].set_xlim(0, 1)
    ax[1].grid(axis='y')
    ax[1].fill_betweenx(depth, net_clp, color='orange', alpha=0.33)
    twin1 = ax[1].twiny()
    twin1.plot(grn, depth, color='green')
    twin1.set_xlim(0, 150)
    twin1.set_xticks([])
    # ax[2].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[2].plot(net_clp2, depth, color='orange', lw=0.25)
    ax[2].set_title('NET_clp2')
    ax[2].set_xlim(0, 1)
    ax[2].grid(axis='y')
    ax[2].fill_betweenx(depth,net_clp2, color='orange', alpha=0.33)
    twin2 = ax[2].twiny()
    twin2.plot(grn, depth, color='green')
    twin2.set_xlim(0, 150)
    twin2.set_xticks([])
    fig.suptitle(well_sel.well.unique()[0], fontsize=14)
    fig.tight_layout()
    if print_flag == 'print':
        fig.savefig(path + well + '.png')
    else:
        pass
    return fig.show()
# for well in net_clp2.well.unique():
    # well_display_net(net_clp2, well, 'Balakhany VIII', 'NET_clp','NET_clp2', 10, 'dontprint', 'C:\\jupyter\\SPP\plots\\net_clp2_bal8\\')

In [18]:
def phit_bal8_net_corr(test_net_clp2):
    test_net_clp2.loc[(test_net_clp2.well == 'B32')  & (test_net_clp2.TST >= 2346) 
                                                    & (test_net_clp2.TST <= 2349), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C10')  & (test_net_clp2.TST >= 1986) 
                                                    & (test_net_clp2.TST <= 1988), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C10')  & (test_net_clp2.TST >= 1988) 
                                                    & (test_net_clp2.TST <= 1990), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C10')  & (test_net_clp2.TST >= 1990) 
                                                    & (test_net_clp2.TST <= 1995), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C10')  & (test_net_clp2.TST >= 1965) 
                                                    & (test_net_clp2.TST <= 1970), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C13Z')  & (test_net_clp2.TST >= 280) 
                                                    & (test_net_clp2.TST <= 284), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C14Z')  & (test_net_clp2.TST >= 123) 
                                                    & (test_net_clp2.TST <= 128), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C14Z')  & (test_net_clp2.TST >= 133) 
                                                    & (test_net_clp2.TST <= 135), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C14Z')  & (test_net_clp2.TST >= 177) 
                                                    & (test_net_clp2.TST <= 180.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C17')  & (test_net_clp2.TST >= 1363) 
                                                    & (test_net_clp2.TST <= 1366), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C17')  & (test_net_clp2.TST >= 1437) 
                                                    & (test_net_clp2.TST <= 1452), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C17')  & (test_net_clp2.TST >= 1397) 
                                                    & (test_net_clp2.TST <= 1399), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C18')  & (test_net_clp2.TST >= 1543) 
                                                    & (test_net_clp2.TST <= 1545), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C18')  & (test_net_clp2.TST >= 1620) 
                                                    & (test_net_clp2.TST <= 1625), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C18')  & (test_net_clp2.TST >= 1555) 
                                                    & (test_net_clp2.TST <= 1557), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C18')  & (test_net_clp2.TST >= 1593.5) 
                                                    & (test_net_clp2.TST <= 1597.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C21')  & (test_net_clp2.TST >= 1440) 
                                                    & (test_net_clp2.TST <= 1450), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C21')  & (test_net_clp2.TST >= 1525.5) 
                                                    & (test_net_clp2.TST <= 1528), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C21')  & (test_net_clp2.TST >= 1528) 
                                                    & (test_net_clp2.TST <= 1531.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C31')  & (test_net_clp2.TST >= 1937) 
                                                    & (test_net_clp2.TST <= 1939), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C31')  & (test_net_clp2.TST >= 2023) 
                                                    & (test_net_clp2.TST <= 2031.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C42')  & (test_net_clp2.TST >= 2050) 
                                                    & (test_net_clp2.TST <= 2056), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C42')  & (test_net_clp2.TST >= 2109) 
                                                    & (test_net_clp2.TST <= 2111), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C42')  & (test_net_clp2.TST >= 2136) 
                                                    & (test_net_clp2.TST <= 2141), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D05ST1') & (test_net_clp2.TST >= 2025) 
                                                    & (test_net_clp2.TST <= 2031), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D09Z') & (test_net_clp2.TST >= 849) 
                                                    & (test_net_clp2.TST <= 855), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D09Z') & (test_net_clp2.TST >= 837.5) 
                                                    & (test_net_clp2.TST <= 843), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D13Y') & (test_net_clp2.TST >= 295.5) 
                                                    & (test_net_clp2.TST <= 298), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D13Y') & (test_net_clp2.TST >= 246.5) 
                                                    & (test_net_clp2.TST <= 247.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D15') & (test_net_clp2.TST >= 3110) 
                                                    & (test_net_clp2.TST <= 3113), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D23X') & (test_net_clp2.TST >= 456) 
                                                    & (test_net_clp2.TST <= 458), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D24')  & (test_net_clp2.TST >= 2900) 
                                                    & (test_net_clp2.TST <= 2902), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D24')  & (test_net_clp2.TST >= 2918) 
                                                    & (test_net_clp2.TST <= 2922), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D24')  & (test_net_clp2.TST >= 2945) 
                                                    & (test_net_clp2.TST <= 2947), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D24')  & (test_net_clp2.TST >= 2948) 
                                                    & (test_net_clp2.TST <= 2951), 'NET_clp2'] = 1
    return test_net_clp2
net_clp2 = phit_bal8_net_corr(net_clp2)

In [19]:
def phit_bal8_net_corr(test_net_clp2):
    test_net_clp2.loc[(test_net_clp2.well == 'B02')  & (test_net_clp2.TST >= 1920) 
                                                    & (test_net_clp2.TST <= 1930), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B02')  & (test_net_clp2.TST >= 1960) 
                                                    & (test_net_clp2.TST <= 1965), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B02Z')  & (test_net_clp2.TST >= 357.5) 
                                                    & (test_net_clp2.TST <= 363.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B02Z')  & (test_net_clp2.TST >= 372.5) 
                                                    & (test_net_clp2.TST <= 375.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B02Z')  & (test_net_clp2.TST >= 441.8) 
                                                    & (test_net_clp2.TST <= 445.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B05')  & (test_net_clp2.TST >= 1930) 
                                                    & (test_net_clp2.TST <= 1932.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B07')  & (test_net_clp2.TST >= 1850.5) 
                                                    & (test_net_clp2.TST <= 1852.3), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B07')  & (test_net_clp2.TST >= 1893) 
                                                    & (test_net_clp2.TST <= 1898), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B08')  & (test_net_clp2.TST >= 1950) 
                                                    & (test_net_clp2.TST <= 1975), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B08')  & (test_net_clp2.TST >= 1959.4) 
                                                    & (test_net_clp2.TST <= 1963.7), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B08Z')  & (test_net_clp2.TST >= 2754.8) 
                                                    & (test_net_clp2.TST <= 2758.6), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B10')  & (test_net_clp2.TST >= 1816.5) 
                                                    & (test_net_clp2.TST <= 1823), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B10')  & (test_net_clp2.TST >= 1827.5) 
                                                    & (test_net_clp2.TST <= 1831), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B10')  & (test_net_clp2.TST >= 1892.7) 
                                                    & (test_net_clp2.TST <= 1900), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B10')  & (test_net_clp2.TST >= 1910.8) 
                                                    & (test_net_clp2.TST <= 1913.2), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B11')  & (test_net_clp2.TST >= 2015.8) 
                                                    & (test_net_clp2.TST <= 2022.2), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B16Z')  & (test_net_clp2.TST >= 1920.5) 
                                                    & (test_net_clp2.TST <= 1925), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B20')  & (test_net_clp2.TST >= 1963) 
                                                    & (test_net_clp2.TST <= 1964.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B22')  & (test_net_clp2.TST >= 2045) 
                                                    & (test_net_clp2.TST <= 2057), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B22')  & (test_net_clp2.TST >= 2038.6) 
                                                    & (test_net_clp2.TST <= 2042.8), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B22')  & (test_net_clp2.TST >= 2047.2) 
                                                    & (test_net_clp2.TST <= 2050.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B23Z')  & (test_net_clp2.TST >= 1961) 
                                                    & (test_net_clp2.TST <= 1969), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B26')  & (test_net_clp2.TST >= 2110.2) 
                                                    & (test_net_clp2.TST <= 2111.2), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B26')  & (test_net_clp2.TST >= 2113.5) 
                                                    & (test_net_clp2.TST <= 2115.7), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B27Z')  & (test_net_clp2.TST >= 118.5) 
                                                    & (test_net_clp2.TST <= 123.7), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B27Z')  & (test_net_clp2.TST >= 172.5) 
                                                    & (test_net_clp2.TST <= 187.7), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B29')  & (test_net_clp2.TST >= 2231) 
                                                    & (test_net_clp2.TST <= 2235.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B30')  & (test_net_clp2.TST >= 1836.7) 
                                                    & (test_net_clp2.TST <= 1843.1), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B31')  & (test_net_clp2.TST >= 1924) 
                                                    & (test_net_clp2.TST <= 1934), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B31')  & (test_net_clp2.TST >= 1961.5) 
                                                    & (test_net_clp2.TST <= 1966.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B34')  & (test_net_clp2.TST >= 2285.5) 
                                                    & (test_net_clp2.TST <= 1966.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B34Z')  & (test_net_clp2.TST >= 675.9) 
                                                    & (test_net_clp2.TST <= 683.2), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B35')  & (test_net_clp2.TST >= 2377) 
                                                    & (test_net_clp2.TST <= 2382), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B37')  & (test_net_clp2.TST >= 118.3) 
                                                    & (test_net_clp2.TST <= 121), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B37')  & (test_net_clp2.TST >= 155.5) 
                                                    & (test_net_clp2.TST <= 157), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B39')  & (test_net_clp2.TST >= 803.7) 
                                                    & (test_net_clp2.TST <= 810), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B40')  & (test_net_clp2.TST >= 164.4) 
                                                    & (test_net_clp2.TST <= 167.3), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C01')  & (test_net_clp2.TST >= 2016.1) 
                                                    & (test_net_clp2.TST <= 2017.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C02')  & (test_net_clp2.TST >= 1914.3) 
                                                    & (test_net_clp2.TST <= 1918.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C02')  & (test_net_clp2.TST >= 1914.3) 
                                                    & (test_net_clp2.TST <= 1918.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C02')  & (test_net_clp2.TST >= 1919.4) 
                                                    & (test_net_clp2.TST <= 1920.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C06')  & (test_net_clp2.TST >= 2049) 
                                                    & (test_net_clp2.TST <= 2056), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C07')  & (test_net_clp2.TST >= 2035.3) 
                                                    & (test_net_clp2.TST <= 2041.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C10')  & (test_net_clp2.TST >= 2021) 
                                                    & (test_net_clp2.TST <= 2022), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C10')  & (test_net_clp2.TST >= 2078.4) 
                                                    & (test_net_clp2.TST <= 2080), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C11')  & (test_net_clp2.TST >= 2081) 
                                                    & (test_net_clp2.TST <= 2097), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C11')  & (test_net_clp2.TST >= 2090) 
                                                    & (test_net_clp2.TST <= 2091), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C12')  & (test_net_clp2.TST >= 1876.8) 
                                                    & (test_net_clp2.TST <= 1877.8), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C12')  & (test_net_clp2.TST >= 1890) 
                                                    & (test_net_clp2.TST <= 1891.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C12')  & (test_net_clp2.TST >= 1899.3) 
                                                    & (test_net_clp2.TST <= 1901.7), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C13Z')  & (test_net_clp2.TST >= 278.3) 
                                                    & (test_net_clp2.TST <= 285.7), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C17')  & (test_net_clp2.TST >= 1369.3) 
                                                    & (test_net_clp2.TST <= 1372.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C17')  & (test_net_clp2.TST >= 1385.5) 
                                                    & (test_net_clp2.TST <= 1405), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C17')  & (test_net_clp2.TST >= 1435) 
                                                    & (test_net_clp2.TST <= 1440.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C20Z')  & (test_net_clp2.TST >= 1578) 
                                                    & (test_net_clp2.TST <= 1582.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C21')  & (test_net_clp2.TST >= 1529) 
                                                    & (test_net_clp2.TST <= 1535), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C21Z')  & (test_net_clp2.TST >= 312) 
                                                    & (test_net_clp2.TST <= 323), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C21Z')  & (test_net_clp2.TST >= 367) 
                                                    & (test_net_clp2.TST <= 371.1), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C29Y')  & (test_net_clp2.TST >= 441.7) 
                                                    & (test_net_clp2.TST <= 442.7), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C29Y')  & (test_net_clp2.TST >= 456.3) 
                                                    & (test_net_clp2.TST <= 460), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C31')  & (test_net_clp2.TST >= 1991) 
                                                    & (test_net_clp2.TST <= 1997), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C33Z')  & (test_net_clp2.TST >= 189.5) 
                                                    & (test_net_clp2.TST <= 192.9), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C34')  & (test_net_clp2.TST >= 2053.1) 
                                                    & (test_net_clp2.TST <= 2058.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C40')  & (test_net_clp2.TST >= 1891.2) 
                                                    & (test_net_clp2.TST <= 1894.7), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C40')  & (test_net_clp2.TST >= 1935.2) 
                                                    & (test_net_clp2.TST <= 1938.7), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C41')  & (test_net_clp2.TST >= 2218.2) 
                                                    & (test_net_clp2.TST <= 2222.7), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C41')  & (test_net_clp2.TST >= 2252.1) 
                                                    & (test_net_clp2.TST <= 2253.4), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C41')  & (test_net_clp2.TST >= 2288) 
                                                    & (test_net_clp2.TST <= 2292), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C42')  & (test_net_clp2.TST >= 2133) 
                                                    & (test_net_clp2.TST <= 2144), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C42')  & (test_net_clp2.TST >= 2180) 
                                                    & (test_net_clp2.TST <= 2190), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D02Z')  & (test_net_clp2.TST >= 274.4) 
                                                    & (test_net_clp2.TST <= 277), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D03Z')  & (test_net_clp2.TST >= 295) 
                                                    & (test_net_clp2.TST <= 301), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D03Z')  & (test_net_clp2.TST >= 343) 
                                                    & (test_net_clp2.TST <= 352), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D04Z')  & (test_net_clp2.TST >= 755) 
                                                    & (test_net_clp2.TST <= 760.7), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D04Z')  & (test_net_clp2.TST >= 768.5) 
                                                    & (test_net_clp2.TST <= 771.7), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D07')  & (test_net_clp2.TST >= 2962) 
                                                    & (test_net_clp2.TST <= 2966), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D12Z')  & (test_net_clp2.TST >= 2120) 
                                                    & (test_net_clp2.TST <= 2130), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D12Z')  & (test_net_clp2.TST >= 2219.4) 
                                                    & (test_net_clp2.TST <= 2224.7), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D13')  & (test_net_clp2.TST >= 3063) 
                                                    & (test_net_clp2.TST <= 3070), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D13Y')  & (test_net_clp2.TST >= 231) 
                                                    & (test_net_clp2.TST <= 250), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D13Y')  & (test_net_clp2.TST >= 290) 
                                                    & (test_net_clp2.TST <= 301.8), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D13Y')  & (test_net_clp2.TST >= 303.7) 
                                                    & (test_net_clp2.TST <= 311.8), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D15')  & (test_net_clp2.TST >= 3118) 
                                                    & (test_net_clp2.TST <= 3127.8), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D24')  & (test_net_clp2.TST >= 2942.5) 
                                                    & (test_net_clp2.TST <= 2953.8), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D25')  & (test_net_clp2.TST >= 2942.5) 
                                                    & (test_net_clp2.TST <= 2953.8), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D27')  & (test_net_clp2.TST >= 2158) 
                                                    & (test_net_clp2.TST <= 2163), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D27')  & (test_net_clp2.TST >= 2209) 
                                                    & (test_net_clp2.TST <= 2213.3), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D33')  & (test_net_clp2.TST >= 2986.6) 
                                                    & (test_net_clp2.TST <= 2989.9), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D36')  & (test_net_clp2.TST >= 2882.4) 
                                                    & (test_net_clp2.TST <= 2884.7), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D36')  & (test_net_clp2.TST >= 2894.7) 
                                                    & (test_net_clp2.TST <= 2901.1), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D37')  & (test_net_clp2.TST >= 2915) 
                                                    & (test_net_clp2.TST <= 2920), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D38')  & (test_net_clp2.TST >= 2441) 
                                                    & (test_net_clp2.TST <= 2454), 'NET_clp2'] = 1
    return test_net_clp2
net_clp2 = phit_bal8_net_corr(net_clp2)

In [20]:
def well_net_corr_display(dataset, formation, well):
    well_sel = dataset[(dataset.well == well) & (dataset.FORMATION_up == formation)]
    depth = well_sel['TST']
    grn = well_sel['GR_N']
    net = well_sel['NET_clp2']*100
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=grn, y=depth, mode='lines', name='GR', marker=dict(color='green')))
    fig.update_xaxes(range = [0,150])
    fig.update_yaxes(autorange="reversed")
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  marker=dict(color='rgba(245, 190, 20, 0.75)'), fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'))
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(height=700, width=250, title_text=well, showlegend=False,
                    margin=dict(l=10,r=10,b=10,t=50))
    fig.show()
# well_net_corr_display(net_clp2, 'Balakhany VIII', 'D37')

In [21]:
df_bal8_azr_clp2 = df_interp_bal8_azr.set_index(['well','TST']).join(
                   net_clp2.drop(['FORMATION_up','MD','GR_N','NET','FORMATION'], axis=1).set_index(['well','TST'])).reset_index()

### netvsh_clp2 bal10

In [ ]:
def netvsh_calc_by_gr10(df_interp):
    df_interp.loc[(df_interp.well == 'A01ST1')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                        & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'A02')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 50), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'A06X')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'A09')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 70), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'A12')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 70), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'A12V')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 70), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'A12W')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'A12X')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 50), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'B05ST1')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'B20')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 65), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'B28')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'C11')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 55), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'C25Z')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'C31')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 50), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'D01Y')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'D26')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 70), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'E08')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 65), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'E19')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 65), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'E23')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 65), 'NET'] = 1 
    df_interp.loc[(df_interp.well == 'F02')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 65), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'J03')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 60), 'NET'] = 1
    df_interp.loc[(df_interp.well == 'J15')  & (df_interp.FORMATION_up == 'Balakhany X') 
                                            & (df_interp.GR_N <= 50), 'NET'] = 1
    return df_interp
df_interp = netvsh_calc_by_gr10(df_interp)    

def vsh_calc_by_gr10(df_interp): 
    gr10_well_list = ['A01ST1', 'A02', 'A02W', 'A02X', 'A02Y', 'A03', 'A06', 'A08X',
                    'A08Y', 'A10Z', 'A11', 'A13', 'A14U', 'A14Z', 'A15', 'A15ST1',
                    'A16ST1', 'A16ST3', 'A16W', 'A19Y', 'AZERI1', 'B04Z', 'B05ST1',
                    'B10Y', 'B10Z', 'B12Z', 'B16W', 'B17', 'B18', 'B18X', 'B22Z',
                    'B25', 'B27', 'B28', 'B33', 'C03', 'C07Z', 'C08', 'C11Z', 'C13',
                    'C15Z', 'C19', 'C22', 'C23', 'C24', 'C25Z', 'C29', 'C30', 'C39',
                    'D05Y', 'D11Y', 'D14Z', 'D16', 'D17Z', 'D18', 'D19', 'D20X',
                    'D20Z', 'D22', 'D23Y', 'D23Z', 'D28', 'E03', 'E04Z', 'E09Y',
                    'E10X', 'E11Y', 'E12', 'E13', 'E15Z', 'E17', 'E18', 'E20', 'E20Z',
                    'E22', 'E38Z', 'F03', 'F04', 'F04Z', 'F05', 'F06', 'G02Z', 'G03',
                    'G04', 'H02', 'H03', 'H04', 'H05', 'H06', 'H07Z', 'H08Z', 'J02Y',
                    'J02Z', 'J07', 'J11', 'J13Y', 'J13Z', 'J14', 'J21']
    df_interp.loc[(df_interp.well.isin(gr10_well_list))  & (df_interp.FORMATION_up == 'Balakhany X') 
                                                         & (df_interp.VSH <= 0.5), 'NET'] = 1
    return df_interp
df_interp = vsh_calc_by_gr10(df_interp)

def well_gr_corr_display(dataset, formation, well):
    well_sel = dataset[(dataset.well == well) & (dataset.FORMATION_up == formation)]
    depth = well_sel['TST']
    grn = well_sel['GR_N']
    net = well_sel['NET']*100
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=grn, y=depth, mode='lines', name='GR', marker=dict(color='green')))
    fig.update_xaxes(range=[0, 150])
    fig.update_yaxes(autorange="reversed")
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  marker=dict(color='rgba(245, 190, 20, 0.75)'), fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'))
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(height=700, width=250, title_text=well, showlegend=False,
                    margin=dict(l=10,r=10,b=10,t=40))
    fig.show()
# well_gr_corr_display(df_interp, 'Balakhany X', 'J11')

In [ ]:
def net_flag_cleaning(dataset_main, fm):
    df_net_bal = dataset_main

    print('Calculation dataframe with h_tst from MD to TST')
    def ntd_calculation_brief(dataset,well, desired_fm, net_var):
        data = dataset[(dataset.well==well)]
        data['TST'].iloc[0] = 0
        data['TST'].iloc[-1] = 0
        tst_top = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i-1][net_var]==0)]
        tst_bot = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i+1][net_var]==0)]
        tops = zip(tst_top, tst_bot)
        df_htst = pd.DataFrame(tops, columns=['tst_top', 'tst_bot'])
        df_htst['FORMATION_up'] = desired_fm
        df_htst['well'] = well
        df_htst['h_tst'] = df_htst.tst_bot - df_htst.tst_top
        df_htst = df_htst[['well','FORMATION_up','tst_top','tst_bot','h_tst']]
        return df_htst
    df_list = []
    for well in tqdm(df_net_bal.well.unique()):
        df = ntd_calculation_brief(df_net_bal, well, fm, 'NET')
        df_list.append(df)
    ntd_net = pd.concat(df_list)
    # ntd_net

    # Cleaning NET variable and making up NET_clp with clipped data, join NET_clp to main dataframe
    print('Cleaning NET variable and making up NET_clp with clipped data')
    def ntd_htst_cleaning(dataset_main, dataset_ntd, cutoff):
        ntd_well = dataset_ntd[(dataset_ntd.well ==well)]
        ntd_well_cutoff = ntd_well[ntd_well.h_tst >= cutoff]
        well_short = dataset_main[['well', 'FORMATION', 'FORMATION_up', 'MD', 'TST', 'GR_N', 'NET', ]]
        net_well = well_short[(well_short.well==well)]
        net_well['NET_clp'] = 0
        for j in range(len(ntd_well_cutoff.well)):
            ntd_top = ntd_well_cutoff.iloc[j, 2].round(3)
            ntd_bot = ntd_well_cutoff.iloc[j, 3].round(3)
            for i in range(len(net_well.TST)):
                well_tst = net_well['TST'].iloc[i].round(3)
                if well_tst >= ntd_top and well_tst <= ntd_bot:
                    net_well['NET_clp'].iloc[i] = 1
        return net_well
    df_list_ntd = []
    for well in tqdm(ntd_net.well.unique()):
        net_well = ntd_htst_cleaning(df_net_bal, ntd_net, 1)
        df_list_ntd.append(net_well)
    net_clp = pd.concat(df_list_ntd)
    # net_clp

    # Cleaning NET_clp from 1-point zero
    print('Cleaning NET_clp from 1-point zero')
    for i in tqdm(range(len(net_clp.NET_clp)-1)):
        if (net_clp.NET_clp.iloc[i] == 0 and  
            net_clp.NET_clp.iloc[i-1] == 1 and 
            net_clp.NET_clp.iloc[i+1] == 1):
            net_clp.NET_clp.iloc[i] = 1        
    df_zero_bal = net_clp[['well', 'MD', 'TST', 'NET_clp', 'FORMATION_up']]
    # df_zero_bal

    print('NET-zero layers searching')
    def ntd_calculation_zero(dataset,well,formation, net_var):
        data = dataset[(dataset.well==well)]
        data.iloc[0, 3] = 1
        data.iloc[-1, 3] = 1
        tst_zero_top = [data.iloc[i]['TST'].round(3) for i in range(len(data)-1)[1:]
                    if (data.iloc[i][net_var] == 0 and data.iloc[i-1][net_var] == 1)]
        tst_zero_bot = [data.iloc[i]['TST'].round(3) for i in range(len(data)-1)[1:] 
                    if (data.iloc[i][net_var] == 0 and data.iloc[i+1][net_var] == 1)]
        tops_zero = zip(tst_zero_top, tst_zero_bot)
        df_zero_htst = pd.DataFrame(tops_zero, columns=['tst_zero_top', 'tst_zero_bot'])
        df_zero_htst['FORMATION_up'] = formation
        df_zero_htst['well'] = well
        df_zero_htst['h_tst_zero'] = df_zero_htst.tst_zero_bot - df_zero_htst.tst_zero_top
        df_zero_htst = df_zero_htst[['well','FORMATION_up','tst_zero_top','tst_zero_bot','h_tst_zero']]
        return df_zero_htst
    df_zero_list = []
    for well in tqdm(df_zero_bal.well.unique()):
        df = ntd_calculation_zero(df_zero_bal, well, fm, 'NET_clp')
        df_zero_list.append(df)
    ntd_zero = pd.concat(df_zero_list)
    # ntd_zero

    print('NET-zero layers removing')
    def ntd_htst_zero_cleaning(dataset_main, dataset_ntd, cutoff, net_var1, net_var2):
        ntd_well_zero = dataset_ntd[(dataset_ntd.well ==well)]
        ntd_well_zero_sel = ntd_well_zero[ntd_well_zero.h_tst_zero <= cutoff]
        well_zero_short = dataset_main[['well','FORMATION_up','MD','TST', net_var1, 'GR_N', 'NET', 'FORMATION']]
        well_zero_short[net_var2] = well_zero_short[net_var1]
        well_zero_sel = well_zero_short[(well_zero_short.well==well)]
        for j in range(len(ntd_well_zero_sel.well)):
            ntd_zero_top = ntd_well_zero_sel.iloc[j, 2].round(3)
            ntd_zero_bot = ntd_well_zero_sel.iloc[j, 3].round(3)
            for i in range(len(well_zero_sel.TST)):
                well_zero_tst = well_zero_sel['TST'].iloc[i].round(3)
                if well_zero_tst >= ntd_zero_top and well_zero_tst <= ntd_zero_bot:
                    well_zero_sel[net_var2].iloc[i] = 1
        return well_zero_sel
    df_list_ntd_zero = []
    for well in tqdm(ntd_zero.well.unique()):
        well_zero_sel = ntd_htst_zero_cleaning(net_clp, ntd_zero, 0.75, 'NET_clp', 'NET_clp2')
        df_list_ntd_zero.append(well_zero_sel)
    net_clp2 = pd.concat(df_list_ntd_zero)
    return net_clp2
black_list_wells10 = ['AZERI4', 'AZERI5', 'CHIRAG1', 'CHIRAG4', 'CHIRAG5', 'CHIRAG6', 'E05Z', 'E13Y', 'E21A',
                    'GCA1','GCA2', 'GCA4', 'GCA4Z', 'GCA5', 'GCA5Z', 'GCA6', 'GCA6Y', 'GCA6Z','GCA7']
df_bal10_vsh = df_interp[(df_interp.well.isin(vsh_clean_bal10.well.unique()))
                            & (~df_interp.well.isin(black_list_wells10)) 
                            & (df_interp.FORMATION_up == 'Balakhany X')]
netvsh_clp2_bal10 = net_flag_cleaning(df_bal10_vsh, 'Balakhany X')

def vsh_bal10_net_corr(test_net_clp2):
    test_net_clp2 = netvsh_clp2_bal10.copy()
    test_net_clp2.loc[(test_net_clp2.well == 'A03Y')  & (test_net_clp2.TST >= 1274.5) 
                                                    & (test_net_clp2.TST <= 1279), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A06X')  & (test_net_clp2.TST >= 485.5) 
                                                    & (test_net_clp2.TST <= 490), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A06X')  & (test_net_clp2.TST >= 521.5) 
                                                    & (test_net_clp2.TST <= 526.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A06X')  & (test_net_clp2.TST >= 531) 
                                                    & (test_net_clp2.TST <= 537), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A07')  & (test_net_clp2.TST >= 638) 
                                                    & (test_net_clp2.TST <= 642), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A07')  & (test_net_clp2.TST >= 645) 
                                                    & (test_net_clp2.TST <= 649), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A08')  & (test_net_clp2.TST >= 571.8) 
                                                    & (test_net_clp2.TST <= 573.3), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A08')  & (test_net_clp2.TST >= 602) 
                                                    & (test_net_clp2.TST <= 607.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A08')  & (test_net_clp2.TST >= 624) 
                                                    & (test_net_clp2.TST <= 631.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A08Z')  & (test_net_clp2.TST >= 962.6) 
                                                    & (test_net_clp2.TST <= 966.3), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A12X')  & (test_net_clp2.TST >= 170) 
                                                    & (test_net_clp2.TST <= 175.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A12X')  & (test_net_clp2.TST >= 182.5) 
                                                    & (test_net_clp2.TST <= 187.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A12YST1')  & (test_net_clp2.TST >= 1018.4) 
                                                    & (test_net_clp2.TST <= 1030.4), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A12YST1')  & (test_net_clp2.TST >= 1043.5) 
                                                    & (test_net_clp2.TST <= 1053.3), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A12YST1')  & (test_net_clp2.TST >= 1057.1) 
                                                    & (test_net_clp2.TST <= 1059.2), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A20')  & (test_net_clp2.TST >= 550) 
                                                    & (test_net_clp2.TST <= 565), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'A20')  & (test_net_clp2.TST >= 677.2) 
                                                    & (test_net_clp2.TST <= 683.1), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'A20')  & (test_net_clp2.TST >= 625) 
                                                    & (test_net_clp2.TST <= 626), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'A20')  & (test_net_clp2.TST >= 642.8) 
                                                    & (test_net_clp2.TST <= 644), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'A20Y')  & (test_net_clp2.TST >= 55.6) 
                                                    & (test_net_clp2.TST <= 58.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B07')  & (test_net_clp2.TST >= 2161.6) 
                                                    & (test_net_clp2.TST <= 2163.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B07')  & (test_net_clp2.TST >= 2167) 
                                                    & (test_net_clp2.TST <= 2174), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B11')  & (test_net_clp2.TST >= 2211) 
                                                    & (test_net_clp2.TST <= 2220), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B11')  & (test_net_clp2.TST >= 2230) 
                                                    & (test_net_clp2.TST <= 2241.6), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B28')  & (test_net_clp2.TST >= 2295.5) 
                                                & (test_net_clp2.TST <= 2301.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'B32')  & (test_net_clp2.TST >= 2540) 
                                                    & (test_net_clp2.TST <= 2550), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B34')  & (test_net_clp2.TST >= 2477) 
                                                    & (test_net_clp2.TST <= 2483), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'B34')  & (test_net_clp2.TST >= 2548) 
                                                    & (test_net_clp2.TST <= 2557), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C02')  & (test_net_clp2.TST >= 2163.7) 
                                                    & (test_net_clp2.TST <= 2166.8), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C03Z')  & (test_net_clp2.TST >= 993) 
                                                    & (test_net_clp2.TST <= 1001), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'C03Z')  & (test_net_clp2.TST >= 1002) 
                                                    & (test_net_clp2.TST <= 1008.3), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C03Z')  & (test_net_clp2.TST >= 1060.4) 
                                                    & (test_net_clp2.TST <= 1064), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C11')  & (test_net_clp2.TST >= 2327.4) 
                                                    & (test_net_clp2.TST <= 2330.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C11')  & (test_net_clp2.TST >= 2356) 
                                                    & (test_net_clp2.TST <= 2360), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C11')  & (test_net_clp2.TST >= 2394) 
                                                    & (test_net_clp2.TST <= 2399), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C11')  & (test_net_clp2.TST >= 2400.5) 
                                                    & (test_net_clp2.TST <= 2402), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C25Z')  & (test_net_clp2.TST >= 624.5) 
                                                    & (test_net_clp2.TST <= 632.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C25Z')  & (test_net_clp2.TST >= 696.5) 
                                                    & (test_net_clp2.TST <= 701.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C31')  & (test_net_clp2.TST >= 2235.5) 
                                                    & (test_net_clp2.TST <= 2249), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'C32')  & (test_net_clp2.TST >= 2082) 
                                                    & (test_net_clp2.TST <= 2086), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D02Z')  & (test_net_clp2.TST >= 440) 
                                                    & (test_net_clp2.TST <= 495), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D02Z')  & (test_net_clp2.TST >= 504.8) 
                                                    & (test_net_clp2.TST <= 506.6), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D02Z')  & (test_net_clp2.TST >= 523) 
                                                    & (test_net_clp2.TST <= 536), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D03')  & (test_net_clp2.TST >= 3203) 
                                                    & (test_net_clp2.TST <= 3223), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D04')  & (test_net_clp2.TST >= 3161) 
                                                    & (test_net_clp2.TST <= 3210), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D04')  & (test_net_clp2.TST >= 3259) 
                                                    & (test_net_clp2.TST <= 3281), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D06')  & (test_net_clp2.TST >= 3105) 
                                                    & (test_net_clp2.TST <= 3147), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D06')  & (test_net_clp2.TST >= 3156) 
                                                    & (test_net_clp2.TST <= 3165), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D06')  & (test_net_clp2.TST >= 3236) 
                                                    & (test_net_clp2.TST <= 3245), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D06')  & (test_net_clp2.TST >= 3226) 
                                                    & (test_net_clp2.TST <= 3235), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D09')  & (test_net_clp2.TST >= 3359) 
                                                    & (test_net_clp2.TST <= 3362.8), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'D15')  & (test_net_clp2.TST >= 3320) 
                                                    & (test_net_clp2.TST <= 3392), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D15')  & (test_net_clp2.TST >= 3425) 
                                                    & (test_net_clp2.TST <= 3447), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D20')  & (test_net_clp2.TST >= 2966) 
                                                    & (test_net_clp2.TST <= 3011), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D20')  & (test_net_clp2.TST >= 3055) 
                                                    & (test_net_clp2.TST <= 3097), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D24')  & (test_net_clp2.TST >= 3080) 
                                                    & (test_net_clp2.TST <= 3127), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D24')  & (test_net_clp2.TST >= 3157) 
                                                    & (test_net_clp2.TST <= 3175), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'D26')  & (test_net_clp2.TST >= 2877.1) 
                                                    & (test_net_clp2.TST <= 2880.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'E01Z')  & (test_net_clp2.TST >= 66.8) 
                                                    & (test_net_clp2.TST <= 69.3), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'E04')  & (test_net_clp2.TST >= 0) 
                                                    & (test_net_clp2.TST <= 5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'E04Y')  & (test_net_clp2.TST >= 0) 
                                                    & (test_net_clp2.TST <= 15.5), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'E04Y')  & (test_net_clp2.TST >= -8) 
                                                    & (test_net_clp2.TST <= 0), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'E08')  & (test_net_clp2.TST >= 31) 
                                                    & (test_net_clp2.TST <= 35.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'E10W')  & (test_net_clp2.TST >= 2800) 
                                                    & (test_net_clp2.TST <= 2810), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'E10W')  & (test_net_clp2.TST >= 2839) 
                                                    & (test_net_clp2.TST <= 2841), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'E10W')  & (test_net_clp2.TST >= 2843.3) 
                                                    & (test_net_clp2.TST <= 2844.4), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'E10W')  & (test_net_clp2.TST >= 2848.2) 
                                                    & (test_net_clp2.TST <= 2849), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'E11Z')  & (test_net_clp2.TST >= 24.8) 
                                                    & (test_net_clp2.TST <= 28.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'E16X')  & (test_net_clp2.TST >= 713) 
                                                    & (test_net_clp2.TST <= 724), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'E16X')  & (test_net_clp2.TST >= 740) 
                                                    & (test_net_clp2.TST <= 747), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'E16X')  & (test_net_clp2.TST >= 749.5) 
                                                    & (test_net_clp2.TST <= 756.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'E19')  & (test_net_clp2.TST >= 2140) 
                                                    & (test_net_clp2.TST <= 2150), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'F01Z')  & (test_net_clp2.TST >= -118) 
                                                    & (test_net_clp2.TST <= -94), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'F01Z')  & (test_net_clp2.TST >= -39) 
                                                    & (test_net_clp2.TST <= -36), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'F01Z')  & (test_net_clp2.TST >= -35.7) 
                                                    & (test_net_clp2.TST <= -32.8), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'G02')  & (test_net_clp2.TST >= 2530) 
                                                    & (test_net_clp2.TST <= 2540), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'G05')  & (test_net_clp2.TST >= 1933.3) 
                                                    & (test_net_clp2.TST <= 1940.6), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'H01Z')  & (test_net_clp2.TST >= 445) 
                                                    & (test_net_clp2.TST <= 446), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'H01Z')  & (test_net_clp2.TST >= 458.5) 
                                                    & (test_net_clp2.TST <= 461.1), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'J03')  & (test_net_clp2.TST >= 2430) 
                                                    & (test_net_clp2.TST <= 2432), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'J13')  & (test_net_clp2.TST >= 2295) 
                                                    & (test_net_clp2.TST <= 2298), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'J13')  & (test_net_clp2.TST >= 2334) 
                                                    & (test_net_clp2.TST <= 2337), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'J15')  & (test_net_clp2.TST >= 2491) 
                                                    & (test_net_clp2.TST <= 2496), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'J15')  & (test_net_clp2.TST >= 2531.5) 
                                                    & (test_net_clp2.TST <= 2533.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'J18A')  & (test_net_clp2.TST >= 1825.5) 
                                                    & (test_net_clp2.TST <= 1830.5), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'J30')  & (test_net_clp2.TST >= 1759) 
                                                    & (test_net_clp2.TST <= 1763), 'NET_clp2'] = 0
    test_net_clp2.loc[(test_net_clp2.well == 'J30')  & (test_net_clp2.TST >= 1775) 
                                                    & (test_net_clp2.TST <= 1780), 'NET_clp2'] = 1
    test_net_clp2.loc[(test_net_clp2.well == 'J32')  & (test_net_clp2.TST >= 1803) 
                                                    & (test_net_clp2.TST <= 1805), 'NET_clp2'] = 0
    return test_net_clp2
netvsh_clp2_bal10 = vsh_bal10_net_corr(netvsh_clp2_bal10)

df_bal10_vshclp2 = df_bal10_vsh.set_index(['well','TST']).join(
                   netvsh_clp2_bal10.drop(['FORMATION_up','MD','GR_N','NET','FORMATION'], axis=1).set_index(['well','TST'])).reset_index()
df_bal10_vshclp2.loc[df_bal10_vshclp2.well.isin(phit_clean_bal10.well.unique()), 'phit_flag'] = 1
df_bal10_vshclp2['phit_flag'] = df_bal10_vshclp2['phit_flag'].fillna(0)

def perm_ds_us_calc(dataset):
    def perm_ds_1(x):
        return 10**(39139.46 * pow(x, 5) - 11140.04 * pow(x, 4) + 855.2176 * pow(x, 3) + 7.04505 * pow(x, 2) + 5.750233 * x - 1.997085)
    def perm_us_1(x):
        return 10**(1638286 * pow(x, 5) - 1396883 * pow(x, 4) + 468324.8 * pow(x, 3) - 76974.79 * pow(x, 2) + 6217.262 * x - 198.5042)
    def perm_ds_2(x):
        return 10**(+ 5675.143 * pow(x, 5) - 11106.91 * pow(x, 4) + 8608.366 * pow(x, 3) - 3318.893 * pow(x, 2) + 644.7713 * x - 48.16968)
    def perm_us_2(x):
        return 10**(+ 81.59968 * pow(x, 5) - 275.5442 * pow(x, 4) + 364.9522 * pow(x, 3) - 238.8838 * pow(x, 2) + 79.97139 * x - 7.15)
    df = dataset
    try:
        df.loc[(df.PHIT >= 0.13) & (df.PHIT <=0.2), 'PERM_DS'] = df.PHIT.apply(perm_ds_1)
        df.loc[(df.PHIT >= 0.13) & (df.PHIT <=0.2), 'PERM_US'] = df.PHIT.apply(perm_us_1)
        df.loc[(df.PHIT >= 0.2) & (df.PHIT <=0.4), 'PERM_DS'] = df.PHIT.apply(perm_ds_2)
        df.loc[(df.PHIT >= 0.2) & (df.PHIT <=0.4), 'PERM_US'] = df.PHIT.apply(perm_us_2)
    except:
        df.loc[(df.PHIT >= 0.13) & (df.PHIT <=0.2), 'PERM_DS'] = 0
        df.loc[(df.PHIT >= 0.13) & (df.PHIT <=0.2), 'PERM_US'] = 0
        df.loc[(df.PHIT >= 0.2) & (df.PHIT <=0.4), 'PERM_DS'] = 0
        df.loc[(df.PHIT >= 0.2) & (df.PHIT <=0.4), 'PERM_US'] = 0
    df['PERM_DS'] = df['PERM_DS'].fillna(0)
    df['PERM_US'] = df['PERM_US'].fillna(0)
    return df
df_bal10_vshclp2 = perm_ds_us_calc(df_bal10_vshclp2)

df_bal10_vshclp2.to_csv('C:\jupyter\SPP\inputoutput\df_bal10_vshclp2.csv')

In [ ]:
for wellname in df_bal10_vshclp2.well.unique():
    well_display_logs(df_bal10_vshclp2, wellname, 'Balakhany X', 'NET', 'net:', 'TST', 5, 'print', 'C:\\jupyter\\SPP\\plots\\logs_vsh_bal10\\')

In [105]:
def well_gr_phit_corr_display(dataset, formation, var, well):
    well_sel = dataset[(dataset.well == well) & (dataset.FORMATION_up == formation)]
    depth = well_sel['TST']
    grn = well_sel['GR_N']
    phit = well_sel['PHIT']
    net = well_sel[var]*100
    phit_cutoff = [0.13 for i in range(len(depth))]
    fig = go.Figure()
    fig = make_subplots(rows=1, cols=2, subplot_titles=('GR', 'PHIT'))
    fig.add_trace(go.Scatter(x=grn, y=depth, mode='lines', name='GR', 
                             line=dict(color='green')), row=1, col=1)
    
    fig.update_xaxes(range = [0,150])
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  
                             line=dict(color='rgba(245, 190, 20, 0.75)'), 
                             fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=1)
    fig.update_xaxes(range = [0, 150], row=1, col=1)

    fig.add_trace(go.Scatter(x=phit, y=depth, mode='lines', name='PHIT', 
                            line = dict(color='firebrick', width=2)), row=1, col=2)
    fig.add_trace(go.Scatter(x=phit_cutoff, y=depth, mode='lines', name='PHIT', 
                            line = dict(color='black', width=2, dash='dash')), row=1, col=2)
    fig.update_xaxes(range = [0.35, 0], row=1, col=2)
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  
                             marker=dict(color='rgba(245, 190, 20, 0.75)'), 
                             fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=2)
    
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(height=850, width=400,
                      title=dict(text=well, font=dict(size=24)),
                      showlegend=False,
                      margin=dict(l=10,r=10,b=10,t=50))
    fig.show()
# well_gr_phit_corr_display(df_bal10_vshclp2, 'Balakhany X','NET_clp2', 'E30Y')

In [ ]:
def well_display_net(dataset, well, formation, net1, net2, depth_step, print_flag, path):
    well_sel = dataset[(dataset.well == well) & (dataset.FORMATION_up == formation)]
    depth = well_sel['TST']
    grn = well_sel['GR_N']
    net = well_sel['NET']
    net_clp = well_sel[net1]
    net_clp2 = well_sel[net2]
    
    fig, ax = plt.subplots(1,3, figsize=(6,8), sharey=True)
    ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[0].plot(net, depth, color='orange', lw=0.25)
    ax[0].fill_betweenx(depth,net, color='orange', alpha=0.33)
    ax[0].invert_yaxis()
    ax[0].set_xlim(0, 1) 
    ax[0].set_title('NET')
    well_bal_tops = well_sel.groupby('FORMATION')['TST'].apply(lambda x: x.iloc[0]).reset_index()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains('Balakhany VIII')].FORMATION:
        ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i]['TST'].iloc[0], xmin=0, xmax=150, color='black', lw=2, alpha=0.66)
        ax[0].text(0.1, well_bal_tops[well_bal_tops.FORMATION==i]['TST'].iloc[0]+3, i, fontsize = 7, color ="black")
    twin0 = ax[0].twiny()
    twin0.plot(grn, depth, color='green')
    twin0.set_xlim(0, 150)
    twin0.set_xticks([])
    # ax[1].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[1].plot(net_clp, depth, color='orange', lw=0.25)
    ax[1].set_title('NET_clp')
    ax[1].set_xlim(0, 1)
    ax[1].grid(axis='y')
    ax[1].fill_betweenx(depth, net_clp, color='orange', alpha=0.33)
    twin1 = ax[1].twiny()
    twin1.plot(grn, depth, color='green')
    twin1.set_xlim(0, 150)
    twin1.set_xticks([])
    # ax[2].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[2].plot(net_clp2, depth, color='orange', lw=0.25)
    ax[2].set_title('NET_clp2')
    ax[2].set_xlim(0, 1)
    ax[2].grid(axis='y')
    ax[2].fill_betweenx(depth,net_clp2, color='orange', alpha=0.33)
    twin2 = ax[2].twiny()
    twin2.plot(grn, depth, color='green')
    twin2.set_xlim(0, 150)
    twin2.set_xticks([])
    fig.suptitle(well_sel.well.unique()[0], fontsize=14)
    fig.tight_layout()
    if print_flag == 'print':
        fig.savefig(path + well + '.png')
    else:
        pass
    return fig.show()
# for well in netvsh_clp2_bal10.well.unique():
#     well_display_net(netvsh_clp2_bal10, well, 'Balakhany X', 'NET_clp','NET_clp2', 10, 'print', 'C:\\jupyter\\SPP\plots\\netvsh_clp2_bal10\\')

## NET data uploading

In [30]:
## Разница df_bal8_azr_vshclp2 и df_bal8_azr_vshclp2_flag
# 'B04', 'B04Z', 'B09', 'B12Z', 'B17', 'B18', 'B18X', 'B18Y', 'B22Z',
# 'B25', 'B27', 'B28', 'B29', 'B30', 'B31', 'B33', 'B35', 'B36',
# 'B38Y', 'B40', 'B41', 'C01AY', 'C03', 'C05', 'C07Z', 'C08', 'C08Z',
# 'C11Z', 'C13', 'C15Z', 'C18X', 'C19', 'C22', 'C23', 'C24', 'C25Z',
# 'C27Z', 'C28', 'C29', 'C29Y', 'C30', 'C34Z', 'C37Z', 'C38', 'D01X',
# 'D01Y', 'D03Z', 'D04Z', 'D05Y', 'D05Z', 'D10', 'D14Z', 'D16',
# 'D17', 'D17Z', 'D18', 'D19', 'D20X', 'D20Y', 'D20Z', 'D21', 'D22',
# 'D23Y', 'D23Z', 'D26', 'D28', 'D29', 'D32', 'D33', 'D35', 'D36',
# 'D38'

# B04, B29, B31, B40, C01AY, C08Z, C38, D04Z, D29 - добавить в список хороших поросити скважин т.к. поросити в коллекторе есть
# B30, B35, B36, B38Y, B41, C27Z, C28, C37Z, D01X, D03Z, D21, D32, D33, D35, D36,D38 - оставить в списке вишейл скважин т.к. поросити брак
# B09, B18, D26 - найти поросити

In [ ]:
# for wellname in df_bal8_azr_vshclp2.well.unique():
#     net2 = (df_bal8_azr_vshclp2[df_bal8_azr_vshclp2.well == wellname]['NET_clp2'].sum()*0.1).round(1)
#     tst_int = int(df_bal8_azr_vshclp2[df_bal8_azr_vshclp2.well == wellname]['TST'].iloc[-1] - df_bal8_azr_vshclp2[df_bal8_azr_vshclp2.well == wellname]['TST'].iloc[0])
#     well_display_logs(df_bal8_azr_vshclp2, wellname, 'Balakhany VIII','NET_clp2', f'tst: {tst_int}m net2: {net2}m',  
#                       'TST', 10, 'print', 'C:\jupyter\SPP\plots\logs_vsh_bal8\\')

# for wellname in df_bal10_vshclp2.well.unique():
#     net2 = (df_bal10_vshclp2[df_bal10_vshclp2.well == wellname]['NET_clp2'].sum()*0.1).round(1)
#     tst_int = int(df_bal10_vshclp2[df_bal10_vshclp2.well == wellname]['TST'].iloc[-1] - df_bal10_vshclp2[df_bal10_vshclp2.well == wellname]['TST'].iloc[0])
#     well_display_logs(df_bal10_vshclp2, wellname, 'Balakhany X','NET_clp2', f'tst: {tst_int}m net2: {net2}m', 
#                       'TST', 10, 'print', 'C:\jupyter\SPP\plots\logs_vsh_bal10\\')

In [ ]:
df_bal8_azr_vshclp2 = pd.read_csv('C:\jupyter\SPP\inputoutput\df_bal8_azr_vshclp2.csv').drop('Unnamed: 0', axis=1)
well_phit_flag8 = df_bal8_azr_vshclp2[df_bal8_azr_vshclp2.phit_flag==1].groupby('well')['phit_flag'].apply(lambda x: x.iloc[0]).reset_index().well.unique()

df_bal10_vshclp2 = pd.read_csv('C:\jupyter\SPP\inputoutput\df_bal10_vshclp2.csv').drop('Unnamed: 0', axis=1)
well_phit_flag10 = df_bal10_vshclp2[df_bal10_vshclp2.phit_flag==1].groupby('well')['phit_flag'].apply(lambda x: x.iloc[0]).reset_index().well.unique()

df_bal8_azr_vshclp2_flag = df_bal8_azr_vshclp2[df_bal8_azr_vshclp2.well.isin(well_phit_flag8)]
df_bal10_vshclp2_flag = df_bal10_vshclp2[df_bal10_vshclp2.well.isin(well_phit_flag10)]

In [20]:
def well_display_logs_cube(dataset, dataset_cube, wellname, fmname, net_var, comments, ref_depth, depth_step, print_flag, path):
    """
    dataset = df_bal or something else
    net_var = NET or FLUIDS_int
    comments = put what you want
    ref_depth = MD or TST
    depth_step = step for ticks on the diagramm
    print = 1 if we want to print the plot
    well_display_logs(df_bal8_azr_vshclp2_flag, 'C01', 'Balakhany VIII', 'NET_clp2', 'df_interp', 'TST')
    """
    data = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname)]
    data_cube = dataset_cube[(dataset_cube.well==wellname) & (dataset_cube.FORMATION_up == fmname)]
    depth = data[ref_depth]
    depth_cube = data_cube[ref_depth]
    grn = data['GR_N']
    vsh = data['VSH']
    vsh_cube = data_cube['VSH_cube']
    rhob = data['RHOB'] 
    npss = data['NPSS']
    rdeep = data['RDEEP']
    phit = data['PHIT']
    phit_cube = data_cube['PHIT_cube']
    net_init = data['NET'] 
    net = data[net_var]
    perm = data['LPERM']
    perm1 = data['PERM_DS']
    perm2 = data['PERM_US']
    fig, ax = plt.subplots(1,4, figsize=(7,7), sharey=True)
    well_bal_tops = dataset[(dataset.well == wellname)].groupby('FORMATION')[ref_depth].apply(lambda x: x.iloc[0]).reset_index()
    ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[0].plot(grn, depth, color='lightgreen', lw=3, zorder=10)
    ax[0].invert_yaxis() 
    ax[0].set_xlim(0, 150) 
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
                    xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    twin0 = ax[0].twiny()
    twin0.plot(vsh, depth, color='black', alpha=0.33, zorder=5)
    twin0.plot(vsh_cube, depth_cube, color='#141313', alpha=0.5, zorder=5)
    twin0.xaxis.set_ticks(np.arange(0, 1.5, 0.5))
    twin0.set_xlim(0, 1.5)
    twin0.grid(axis='x')
    ax[1].plot(rhob, depth, color='red', alpha=0.75) 
    ax[1].invert_yaxis() 
    ax[1].xaxis.set_ticks(np.arange(1.65, 2.65, 0.3))
    ax[1].grid(axis='x')
    ax[1].set_xlim(1.65, 2.65)
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[1].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
        xmin=0, xmax=150, linewidth=2, color='black', lw=2, alpha=0.33)
        ax[1].text(1.67, well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0]+0.5*depth_step, i, fontsize = 8, color ="black")
    twin1 = ax[1].twiny()
    twin1.plot(npss, depth, color='blue', alpha=0.75)
    twin1.set_xlim(0.6, 0)

    twin1.plot(net_init, depth, color='orange', linewidth=0.1)
    twin1.fill_betweenx(depth,net_init, color='orange', alpha=0.25)

    ax[2].plot(phit, depth, color='green', linestyle='dashed')
    ax[2].plot(phit_cube, depth_cube, color='#e91717', linestyle='dashed', alpha=1)
    ax[2].set_xlim(0.3, 0)
    ax[2].xaxis.set_ticks(np.arange(0, 0.3, 0.1))
    ax[2].grid(axis='x')
    ax[2].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[2].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    ax[2].vlines(0.13, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed')
    twin2 = ax[2].twiny()
    twin2.plot(net, depth, color='orange', linewidth=0.5, alpha=1)
    twin2.fill_betweenx(depth,net, color='orange', alpha=0.33)
    twin2.set_xlim(0, 1)
    twin2.set_ylim(min(depth), max(depth))
    # twin2.grid(axis='x')
    ax[3].plot(perm, depth, color='purple', alpha=0.75)
    ax[3].plot(perm1, depth, color='purple', alpha=0.5)
    ax[3].plot(perm2, depth, color='purple', alpha=0.5)
    ax[3].set_xscale('log')
    ax[3].set_xlim(0.1, 1000)
    ax[3].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[3].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    # fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(round(max(kh.dropna()),0)) + ' ' + str(comments), fontsize=14)
    fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(comments), fontsize=14)
    fig.tight_layout()
    if print_flag == 'print':
        fig.savefig(path + wellname + '.png')
    else:
        pass
    plt.show()
# for wellname in df_bal8_azr_vshclp2.well.unique():
#     net2 = (df_bal8_azr_vshclp2[df_bal8_azr_vshclp2.well == wellname]['NET_clp2'].sum()*0.1).round(1)
#     tst_int = int(df_bal8_azr_vshclp2[df_bal8_azr_vshclp2.well == wellname]['TST'].iloc[-1] - df_bal8_azr_vshclp2[df_bal8_azr_vshclp2.well == wellname]['TST'].iloc[0])
#     well_display_logs_cube(df_bal8_azr_vshclp2, df_cube, wellname, 'Balakhany VIII', 'NET_clp2', f'tst: {tst_int}m net2: {net2}m', 'TST', 10, 
#                            'print', 'C:\\jupyter\\SPP\\plots\\logs_vsh_bal8_cube\\')

# for wellname in df_bal10_vshclp2.well.unique():
#     net2 = (df_bal10_vshclp2[df_bal10_vshclp2.well == wellname]['NET_clp2'].sum()*0.1).round(1)
#     tst_int = int(df_bal10_vshclp2[df_bal10_vshclp2.well == wellname]['TST'].iloc[-1] - df_bal10_vshclp2[df_bal10_vshclp2.well == wellname]['TST'].iloc[0])
#     well_display_logs_cube(df_bal10_vshclp2, df_cube, wellname, 'Balakhany X', 'NET_clp2', f'tst: {tst_int}m net2: {net2}m', 'TST', 10, 
#                            'print', 'C:\\jupyter\\SPP\\plots\\logs_vsh_bal10_cube\\')

In [ ]:
def ntd_dataframe_calculation(dataset, fm, net_var):
    print('Calculation NTD based on NET_clp2')
    def ntd_calculation(dataset, well, fm, net_var):
        data = dataset[(dataset.well==well)]
        data.iloc[0, 3] = 0
        data.iloc[-1, 3] = 0
        tst_top = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i-1][net_var]==0)]
        tst_bot = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i+1][net_var]==0)]
        tops = zip(tst_top, tst_bot)
        df_htst = pd.DataFrame(tops, columns=['tst_top', 'tst_bot'])
        df_htst['FORMATION_up'] = fm
        df_htst['well'] = well
        df_htst['h_tst'] = df_htst.tst_bot - df_htst.tst_top
        df_htst = df_htst[['well','FORMATION_up','tst_top','tst_bot','h_tst']]
        return df_htst
    df_recalc_list = []
    for well in tqdm(dataset.well.unique()):
        df = ntd_calculation(dataset, well, fm, net_var)
        df_recalc_list.append(df)
    ntd_net2 = pd.concat(df_recalc_list)
    ntd_net2.drop_duplicates(inplace=True)

    print('Calculation values for NTD')
    def ntd_properties_averaging(dataset_ntd, dataset_logs, fm):
        well_data = []
        well_formation = fm
        df_lst = []
        for well in tqdm(dataset_ntd.well.unique()):
            ntd_well_avgprop = dataset_ntd[(dataset_ntd.well ==well)]
            well_avgprop_sel = dataset_logs[(dataset_logs.well==well)]
            fm_top = dataset_logs[(dataset_logs.well==well)]['TST'].iloc[0]
            fm_bot = dataset_logs[(dataset_logs.well==well)]['TST'].iloc[-1]
            well_phit = []
            well_vsh = []
            well_gperm = []
            well_top = []
            well_bot = []
            well_h = []
            well_fm_top = []
            well_fm_bot = []
            well_name = []
            well_fm = []
            for layers in range(len(ntd_well_avgprop.well)):
                ntd_top = ntd_well_avgprop.iloc[layers, 2].round(3)
                ntd_bot = ntd_well_avgprop.iloc[layers, 3].round(3)
                ntd_h = ntd_well_avgprop.iloc[layers, 4].round(3)
                phit_lst = []
                vsh_lst = []
                perm_lst = []
                for depth in range(len(well_avgprop_sel.TST)):
                    well_avgprop_tst = well_avgprop_sel['TST'].iloc[depth].round(3)
                    if well_avgprop_tst >= ntd_top and well_avgprop_tst <= ntd_bot:
                        phit_lst.append(well_avgprop_sel['PHIT'].iloc[depth])
                        vsh_lst.append(well_avgprop_sel['VSH'].iloc[depth])
                        perm_lst.append(well_avgprop_sel['LPERM'].iloc[depth])
                well_name.append(well)
                well_fm.append(well_formation)
                well_phit.append(mean(phit_lst))
                well_vsh.append(mean(vsh_lst))
                well_gperm.append(gmean(perm_lst))
                well_h.append(ntd_h)
                well_top.append(ntd_top)
                well_bot.append(ntd_bot)
                well_fm_top.append(fm_top)
                well_fm_bot.append(fm_bot)
                well_data = zip(well_name,well_fm,well_phit, well_vsh, well_gperm, well_h, well_top, well_bot, well_fm_top, well_fm_bot)
                well_df = pd.DataFrame(well_data, columns=[ 'well','FORMATION_up',        
                                                            'phit_avg',
                                                            'vsh_avg', 
                                                            'perm_avg',
                                                            'htst',
                                                            'top_tst',
                                                            'bot_tst',
                                                            'fm_top_tst',
                                                            'fm_bot_tst'])
                well_df['not_htst'] = well_df['top_tst'].shift(-1)-well_df['bot_tst']
                well_df = well_df[['well', 'FORMATION_up', 'phit_avg', 'vsh_avg', 'perm_avg', 'htst', 'not_htst','top_tst', 'bot_tst', 'fm_top_tst', 'fm_bot_tst']]
            df_lst.append(well_df)
        result = pd.concat(df_lst)
        return result
    ntd_val = ntd_properties_averaging(ntd_net2, dataset, fm)

    def nothtst_nan_filling(dataset_ntd, fm):
        def nan_change_diff_fmbottom(dataset, wellname, fmname):
            row_change = dataset[(dataset.well == wellname) & (dataset.FORMATION_up == fmname) & (dataset.not_htst.isna())]
            row_change['not_htst'] = row_change['fm_bot_tst'] - row_change['bot_tst']
            return row_change
        df_list = []
        for wellname in dataset_ntd.well.unique():
            df = nan_change_diff_fmbottom(dataset_ntd, wellname, fm)
            df_list.append(df)
        res_df_list = pd.concat(df_list)
        result = pd.concat([dataset_ntd, res_df_list])
        result = result.sort_values(by=['well','top_tst'])
        result_final = result.dropna(subset=['not_htst'], axis=0)

        return result_final
    ntd_val_clean = nothtst_nan_filling(ntd_val, fm)

    def top_phit_bot_clustering(dataset):
        print('Top & bot calculation')
        def top_phit_bot_collection_run(dataset):
            def top_phit_bot_collection(dataset, wellname):
                data = dataset[dataset.well == wellname]
                data['top_htst'] = data['top_tst'] - data['fm_top_tst']
                data['top_htst'].iloc[1:] = data['not_htst'].iloc[:-1]
                data['bot_htst'] = data['not_htst']
                data = data[['well', 'FORMATION_up', 'phit_avg', 'vsh_avg', 'perm_avg',
                            'top_htst','htst','bot_htst', 'fm_top_tst', 'fm_bot_tst']]
                return data
            df_lst = []
            for wellname in tqdm(dataset.well.unique()):
                res_df = top_phit_bot_collection(dataset, wellname)
                df_lst.append(res_df)
            top_phi_bot_cluster = pd.concat(df_lst).reset_index(drop=True)
            return top_phi_bot_cluster
        top_phi_bot_cluster = top_phit_bot_collection_run(dataset)

        def top_phit_bot_ntg_run(dataset):
            def top_phit_bot_ntg(dataset, wellname):
                ntg = []
                data = dataset[dataset.well == wellname].reset_index(drop=True)
                for ind, row in data.iterrows():
                    if ind == 0:
                        ntg.append(row['htst']/(row['bot_htst'] + row['htst']))
                    if ind != 0:
                        ntg.append(row['htst']/(row['bot_htst'] + row['htst'] + row['top_htst']))
                    if ind == len(data):
                        ntg.append(row['htst']/(row['top_htst'] + row['htst']))
                result = pd.concat([data, pd.DataFrame({'ntg':ntg})], axis=1)
                return result
            df_lst = []
            for wellname in dataset.well.unique():
                df = top_phit_bot_ntg(dataset, wellname)
                df_lst.append(df)
            top_phi_bot_cluster_ntg = pd.concat(df_lst).reset_index(drop=True)
            return top_phi_bot_cluster_ntg
        top_phi_bot_cluster_ntg = top_phit_bot_ntg_run(top_phi_bot_cluster)
        
        return top_phi_bot_cluster_ntg
    ntd_top_phi_bot = top_phit_bot_clustering(ntd_val_clean)

    return ntd_top_phi_bot.reset_index(drop=True)
ntd_top_vsh_bot8 = ntd_dataframe_calculation(df_bal8_azr_vshclp2, 'Balakhany VIII', 'NET_clp2')
ntd_top_phi_bot8 = ntd_dataframe_calculation(df_bal8_azr_vshclp2_flag, 'Balakhany VIII', 'NET_clp2')
ntd_top_vsh_bot10 = ntd_dataframe_calculation(df_bal10_vshclp2, 'Balakhany X', 'NET_clp2')
ntd_top_phi_bot10 = ntd_dataframe_calculation(df_bal10_vshclp2_flag, 'Balakhany X', 'NET_clp2')

### net_orig vs net_clp2 comparison

In [22]:
# well_display_logs(df_bal10_vshclp2, 'J11', 'Balakhany X', 'NET_clp2', 'vshclp2', 'TST', 10, 'dontprint', 'C:\jupyter\SPP\plots\logs_ntd_vsh_bal8\\')

In [ ]:
# bar_plot_net_net2_preparation bal8
def bar_plot_net_net2_preparation(dataset):
    df = dataset.groupby('well')[['NET','NET_clp2']].sum().reset_index()
    df['NET'] = (df.NET*0.1).round(0)
    df['NET_clp2'] = (df.NET_clp2*0.1).round(0)
    df['diff'] = df.NET - df.NET_clp2
    df = df.sort_values(by=['diff'], ascending=False)
    return df
df8 = bar_plot_net_net2_preparation(df_bal8_azr_vshclp2)
def bar_plot_net_net2(df, letter, comment, print_flag, path):
    df = df[df.well.str.startswith(letter)]
    fig = go.Figure(data=[  go.Bar(name='NET', x=df['well'], y=df['NET']),
                            go.Bar(name='NET_clp2', x=df['well'], y=df['NET_clp2'])])
    fig.update_layout(autosize=False,
                      width=1600,  # width of the figure
                      height=600, # height of the figure
                      barmode='group',
                      title='Bar Plot for NET and NET_clp2 ' + comment, 
                      margin=dict(l=10,r=10,b=10,t=50))
    if print_flag == 'print':
        fig.write_html(path + letter + '.html')
    fig.show()
bar_plot_net_net2(df8, 'C', 'C Bal8', 'dontprint', r'C:\jupyter\SPP\plots\bal8_net_net2_')
bar_plot_net_net2(df8, 'B', 'B Bal8', 'dontprint', r'C:\jupyter\SPP\plots\bal8_net_net2_')
bar_plot_net_net2(df8, 'D', 'D Bal8', 'dontprint', r'C:\jupyter\SPP\plots\bal8_net_net2_')

In [ ]:
# bar_plot_net_net2_preparation bal10
df10 = bar_plot_net_net2_preparation(df_bal10_vshclp2)
bar_plot_net_net2(df10, 'A', 'A Bal10', 'dontprint', r'C:\jupyter\SPP\plots\bal10_net_net2_')
bar_plot_net_net2(df10, 'E', 'E Bal10', 'dontprint', r'C:\jupyter\SPP\plots\bal10_net_net2_')
bar_plot_net_net2(df10, 'G', 'G Bal10', 'dontprint', r'C:\jupyter\SPP\plots\bal10_net_net2_')
bar_plot_net_net2(df10, 'H', 'H Bal10', 'dontprint', r'C:\jupyter\SPP\plots\bal10_net_net2_')
bar_plot_net_net2(df10, 'J', 'J Bal10', 'dontprint', r'C:\jupyter\SPP\plots\bal10_net_net2_')
bar_plot_net_net2(df10, 'C', 'C Bal10', 'dontprint', r'C:\jupyter\SPP\plots\bal10_net_net2_')
bar_plot_net_net2(df10, 'B', 'B Bal10', 'dontprint', r'C:\jupyter\SPP\plots\bal10_net_net2_')
bar_plot_net_net2(df10, 'D', 'D Bal10', 'dontprint', r'C:\jupyter\SPP\plots\bal10_net_net2_')

In [ ]:
# scatter_plot_net_net2 bal8 & 10
def scatter_plot_net_net2(df, letter, comment, print_flag, path):
    df = df[df.well.str.startswith(letter)]
    fig = go.Figure(data=[go.Scatter(name='NET', 
                                     x=df['NET'], 
                                     y=df['NET_clp2'], 
                                     mode='markers',
                                     marker=dict(size=10, color='green', opacity=0.5, line=dict(color='black', width=1)),
                                     customdata = df[['well','NET', 'NET_clp2']],
                                     hovertemplate="".join(["well: %{customdata[0]} "+
                                                            "net: %{customdata[1]} " + 
                                                            "net_clp2: %{customdata[2]} <extra></extra>"])),
                          go.Scatter(name='Line', 
                                     x=[0,df['NET'].max()+5], 
                                     y=[0,df['NET'].max()+5], 
                                     mode='lines')])
    fig.update_layout(autosize=False,
                      width=800,  
                      height=700,
                      xaxis_title="NET",
                      yaxis_title="NET_clp2",
                      title=' Plot for NET and NET_clp2 ' + comment, 
                      margin=dict(l=10,r=10,b=10,t=50),
                      showlegend=False)
    if print_flag == 'print':
        fig.write_html(path + letter + '.html')
    fig.show()
scatter_plot_net_net2(df8, 'C', 'C Bal8', 'print', r'C:\jupyter\SPP\plots\bal8_net_net2_scatter_')
scatter_plot_net_net2(df8, 'B', 'B Bal8', 'print', r'C:\jupyter\SPP\plots\bal8_net_net2_scatter_')
scatter_plot_net_net2(df8, 'D', 'D Bal8', 'print', r'C:\jupyter\SPP\plots\bal8_net_net2_scatter_')
scatter_plot_net_net2(df10, 'A', 'A Bal10', 'print', r'C:\jupyter\SPP\plots\bal10_net_net2_scatter_')
scatter_plot_net_net2(df10, 'E', 'E Bal10', 'print', r'C:\jupyter\SPP\plots\bal10_net_net2_scatter_')
scatter_plot_net_net2(df10, 'G', 'G Bal10', 'print', r'C:\jupyter\SPP\plots\bal10_net_net2_scatter_')
scatter_plot_net_net2(df10, 'H', 'H Bal10', 'print', r'C:\jupyter\SPP\plots\bal10_net_net2_scatter_')
scatter_plot_net_net2(df10, 'J', 'J Bal10', 'print', r'C:\jupyter\SPP\plots\bal10_net_net2_scatter_')
scatter_plot_net_net2(df10, 'C', 'C Bal10', 'print', r'C:\jupyter\SPP\plots\bal10_net_net2_scatter_')
scatter_plot_net_net2(df10, 'B', 'B Bal10', 'print', r'C:\jupyter\SPP\plots\bal10_net_net2_scatter_')
scatter_plot_net_net2(df10, 'D', 'D Bal10', 'print', r'C:\jupyter\SPP\plots\bal10_net_net2_scatter_')


In [30]:
well_to_check_net = pd.concat([df8[df8['diff'] >= 10], df8[df8['diff'] <= -10]]).well.unique()
# for wellname in well_to_check_net:
#     diff_var = df8[df8.well == wellname]['diff'].iloc[0]
#     well_display_logs(df_bal8_azr_vshclp2, wellname, 'Balakhany VIII','NET_clp2', 'diff: '+ str(diff_var), 'TST', 10, 'dontprint', 
#                       "C:\\jupyter\\SPP\\plots\\logs_vsh_bal8_check\\")

In [31]:
well_to_check_net = pd.concat([df10[df10['diff'] >= 10], df10[df10['diff'] <= -10]]).well.unique()
# for wellname in well_to_check_net:
#     diff_var = df10[df10.well == wellname]['diff'].iloc[0]
#     well_display_logs(df_bal10_vshclp2, wellname, 'Balakhany X','NET_clp2', 'diff: '+ str(diff_var), 'TST', 10, 'dontprint', 
#                       "C:\\jupyter\\SPP\\plots\\logs_vsh_bal10_check\\")

In [28]:
# well_display_logs(df_bal10_vshclp2, 'A12X', 'Balakhany X', 'NET_clp2', 'vshclp2', 'TST', 10, 'dontprint', 'C:\jupyter\SPP\plots\logs_ntd_vsh_bal10\\')

# Clustering

## Data clustering Bal8

In [ ]:
def data_clustering(dataset, feature_list, scaler, cluster_num):
    """
    MinMaxScaler(), StandardScaler()
    """
    data = dataset[feature_list]
    normalized_data = scaler.fit_transform(data)

    kmeans = KMeans(n_clusters=cluster_num, random_state=42)
    kmeans_labels = kmeans.fit_predict(normalized_data)
    kmeans_labels = pd.DataFrame(kmeans_labels, columns=['kmeans'])

    gmm = GaussianMixture(n_components=cluster_num, random_state=42)
    gmm.fit(normalized_data)
    gmm_labels = gmm.predict(normalized_data)
    gmm_labels = pd.DataFrame(gmm_labels, columns=['gmm'])

    agglomerative = AgglomerativeClustering(n_clusters=cluster_num)
    agglomerative_labels = agglomerative.fit_predict(normalized_data)
    agglomerative_labels = pd.DataFrame(agglomerative_labels, columns=['agglomer'])
    result = pd.concat([dataset, kmeans_labels, gmm_labels, agglomerative_labels], axis=1)
    return result
data_phi_clustered8 = data_clustering(ntd_top_phi_bot8, ['phit_avg','htst'], StandardScaler(), 3)
data_phi_clustered8.head(3)

In [ ]:
def histo_clustering(dataset, clustering, comment):
    data = dataset[dataset.phit_avg !=0]
    fig, ax = plt.subplots(1, 4, figsize=(18, 4))
    custom_palette = {2: 'red', 1: 'green', 0: 'blue'}
    sns.histplot(data=data, x='phit_avg', hue=clustering, bins=30, ax=ax[0], kde=True,  palette=custom_palette)
    ax[0].grid(True, axis='x'), ax[0].set_xticks(np.arange(0.12, 0.32, 0.02)), ax[0].tick_params(axis='both', which='major', labelsize=8)
    sns.histplot(data=data[data.htst < 30], x='htst', hue=clustering, bins=30, ax=ax[1], kde=True,  palette=custom_palette)
    ax[1].grid(True, axis='x'), ax[1].set_xticks(np.arange(0, 30, 3)), ax[1].tick_params(axis='both', which='major', labelsize=8)
    sns.histplot(data=data, x='ntg', hue=clustering, bins=30, ax=ax[2], kde=True,  palette=custom_palette)
    ax[2].grid(True, axis='x'), ax[2].set_xticks(np.arange(0, 1, 0.1)), ax[2].tick_params(axis='both', which='major', labelsize=8)
    sns.histplot(data=data, x='vsh_avg', hue=clustering, bins=30, ax=ax[3], kde=True,  palette=custom_palette)
    ax[3].grid(True, axis='x'), ax[3].set_xticks(np.arange(0, 0.6, 0.1)), ax[3].tick_params(axis='both', which='major', labelsize=8)
    fig.suptitle(comment)
    plt.show()
histo_clustering(data_phi_clustered8, 'kmeans', 'kmeans Bal VIII')
histo_clustering(data_phi_clustered8, 'gmm', 'gmm Bal VIII')
histo_clustering(data_phi_clustered8, 'agglomer', 'agglomer Bal VIII')

In [ ]:
def well_collecting_clusters_top_phi_bot(dataset, clustering, fm):
    df_lst = []
    for wellname in dataset.well.unique()[:]:
        data = dataset[dataset.well == wellname]
        well_lst = []
        phit_lst = []
        htst_lst = []
        bot_lst = []
        ntg_lst = []
        vsh_lst = []
        perm_lst = []
        cluster_lst = []
        for ind, row in data.iterrows():
            well_lst.append(wellname)
            well_lst.append(wellname)

            phit_lst.append(0)
            phit_lst.append(row['phit_avg'])

            cluster_lst.append(np.nan)
            cluster_lst.append(row[clustering])

            htst_lst.append(row['top_htst'])
            htst_lst.append(row['htst'])
            
            bot_lst.append(row['bot_htst'])

            ntg_lst.append(0)
            ntg_lst.append(row['ntg'])

            vsh_lst.append(0)
            vsh_lst.append(row['vsh_avg'])

            perm_lst.append(0)
            perm_lst.append(row['perm_avg'])

        phit_lst.append(0)
        cluster_lst.append(np.nan)
        htst_lst.append(data['bot_htst'].iloc[-1])
        well_lst.append(wellname)
        well_collect_cluster_short = pd.DataFrame(zip(well_lst, phit_lst, htst_lst, ntg_lst, vsh_lst, perm_lst, cluster_lst ), 
                                                  columns=[ 'well','phit', 'htst', 
                                                            'ntg', 'vsh', 'perm','cluster'])
        well_last_row = pd.DataFrame({'well':[well_lst[-1]], 'phit':[0], 'htst': [bot_lst[-1]], 
                                      'ntg':[0], 'vsh':[0], 'perm':[0],'cluster':[cluster_lst[-1]]})
        well_collect_cluster = pd.concat([well_collect_cluster_short, well_last_row]).reset_index(drop=True)
        well_collect_cluster['depth'] = well_collect_cluster['htst'].cumsum()
        df_lst.append(well_collect_cluster)
    result = pd.concat(df_lst)
    result['FORMATION_up'] = fm
    return result
ntd_phit8_clusters = well_collecting_clusters_top_phi_bot(data_phi_clustered8, 'kmeans', 'Balakhany VIII')
def coloring_clusters_matrix(dataset, letters_list, rows, columns, clustering, print_flag):
    """
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J']
    """
    def clusters_rectangle(data, k, color):
        # cluster_xy = data['depth'].iloc[k-2]
        cluster_xy = data['depth'].iloc[k-1]
        # cluster_h = data['depth'].iloc[k+1] - data['depth'].iloc[k-2]
        cluster_h = data['depth'].iloc[k] - data['depth'].iloc[k-1]
        rectangle = patches.Rectangle((0, cluster_xy) , 1, cluster_h, edgecolor=color, facecolor=color, alpha=0.25)
        ax[j,i].add_patch(rectangle)
    for letter in letters_list:
        wells_letter = [wellname for wellname in dataset.well.unique() if wellname.startswith(letter)]
        fig, ax = plt.subplots(rows,columns, figsize=(16,rows*2.5))
        counter = 0
        for j in range(0, rows):
            for i in range(0, columns):
                if counter < len(wells_letter):
                    wellname = wells_letter[counter]
                    welldata = dataset[dataset.well==wellname]
                    df_top = pd.DataFrame({'well':[wellname], 'phit':[0], 'htst':[0], 'cluster':welldata['cluster'].iloc[0],'depth':[0]})
                    welldata = pd.concat([df_top, welldata]).reset_index().drop('index', axis=1)
                    ax[j,i].plot(welldata['phit'], welldata['depth'], drawstyle='steps-post', color='black', alpha=1, lw=0.75)
                    ax[j,i].plot(welldata['vsh']*0.35, welldata['depth'], drawstyle='steps-post', color='green', alpha=1, lw=1)
                    ax[j,i].set_xlim(0, 0.35)
                    ax[j,i].set_ylim(0, 150)
                    ax[j,i].invert_yaxis()
                    ax[j,i].set_title(wellname)
                    ax[j,i].xaxis.set_ticks(np.arange(0, 0.35, 0.1))
                    # ax[j,i].tick_params(axis='both', which='major', labelsize=10)
                    ax[j,i].grid()
                    for k in range(len(welldata)):
                        if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == 0:
                            clusters_rectangle(welldata, k, 'blue')
                        if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == 1:
                            clusters_rectangle(welldata, k, 'green')
                        if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == 2:
                            clusters_rectangle(welldata, k, 'red')
                    # labels=['0-2', '2-4', '4-7', '7-100'])
                    # for k in range(len(welldata)):
                    #     if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == '0-2':
                    #         clusters_rectangle(welldata, k, 'green')
                    #     if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == '2-4':
                    #         clusters_rectangle(welldata, k, 'lightgreen')
                    #     if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == '4-7':
                    #         clusters_rectangle(welldata, k, 'yellow')
                    #     if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == '7-100':
                    #         clusters_rectangle(welldata, k, 'orange')
                    fig.suptitle(clustering)
                    fig.tight_layout()
                    counter +=1
        if print_flag == 'print':
            plt.savefig('.\plots\\' + clustering + '_' + str(letter) +'.pdf')
        else:
            pass
        plt.show()
coloring_clusters_matrix(ntd_phit8_clusters, ['C','B','D'], 4, 9, 'kmeans bal8', 'dontprint')

## Data clustering Bal10

In [ ]:
def data_clustering(dataset, feature_list, scaler, cluster_num):
    """
    MinMaxScaler(), StandardScaler()
    """
    data = dataset[feature_list]
    normalized_data = scaler.fit_transform(data)

    kmeans = KMeans(n_clusters=cluster_num, random_state=42)
    kmeans_labels = kmeans.fit_predict(normalized_data)
    kmeans_labels = pd.DataFrame(kmeans_labels, columns=['kmeans'])

    gmm = GaussianMixture(n_components=cluster_num, random_state=42)
    gmm.fit(normalized_data)
    gmm_labels = gmm.predict(normalized_data)
    gmm_labels = pd.DataFrame(gmm_labels, columns=['gmm'])

    agglomerative = AgglomerativeClustering(n_clusters=cluster_num)
    agglomerative_labels = agglomerative.fit_predict(normalized_data)
    agglomerative_labels = pd.DataFrame(agglomerative_labels, columns=['agglomer'])
    result = pd.concat([dataset, kmeans_labels, gmm_labels, agglomerative_labels], axis=1)
    return result
data_phi_clustered10 = data_clustering(ntd_top_phi_bot10, ['phit_avg','htst'], StandardScaler(), 3)
data_phi_clustered10.head(3)

In [ ]:
def histo_clustering(dataset, clustering, comment):
    data = dataset[dataset.phit_avg !=0]
    fig, ax = plt.subplots(1, 4, figsize=(18, 4))
    custom_palette = {2: 'red', 1: 'green', 0: 'blue'}
    sns.histplot(data=data, x='phit_avg', hue=clustering, bins=30, ax=ax[0], kde=True,  palette=custom_palette)
    ax[0].grid(True, axis='x'), ax[0].set_xticks(np.arange(0.12, 0.32, 0.02)), ax[0].tick_params(axis='both', which='major', labelsize=8)
    sns.histplot(data=data[data.htst < 30], x='htst', hue=clustering, bins=30, ax=ax[1], kde=True,  palette=custom_palette)
    ax[1].grid(True, axis='x'), ax[1].set_xticks(np.arange(0, 30, 2)), ax[1].tick_params(axis='both', which='major', labelsize=8)
    sns.histplot(data=data, x='ntg', hue=clustering, bins=30, ax=ax[2], kde=True,  palette=custom_palette)
    ax[2].grid(True, axis='x'), ax[2].set_xticks(np.arange(0, 1, 0.1)), ax[2].tick_params(axis='both', which='major', labelsize=8)
    sns.histplot(data=data, x='vsh_avg', hue=clustering, bins=30, ax=ax[3], kde=True,  palette=custom_palette)
    ax[3].grid(True, axis='x'), ax[3].set_xticks(np.arange(0, 0.6, 0.1)), ax[3].tick_params(axis='both', which='major', labelsize=8)
    fig.suptitle(comment)
    plt.show()
histo_clustering(data_phi_clustered10, 'kmeans', 'kmeans Bal X')
histo_clustering(data_phi_clustered10, 'gmm', 'gmm Bal X')
histo_clustering(data_phi_clustered10, 'agglomer', 'agglomer Bal X')

In [ ]:
def well_collecting_clusters_top_phi_bot(dataset, clustering, fm):
    df_lst = []
    for wellname in dataset.well.unique()[:]:
        data = dataset[dataset.well == wellname]
        well_lst = []
        phit_lst = []
        htst_lst = []
        bot_lst = []
        ntg_lst = []
        vsh_lst = []
        perm_lst = []
        cluster_lst = []
        for ind, row in data.iterrows():
            well_lst.append(wellname)
            well_lst.append(wellname)

            phit_lst.append(0)
            phit_lst.append(row['phit_avg'])

            cluster_lst.append(np.nan)
            cluster_lst.append(row[clustering])

            htst_lst.append(row['top_htst'])
            htst_lst.append(row['htst'])
            
            bot_lst.append(row['bot_htst'])

            ntg_lst.append(0)
            ntg_lst.append(row['ntg'])

            vsh_lst.append(0)
            vsh_lst.append(row['vsh_avg'])

            perm_lst.append(0)
            perm_lst.append(row['perm_avg'])

        phit_lst.append(0)
        cluster_lst.append(np.nan)
        htst_lst.append(data['bot_htst'].iloc[-1])
        well_lst.append(wellname)
        well_collect_cluster_short = pd.DataFrame(zip(well_lst, phit_lst, htst_lst, ntg_lst, vsh_lst, perm_lst, cluster_lst ), 
                                                  columns=[ 'well','phit', 'htst', 
                                                            'ntg', 'vsh', 'perm','cluster'])
        well_last_row = pd.DataFrame({'well':[well_lst[-1]], 'phit':[0], 'htst': [bot_lst[-1]], 
                                      'ntg':[0], 'vsh':[0], 'perm':[0],'cluster':[cluster_lst[-1]]})
        well_collect_cluster = pd.concat([well_collect_cluster_short, well_last_row]).reset_index(drop=True)
        well_collect_cluster['depth'] = well_collect_cluster['htst'].cumsum()
        df_lst.append(well_collect_cluster)
    result = pd.concat(df_lst)
    result['FORMATION_up'] = fm
    return result
ntd_phit10_clusters = well_collecting_clusters_top_phi_bot(data_phi_clustered10, 'kmeans', 'Balakhany VIII')
def coloring_clusters_matrix(dataset, letters_list, rows, columns, clustering, print_flag):
    """
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J']
    """
    def clusters_rectangle(data, k, color):
        # cluster_xy = data['depth'].iloc[k-2]
        cluster_xy = data['depth'].iloc[k-1]
        # cluster_h = data['depth'].iloc[k+1] - data['depth'].iloc[k-2]
        cluster_h = data['depth'].iloc[k] - data['depth'].iloc[k-1]
        rectangle = patches.Rectangle((0, cluster_xy) , 1, cluster_h, edgecolor=color, facecolor=color, alpha=0.25)
        ax[j,i].add_patch(rectangle)
    for letter in letters_list:
        wells_letter = [wellname for wellname in dataset.well.unique() if wellname.startswith(letter)]
        fig, ax = plt.subplots(rows,columns, figsize=(16,rows*2.5))
        counter = 0
        for j in range(0, rows):
            for i in range(0, columns):
                if counter < len(wells_letter):
                    wellname = wells_letter[counter]
                    welldata = dataset[dataset.well==wellname]
                    df_top = pd.DataFrame({'well':[wellname], 'phit':[0], 'htst':[0], 'cluster':welldata['cluster'].iloc[0],'depth':[0]})
                    welldata = pd.concat([df_top, welldata]).reset_index().drop('index', axis=1)
                    ax[j,i].plot(welldata['phit'], welldata['depth'], drawstyle='steps-post', color='black', alpha=1, lw=0.75)
                    ax[j,i].plot(welldata['vsh']*0.35, welldata['depth'], drawstyle='steps-post', color='green', alpha=1, lw=1)
                    ax[j,i].set_xlim(0, 0.35)
                    ax[j,i].set_ylim(0, 150)
                    ax[j,i].invert_yaxis()
                    ax[j,i].set_title(wellname)
                    ax[j,i].xaxis.set_ticks(np.arange(0, 0.35, 0.1))
                    # ax[j,i].tick_params(axis='both', which='major', labelsize=10)
                    ax[j,i].grid()
                    for k in range(len(welldata)):
                        if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == 0:
                            clusters_rectangle(welldata, k, 'blue')
                        if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == 1:
                            clusters_rectangle(welldata, k, 'green')
                        if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == 2:
                            clusters_rectangle(welldata, k, 'red')
                    # labels=['0-2', '2-4', '4-7', '7-100'])
                    # for k in range(len(welldata)):
                    #     if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == '0-2':
                    #         clusters_rectangle(welldata, k, 'green')
                    #     if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == '2-4':
                    #         clusters_rectangle(welldata, k, 'lightgreen')
                    #     if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == '4-7':
                    #         clusters_rectangle(welldata, k, 'yellow')
                    #     if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == '7-100':
                    #         clusters_rectangle(welldata, k, 'orange')
                    fig.suptitle(clustering)
                    fig.tight_layout()
                    counter +=1
        if print_flag == 'print':
            plt.savefig('.\plots\\' + clustering + '_' + str(letter) +'.pdf')
        else:
            pass
        plt.show()
coloring_clusters_matrix(ntd_phit10_clusters, ['C','B','D'], 4, 9, 'kmeans bal10', 'dontprint')

# Htst distrib

### Htst distribution Bal8 BP

In [ ]:
df_bal8_azr_v2 = pd.read_csv('C:\jupyter\SPP\inputoutput\df_bal8_azr_v2.csv').drop('Unnamed: 0', axis=1)
well_phit_flag8 = df_bal8_azr_v2[df_bal8_azr_v2.phit_flag==1].groupby('well')['phit_flag'].apply(lambda x: x.iloc[0]).reset_index().well.unique()
df_bal8_azr_v2_flag = df_bal8_azr_v2[df_bal8_azr_v2.well.isin(well_phit_flag8)]

def ntd_dataframe_calculation_bp(dataset, fm, net_var):
    print('Calculation NTD based on NET_clp2')
    def ntd_calculation(dataset, well, fm, net_var):
        data = dataset[(dataset.well==well)]
        data.iloc[0, 3] = 0
        data.iloc[-1, 3] = 0
        tst_top = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i-1][net_var]==0)]
        tst_bot = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i+1][net_var]==0)]
        tops = zip(tst_top, tst_bot)
        df_htst = pd.DataFrame(tops, columns=['tst_top', 'tst_bot'])
        df_htst['FORMATION_up'] = fm
        df_htst['well'] = well
        df_htst['h_tst'] = df_htst.tst_bot - df_htst.tst_top
        df_htst = df_htst[['well','FORMATION_up','tst_top','tst_bot','h_tst']]
        return df_htst
    df_recalc_list = []
    for well in tqdm(dataset.well.unique()):
        df = ntd_calculation(dataset, well, fm, net_var)
        df_recalc_list.append(df)
    ntd_net2 = pd.concat(df_recalc_list)
    ntd_net2.drop_duplicates(inplace=True)

    print('Calculation values for NTD')
    def ntd_properties_averaging(dataset_ntd, dataset_logs, fm):
        well_data = []
        well_formation = fm
        df_lst = []
        for well in tqdm(dataset_ntd.well.unique()):
            ntd_well_avgprop = dataset_ntd[(dataset_ntd.well ==well)]
            well_avgprop_sel = dataset_logs[(dataset_logs.well==well)]
            fm_top = dataset_logs[(dataset_logs.well==well)]['TST'].iloc[0]
            fm_bot = dataset_logs[(dataset_logs.well==well)]['TST'].iloc[-1]
            well_phit = []
            well_vsh = []
            well_gperm = []
            well_top = []
            well_bot = []
            well_h = []
            well_fm_top = []
            well_fm_bot = []
            well_name = []
            well_fm = []
            for layers in range(len(ntd_well_avgprop.well)):
                ntd_top = ntd_well_avgprop.iloc[layers, 2].round(3)
                ntd_bot = ntd_well_avgprop.iloc[layers, 3].round(3)
                ntd_h = ntd_well_avgprop.iloc[layers, 4].round(3)
                phit_lst = []
                vsh_lst = []
                perm_lst = []
                for depth in range(len(well_avgprop_sel.TST)):
                    well_avgprop_tst = well_avgprop_sel['TST'].iloc[depth].round(3)
                    if well_avgprop_tst >= ntd_top and well_avgprop_tst <= ntd_bot:
                        phit_lst.append(well_avgprop_sel['PHIT'].iloc[depth])
                        vsh_lst.append(well_avgprop_sel['VSH'].iloc[depth])
                        perm_lst.append(well_avgprop_sel['LPERM'].iloc[depth])
                well_name.append(well)
                well_fm.append(well_formation)
                well_phit.append(mean(phit_lst))
                well_vsh.append(mean(vsh_lst))
                well_gperm.append(gmean(perm_lst))
                well_h.append(ntd_h)
                well_top.append(ntd_top)
                well_bot.append(ntd_bot)
                well_fm_top.append(fm_top)
                well_fm_bot.append(fm_bot)
                well_data = zip(well_name,well_fm,well_phit, well_vsh, well_gperm, well_h, well_top, well_bot, well_fm_top, well_fm_bot)
                well_df = pd.DataFrame(well_data, columns=[ 'well','FORMATION_up',        
                                                            'phit_avg',
                                                            'vsh_avg', 
                                                            'perm_avg',
                                                            'htst',
                                                            'top_tst',
                                                            'bot_tst',
                                                            'fm_top_tst',
                                                            'fm_bot_tst'])
                well_df['not_htst'] = well_df['top_tst'].shift(-1)-well_df['bot_tst']
                well_df = well_df[['well', 'FORMATION_up', 'phit_avg', 'vsh_avg', 'perm_avg', 'htst', 'not_htst','top_tst', 'bot_tst', 'fm_top_tst', 'fm_bot_tst']]
            df_lst.append(well_df)
        result = pd.concat(df_lst)
        return result
    ntd_val = ntd_properties_averaging(ntd_net2, dataset, fm)

    def nothtst_nan_filling(dataset_ntd, fm):
        def nan_change_diff_fmbottom(dataset, wellname, fmname):
            row_change = dataset[(dataset.well == wellname) & (dataset.FORMATION_up == fmname) & (dataset.not_htst.isna())]
            row_change['not_htst'] = row_change['fm_bot_tst'] - row_change['bot_tst']
            return row_change
        df_list = []
        for wellname in dataset_ntd.well.unique():
            df = nan_change_diff_fmbottom(dataset_ntd, wellname, fm)
            df_list.append(df)
        res_df_list = pd.concat(df_list)
        result = pd.concat([dataset_ntd, res_df_list])
        result = result.sort_values(by=['well','top_tst'])
        result_final = result.dropna(subset=['not_htst'], axis=0)

        return result_final
    ntd_val_clean = nothtst_nan_filling(ntd_val, fm)

    def top_phit_bot_clustering(dataset):
        print('Top & bot calculation')
        def top_phit_bot_collection_run(dataset):
            def top_phit_bot_collection(dataset, wellname):
                data = dataset[dataset.well == wellname]
                data['top_htst'] = data['top_tst'] - data['fm_top_tst']
                data['top_htst'].iloc[1:] = data['not_htst'].iloc[:-1]
                data['bot_htst'] = data['not_htst']
                data = data[['well', 'FORMATION_up', 'phit_avg', 'vsh_avg', 'perm_avg',
                            'top_htst','htst','bot_htst', 'fm_top_tst', 'fm_bot_tst']]
                return data
            df_lst = []
            for wellname in tqdm(dataset.well.unique()):
                res_df = top_phit_bot_collection(dataset, wellname)
                df_lst.append(res_df)
            top_phi_bot_cluster = pd.concat(df_lst).reset_index(drop=True)
            return top_phi_bot_cluster
        top_phi_bot_cluster = top_phit_bot_collection_run(dataset)

        def top_phit_bot_ntg_run(dataset):
            def top_phit_bot_ntg(dataset, wellname):
                ntg = []
                data = dataset[dataset.well == wellname].reset_index(drop=True)
                for ind, row in data.iterrows():
                    if ind == 0:
                        ntg.append(row['htst']/(row['bot_htst'] + row['htst']))
                    if ind != 0:
                        ntg.append(row['htst']/(row['bot_htst'] + row['htst'] + row['top_htst']))
                    if ind == len(data):
                        ntg.append(row['htst']/(row['top_htst'] + row['htst']))
                result = pd.concat([data, pd.DataFrame({'ntg':ntg})], axis=1)
                return result
            df_lst = []
            for wellname in dataset.well.unique():
                df = top_phit_bot_ntg(dataset, wellname)
                df_lst.append(df)
            top_phi_bot_cluster_ntg = pd.concat(df_lst).reset_index(drop=True)
            return top_phi_bot_cluster_ntg
        # top_phi_bot_cluster_ntg = top_phit_bot_ntg_run(top_phi_bot_cluster)
        
        return top_phi_bot_cluster
    ntd_top_phi_bot = top_phit_bot_clustering(ntd_val_clean)

    return ntd_top_phi_bot.reset_index(drop=True)
ntd_top_vsh_bot8_bp = ntd_dataframe_calculation_bp(df_bal8_azr_v2, 'Balakhany VIII', 'NET')
ntd_top_phi_bot8_bp = ntd_dataframe_calculation_bp(df_bal8_azr_v2_flag, 'Balakhany VIII', 'NET')
ntd_top_phi_bot8_bp.to_csv(r'C:\jupyter\SPP\inputoutput\ntd_top_phi_bot8_bp.csv')

In [ ]:
def well_display_logs_net_corr(dataset, wellname, vsh_cuttoff, fmname, net_var, comments, ref_depth, depth_step, print_flag, path):
    """
    dataset = df_bal or something else
    net_var = NET or FLUIDS_int
    comments = put what you want
    ref_depth = MD or TST
    depth_step = step for ticks on the diagramm
    print = 1 if we want to print the plot
    well_display_logs(df_bal8_azr_vshclp2_flag, 'C01', 'Balakhany VIII', 'NET_clp2', 'df_interp', 'TST')
    """
    data = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname)]
    depth = data[ref_depth]
    grn = data['GR_N']
    vsh = data['VSH']
    rhob = data['RHOB'] 
    npss = data['NPSS']
    rdeep = data['RDEEP']
    phit = data['PHIT']
    net_init = data['NET_VSH']*10 
    net = data[net_var]*10
    perm = data['LPERM']
    perm1 = data['PERM_DS']
    perm2 = data['PERM_US']
    fig, ax = plt.subplots(1,4, figsize=(7,15), sharey=True)
    well_bal_tops = dataset[(dataset.well == wellname)].groupby('FORMATION')[ref_depth].apply(lambda x: x.iloc[0]).reset_index()
    ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[0].plot(grn, depth, color='lightgreen', lw=3, zorder=10)
    ax[0].invert_yaxis() 
    ax[0].set_xlim(0, 150) 
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
                    xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    twin0 = ax[0].twiny()
    twin0.plot(vsh, depth, color='black', alpha=0.33, zorder=5)
    twin0.vlines(vsh_cuttoff, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    twin0.xaxis.set_ticks(np.arange(0, 1.5, 0.5))
    twin0.set_xlim(0, 1.1)
    twin0.grid(True)
    twin0.plot(net_init, depth, color='orange', linewidth=0.5, alpha=1)
    twin0.fill_betweenx(depth,net_init, color='orange', alpha=0.25)
    twin0.text(0.25, depth.iloc[15], 'NET_VSH', fontsize=12)
    ax[1].plot(rhob, depth, color='red', alpha=0.75) 
    ax[1].invert_yaxis() 
    ax[1].xaxis.set_ticks(np.arange(1.65, 2.65, 0.3))
    ax[1].grid(True)
    ax[1].set_xlim(1.65, 2.65)
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[1].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
        xmin=0, xmax=150, linewidth=2, color='black', lw=2, alpha=0.33)
        ax[1].text(1.67, well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0]+0.5*depth_step, i, fontsize = 8, color ="black")
    twin1 = ax[1].twiny()
    twin1.plot(npss, depth, color='blue', alpha=0.75)
    twin1.set_xlim(0.6, 0)
    twin1.grid(False)
    # twin1.plot(net_init, depth, color='orange', linewidth=0.5, alpha=1)
    # twin1.fill_betweenx(depth,net_init, color='orange', alpha=0.25)
    # twin1.plot(net, depth, color='orange', linewidth=0.5, alpha=1)
    # twin1.fill_betweenx(depth,net, color='orange', alpha=0.33)
    ax[2].plot(phit, depth, color='green', linestyle='dashed')
    ax[2].set_xlim(0.3, 0)
    ax[2].xaxis.set_ticks(np.arange(0, 0.3, 0.1))
    ax[2].tick_params(labelsize=10)
    ax[2].grid(True)
    ax[2].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[2].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    ax[2].vlines(0.13, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    twin2 = ax[2].twiny()
    twin2.plot(net, depth, color='orange', linewidth=0.5, alpha=1)
    twin2.fill_betweenx(depth, net, color='orange', alpha=0.33)
    twin2.text(0.25, depth.iloc[15], net_var, fontsize=12)
    twin2.set_xlim(0, 1)
    twin2.set_ylim(min(depth), max(depth))
    twin2.xaxis.set_ticks([])
    twin2.grid(False)
    twin3 = ax[2].twiny()
    twin3.plot(rdeep, depth, color='black', linewidth=1, alpha=1)
    twin3.set_xscale('log')
    twin3.set_xlim(1, 200)
    twin3.grid(False)
    ax[3].plot(perm, depth, color='purple', alpha=0.75)
    ax[3].plot(perm1, depth, color='purple', alpha=0.5)
    ax[3].plot(perm2, depth, color='purple', alpha=0.5)
    ax[3].set_xscale('log')
    ax[3].set_xlim(0.1, 1000)
    ax[3].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[3].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    # fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(round(max(kh.dropna()),0)) + ' ' + str(comments), fontsize=14)
    fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(comments), fontsize=14)
    fig.tight_layout()
    if print_flag == 'print':
        fig.savefig(path + wellname + '.png')
    else:
        pass
    plt.show()

# well_display_logs_net_corr(df_bal8_azr_v2, 'C34Z', vsh_cuttoff, 'Balakhany VIII', 'NET', 'df_interp', 'TST', 10, 'dontprint', 'C:\jupyter\SPP\inputoutput\plots\\')

In [ ]:
def histo_htst_bp(dataset):
    fig, ax = plt.subplots(1,2, figsize=(18,4))
    sns.histplot(dataset.htst, bins=25, kde=True, ax=ax[0], color='orange', label='htst_bp')
    sns.histplot(dataset.bot_htst, bins=50, kde=True, ax=ax[1], color='green')
    ax[0].set_xticks(np.arange(0, 30, 1))
    ax[1].set_xticks(np.arange(0, 50, 2))
    ax[0].set_title('htst_bp', fontsize=16)
    ax[1].set_title('bot_htst_bp', fontsize=16);
histo_htst_bp(ntd_top_vsh_bot8_bp) 
histo_htst_bp(ntd_top_phi_bot8_bp) 

In [ ]:
def htst_distrib_map_customize_bp(dataset, dataset_xy, percentiles, round, hist_flag, print_flag, path):
    def bins_calculation(dataset, percentiles):
        dataset_version = dataset[~(dataset.well.isin(['E20Y','E10Z', 'J13X']))]
        qunatiles = dataset_version['htst'].quantile(percentiles).reset_index()['htst']
        bins = [0, qunatiles.iloc[0].round(round), qunatiles.iloc[1].round(round), 100]
        dataset_version.loc[((dataset_version.htst > 0) & (dataset_version.htst <= bins[1])), 'htst_interv'] = f'{bins[0]}-{bins[1]}'
        dataset_version.loc[((dataset_version.htst > bins[1]) & (dataset_version.htst <= bins[2])), 'htst_interv'] = f'{bins[1]}-{bins[2]}'
        dataset_version.loc[((dataset_version.htst > bins[2]) & (dataset_version.htst <= bins[3])), 'htst_interv'] = f'{bins[2]}-{bins[3]}'
        return dataset_version, bins
    dataset_version, bins = bins_calculation(dataset, percentiles)
    print(bins)
    def display_histplot(dataset, bins, comment):
        fig, ax = plt.subplots(1, 2, figsize=(8, 2))
        custom_palette = {f'{bins[0]}-{bins[1]}': 'green', f'{bins[1]}-{bins[2]}': 'yellow', f'{bins[2]}-{bins[3]}': 'orange'}
        ax[0] = sns.histplot(data=dataset, x='htst', bins=50, kde=True, log_scale=False, ax=ax[0])
        ax[0].tick_params(axis='both', which='major', labelsize=8)
        ax[0].grid(True)
        ax[1] = sns.histplot(data=dataset, x='htst', bins=50, hue='htst_interv', kde=True, 
                            log_scale=False, ax=ax[1], palette=custom_palette)
        ax[1].tick_params(axis='both', which='major', labelsize=8)
        ax[1].grid(True)
        fig.suptitle(comment, fontsize=12) 
        plt.show();
    if hist_flag == 'hist':
        display_histplot(dataset_version, bins, 'Bal8 Azeri selected bins')
    else:
        pass

    phi8_range_test = dataset_version.groupby(['well','htst_interv']).agg({'htst':['sum','count']}).reset_index()
    phi8_range_test.columns = phi8_range_test.columns.map('_'.join)
    phi8_range_test = phi8_range_test.rename(columns={'well_':'well', 'htst_interv_':'htst_interv'})
    def ranged_data_transpose(dataset, var):
        df_lst = []
        for wellname in dataset.well.unique():
            data = dataset[dataset.well == wellname][['htst_interv', var]].T.reset_index()
            data.columns = data.iloc[0]
            data = data.iloc[1:]
            data['well'] = wellname
            df_lst.append(data)
        df = pd.concat(df_lst).reset_index(drop=True)
        df.insert(0, 'well', df.pop('well'))
        df_sum = dataset.groupby('well')['htst_sum'].sum()
        df = df.set_index('well').join(df_sum).reset_index()
        return df
    phit8_range_hcount_test = ranged_data_transpose(phi8_range_test, 'htst_count')

    xy8 = dataset_xy.groupby('well')[['TST','Xmean','Ymean']].agg({
                                                                    'Xmean': lambda x: x.iloc[0],
                                                                    'Ymean': lambda x: x.iloc[0],
                                                                    'TST': lambda x: x.iloc[-1] - x.iloc[0]
                                                                    }).dropna()
    phit8_range_hcount_xy_test = phit8_range_hcount_test.set_index('well').join(xy8).reset_index().dropna()
    phit8_range_hcount_xy_test = phit8_range_hcount_xy_test.rename(columns={'TST':'fmtst_gross'})
    phit8_range_hcount_xy_test['htst_sum_vv'] = phit8_range_hcount_xy_test['htst_sum'] / phit8_range_hcount_xy_test['fmtst_gross'].round(2)

    def piechart_map_version(dataset_map, bins, comment, print_flag, path):
        fig, ax = plt.subplots(figsize=(15,10))
        for ind, row in dataset_map.iterrows():
                ax.pie([row[f'{bins[0]}-{bins[1]}'], row[f'{bins[1]}-{bins[2]}'], row[f'{bins[2]}-{bins[3]}']], 
                        radius=row['htst_sum']*0.0025, center=(row['Xmean']/1000, row['Ymean']/1000), wedgeprops={"linewidth": 0.5, "edgecolor": "gray", "alpha":0.75},
                        colors=['green', 'yellow','orange'], frame=True)
                ax.scatter(x = dataset_map['Xmean']/1000, y = dataset_map['Ymean']/1000, s=0.5, c='gray') 
                # ax.text(row['Xmean']/1000, row['Ymean']/1000, row['well'], fontsize=7, ha='left', va='bottom')
                if ind % 2 == 0:
                    ax.text(row['Xmean']/1000+0.1, row['Ymean']/1000+0.1, row['well'], fontsize=6, ha='left', va='bottom')
                else:
                    ax.text(row['Xmean']/1000-0.1, row['Ymean']/1000-0.1, row['well'], fontsize=6, ha='left', va='bottom')
        stats = (' orange = htst '+ f'{bins[2]}-{bins[3]}' 
                '\n lightgreen = htst '+ f'{bins[1]}-{bins[2]}' 
                '\n green = htst ' + f'{bins[0]}-{bins[1]}')
        bbox = dict(boxstyle='round', fc='white', ec='black', alpha=1)
        ax.text(0.8, 0.85, stats, fontsize=8, bbox=bbox, transform=ax.transAxes, horizontalalignment='left')
        ax.set_title(comment, fontsize=20)
        if print_flag == 'print':
            plt.savefig(path, format='pdf', bbox_inches='tight')
        plt.show();       
    piechart_map_version(phit8_range_hcount_xy_test, bins, f'Bal8 piechart test:htst_count, size:htst_sum, percentiles: {percentiles}', print_flag, path)
    return dataset_version, phit8_range_hcount_xy_test, bins

ntd_top_phi_bot8_bp = ntd_top_phi_bot8_bp[ntd_top_phi_bot8_bp.htst >= 1]
percentiles = [0.3, 0.8]
dataset_version8_bp, vsh8_range_bp, bins = htst_distrib_map_customize_bp(ntd_top_phi_bot8_bp, df_bal8_azr_vshclp2, percentiles, 0, 'donthist', 'dontprint', 
                                                     'C:\\jupyter\\SPP\plots\\bal8_htst_count_bp.pdf')

In [ ]:
fig, ax = plt.subplots(figsize=(18,6))
sns.scatterplot(data=dataset_version8_bp[dataset_version8_bp.phit_avg>0], x='phit_avg', y='htst', hue='htst_interv', alpha=0.5)

In [73]:
dataset_version8_bp.to_csv(r'C:\jupyter\SPP\inputoutput\dataset_version8_bp.csv')

### Htst distributiom Bal8 gr_cube

In [ ]:
df_cube8 = df_cube[(df_cube.FORMATION_up == 'Balakhany VIII') & ((df_cube.well.str.startswith('C')) |
                                                                 (df_cube.well.str.startswith('B')) |
                                                                 (df_cube.well.str.startswith('D')))]
def interpolate_gr_cube(df, step):
    df_tst = df[df.TST.notna()].round({'MD':1})
    
    def interpolate_by_depth_fm(one_well, step):
        one_well = one_well.sort_values(by='TST')
        well_name = one_well["well"].iloc[0]
        data_range = np.floor((one_well["TST"].max() - one_well["TST"].min())/step)
        starting_tst = one_well["TST"].iloc[0]
        new_TST_values = [starting_tst + i*0.1 for i in range(1,int(data_range))]
        interp_MD = interp1d(one_well['TST'], one_well['MD'], kind='linear', fill_value="extrapolate")
        interp_NET = interp1d(one_well['TST'], one_well['NET'], kind='linear', fill_value="extrapolate")
        interp_PHIT = interp1d(one_well['TST'], one_well['PHIT_cube'], kind='linear', fill_value="extrapolate")
        interp_GR = interp1d(one_well['TST'], one_well['GR_cube'], kind='linear', fill_value="extrapolate")
        interp_VSH = interp1d(one_well['TST'], one_well['VSH_cube'], kind='linear', fill_value="extrapolate")
   
        # Create a new DataFrame with the interpolated values for new TVD_SCS
        new_data = {
            'well': [well_name for _ in range(len(new_TST_values))],
            'tst_index': [_ for _ in range(len(new_TST_values))],
            'MD': interp_MD(new_TST_values),
            'TST': new_TST_values,
            'NET': interp_NET(new_TST_values),
            'PHIT_cube': interp_PHIT(new_TST_values),
            'GR_cube': interp_GR(new_TST_values),
            'VSH_cube': interp_VSH(new_TST_values),
        }
        new_df = pd.DataFrame(new_data)
        return new_df
    df_lst = []
    print('Start interpolation')
    for wellnames in tqdm(df_tst.well.unique()):
        well_sel = df_tst[df_tst.well == wellnames]
        well_interp = interpolate_by_depth_fm(well_sel, step)
        df_lst.append(well_interp)
    df_interp = pd.concat(df_lst)
    df_interp = df_interp.round({'MD':1, 'TVD_SCS':1, 'TST':1})

    print('Start joining')
    def well_bal_interp_join(dataset):
        df_tst = df[(df.TST.notna()) & (df.FORMATION_up.notna())].round({'MD':1})
        data_fu = df_tst[['well','MD','FORMATION_up', 'FORMATION']]
        well_join = dataset.set_index(['well','MD']).join(data_fu.set_index(['well','MD'])).reset_index()
        well_join.insert(3, 'FORMATION_up', well_join.pop('FORMATION_up'))
        well_join.insert(4, 'FORMATION', well_join.pop('FORMATION'))
        well_join.insert(5, 'tst_index', well_join.pop('tst_index'))
        return well_join
    well_interp_v2 = well_bal_interp_join(df_interp)
    # well_interp_v2.loc[well_interp_v2.NET_VSH > 0, 'NET_VSH'] = 1
    well_interp_v2.loc[well_interp_v2.NET > 0, 'NET'] = 1
    return well_interp_v2
df_cube8_interp = interpolate_gr_cube(df_cube8, 0.1)
df_cube8_interp['FORMATION_up'] = df_cube8_interp['FORMATION_up'].fillna(method='ffill')
df_cube8_interp['FORMATION'] = df_cube8_interp['FORMATION'].fillna(method='ffill')

def well_gr_cube_display(dataset, formation, var, well):
    well_sel = dataset[(dataset.well == well) & (dataset.FORMATION_up == formation)]
    depth = well_sel['TST']
    grn = well_sel['VSH_cube']
    phit = well_sel['PHIT_cube']
    # rdeep = well_sel['RDEEP']
    net = well_sel[var]*100
    phit_cutoff = [0.13 for i in range(len(depth))]
    fig = go.Figure()
    fig = make_subplots(rows=1, cols=2, subplot_titles=('GR', 'PHIT', 'RDEEP'))
    fig.add_trace(go.Scatter(x=grn, y=depth, mode='lines', name='VSH_cube', 
                             line=dict(color='green')), row=1, col=1)
    
    fig.update_xaxes(range = [0,1])
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  
                             line=dict(color='rgba(245, 190, 20, 0.75)'), 
                             fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=1)
    fig.update_xaxes(range = [0, 1], row=1, col=1)

    fig.add_trace(go.Scatter(x=phit, y=depth, mode='lines', name='PHIT', 
                            line = dict(color='firebrick', width=2)), row=1, col=2)
    fig.add_trace(go.Scatter(x=phit_cutoff, y=depth, mode='lines', name='PHIT', 
                            line = dict(color='black', width=2, dash='dash')), row=1, col=2)
    fig.update_xaxes(range = [0.35, 0], row=1, col=2)
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  
                             marker=dict(color='rgba(245, 190, 20, 0.75)'), 
                             fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=2)
    # fig.add_trace(go.Scatter(x=rdeep, y=depth, mode='lines', name='RDEEP', 
    #                         line = dict(color='black', width=2, dash='dash')), row=1, col=3)
    # fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  
    #                          marker=dict(color='rgba(245, 190, 20, 0.75)'), 
    #                          fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=3)
    # fig.update_xaxes(range = [0, 2], type='log', row=1, col=3)
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(height=850, width=400,
                      title=dict(text=well, font=dict(size=24)),
                      showlegend=False,
                      margin=dict(l=10,r=10,b=10,t=50))
    fig.show()
# well_gr_cube_display(df_cube8_interp, 'Balakhany VIII','NET', 'B43')

In [232]:
df_cube8_interp_copy = df_cube8_interp.copy()
def comparison_gr_cube_bal8(df_cube, df_bal8):
    print('Wells in gr_cube not in df_bal8')   
    for wellcube in df_cube.well.unique():
        if wellcube in df_bal8.well.unique():
            pass
        else:
            print(wellcube)
    print('Wells in df_bal8 not in gr_cube')  
    for welllog in df_bal8.well.unique():
        if welllog in df_cube.well.unique():
            pass
        else:
            print(welllog)
# comparison_gr_cube_bal8(df_cube8_interp, df_bal8_azr_v2)

In [ ]:
def gr_cube_vsh_cutoff(dataset_cube,dataset_logs, vsh_cutoff):
    dataset_cube['NET'] = 0
    dataset_cube.loc[dataset_cube['VSH_cube'] <= vsh_cutoff, 'NET'] = 1

    def ntd_dataframe_calculation_cube(dataset, fm, net_var):
        print(f'Calculation NTD based on {net_var} with cutoff {vsh_cutoff}')
        def ntd_calculation(dataset, well, fm, net_var):
            data = dataset[(dataset.well==well)]
            data.iloc[0, 3] = 0
            data.iloc[-1, 3] = 0
            tst_top = [data.iloc[i]['TST'] for i in range(len(data)-1)
                        if (data.iloc[i][net_var] == 1 and data.iloc[i-1][net_var]==0)]
            tst_bot = [data.iloc[i]['TST'] for i in range(len(data)-1)
                        if (data.iloc[i][net_var] == 1 and data.iloc[i+1][net_var]==0)]
            tops = zip(tst_top, tst_bot)
            df_htst = pd.DataFrame(tops, columns=['tst_top', 'tst_bot'])
            df_htst['FORMATION_up'] = fm
            df_htst['well'] = well
            df_htst['h_tst'] = df_htst.tst_bot - df_htst.tst_top
            df_htst = df_htst[['well','FORMATION_up','tst_top','tst_bot','h_tst']]
            return df_htst
        df_recalc_list = []
        for well in tqdm(dataset.well.unique()):
            df = ntd_calculation(dataset, well, fm, net_var)
            df_recalc_list.append(df)
        ntd_net2 = pd.concat(df_recalc_list)
        ntd_net2.drop_duplicates(inplace=True)

        print('Calculation values for NTD')
        def ntd_properties_averaging(dataset_ntd, dataset_logs, fm):
            well_data = []
            well_formation = fm
            df_lst = []
            for well in tqdm(dataset_ntd.well.unique()):
                ntd_well_avgprop = dataset_ntd[(dataset_ntd.well ==well)]
                well_avgprop_sel = dataset_logs[(dataset_logs.well==well)]
                fm_top = dataset_logs[(dataset_logs.well==well)]['TST'].iloc[0]
                fm_bot = dataset_logs[(dataset_logs.well==well)]['TST'].iloc[-1]
                well_phit = []
                well_vsh = []
                well_gperm = []
                well_top = []
                well_bot = []
                well_h = []
                well_fm_top = []
                well_fm_bot = []
                well_name = []
                well_fm = []
                for layers in range(len(ntd_well_avgprop.well)):
                    ntd_top = ntd_well_avgprop.iloc[layers, 2].round(3)
                    ntd_bot = ntd_well_avgprop.iloc[layers, 3].round(3)
                    ntd_h = ntd_well_avgprop.iloc[layers, 4].round(3)
                    phit_lst = []
                    vsh_lst = []
                    perm_lst = []
                    for depth in range(len(well_avgprop_sel.TST)):
                        well_avgprop_tst = well_avgprop_sel['TST'].iloc[depth].round(3)
                        if well_avgprop_tst >= ntd_top and well_avgprop_tst <= ntd_bot:
                            phit_lst.append(well_avgprop_sel['PHIT'].iloc[depth])
                            vsh_lst.append(well_avgprop_sel['VSH'].iloc[depth])
                            perm_lst.append(well_avgprop_sel['LPERM'].iloc[depth])
                    well_name.append(well)
                    well_fm.append(well_formation)
                    well_phit.append(mean(phit_lst))
                    well_vsh.append(mean(vsh_lst))
                    well_gperm.append(gmean(perm_lst))
                    well_h.append(ntd_h)
                    well_top.append(ntd_top)
                    well_bot.append(ntd_bot)
                    well_fm_top.append(fm_top)
                    well_fm_bot.append(fm_bot)
                    well_data = zip(well_name,well_fm,well_phit, well_vsh, well_gperm, well_h, well_top, well_bot, well_fm_top, well_fm_bot)
                    well_df = pd.DataFrame(well_data, columns=[ 'well','FORMATION_up',        
                                                                'phit_avg',
                                                                'vsh_avg', 
                                                                'perm_avg',
                                                                'htst',
                                                                'top_tst',
                                                                'bot_tst',
                                                                'fm_top_tst',
                                                                'fm_bot_tst'])
                    well_df['not_htst'] = well_df['top_tst'].shift(-1)-well_df['bot_tst']
                    well_df = well_df[['well', 'FORMATION_up', 'phit_avg', 'vsh_avg', 'perm_avg', 'htst', 'not_htst','top_tst', 'bot_tst', 'fm_top_tst', 'fm_bot_tst']]
                df_lst.append(well_df)
            result = pd.concat(df_lst)
            return result
        # ntd_val = ntd_properties_averaging(ntd_net2, dataset, fm)

        def nothtst_nan_filling(dataset_ntd, fm):
            def nan_change_diff_fmbottom(dataset, wellname, fmname):
                row_change = dataset[(dataset.well == wellname) & (dataset.FORMATION_up == fmname) & (dataset.not_htst.isna())]
                row_change['not_htst'] = row_change['fm_bot_tst'] - row_change['bot_tst']
                return row_change
            df_list = []
            for wellname in dataset_ntd.well.unique():
                df = nan_change_diff_fmbottom(dataset_ntd, wellname, fm)
                df_list.append(df)
            res_df_list = pd.concat(df_list)
            result = pd.concat([dataset_ntd, res_df_list])
            result = result.sort_values(by=['well','top_tst'])
            result_final = result.dropna(subset=['not_htst'], axis=0)

            return result_final
        # ntd_val_clean = nothtst_nan_filling(ntd_val, fm)

        def top_phit_bot_clustering(dataset):
            print('Top & bot calculation')
            def top_phit_bot_collection_run(dataset):
                def top_phit_bot_collection(dataset, wellname):
                    data = dataset[dataset.well == wellname]
                    data['top_htst'] = data['top_tst'] - data['fm_top_tst']
                    data['top_htst'].iloc[1:] = data['not_htst'].iloc[:-1]
                    data['bot_htst'] = data['not_htst']
                    data = data[['well', 'FORMATION_up', 'phit_avg', 'vsh_avg', 'perm_avg',
                                'top_htst','htst','bot_htst', 'fm_top_tst', 'fm_bot_tst']]
                    return data
                df_lst = []
                for wellname in tqdm(dataset.well.unique()):
                    res_df = top_phit_bot_collection(dataset, wellname)
                    df_lst.append(res_df)
                top_phi_bot_cluster = pd.concat(df_lst).reset_index(drop=True)
                return top_phi_bot_cluster
            top_phi_bot_cluster = top_phit_bot_collection_run(dataset)

            def top_phit_bot_ntg_run(dataset):
                def top_phit_bot_ntg(dataset, wellname):
                    ntg = []
                    data = dataset[dataset.well == wellname].reset_index(drop=True)
                    for ind, row in data.iterrows():
                        if ind == 0:
                            ntg.append(row['htst']/(row['bot_htst'] + row['htst']))
                        if ind != 0:
                            ntg.append(row['htst']/(row['bot_htst'] + row['htst'] + row['top_htst']))
                        if ind == len(data):
                            ntg.append(row['htst']/(row['top_htst'] + row['htst']))
                    result = pd.concat([data, pd.DataFrame({'ntg':ntg})], axis=1)
                    return result
                df_lst = []
                for wellname in dataset.well.unique():
                    df = top_phit_bot_ntg(dataset, wellname)
                    df_lst.append(df)
                top_phi_bot_cluster_ntg = pd.concat(df_lst).reset_index(drop=True)
                return top_phi_bot_cluster_ntg
            # top_phi_bot_cluster_ntg = top_phit_bot_ntg_run(top_phi_bot_cluster)
            
            return top_phi_bot_cluster
        # ntd_top_phi_bot = top_phit_bot_clustering(ntd_val_clean)

        # return ntd_top_phi_bot.reset_index(drop=True)
        return ntd_net2.reset_index(drop=True)
    ntd_top_cube_bot8 = ntd_dataframe_calculation_cube(dataset_cube, 'Balakhany VIII', 'NET')
    ntd_top_cube_bot8 = ntd_top_cube_bot8.rename(columns={'h_tst':'htst'})

    def groupby_quantiles(dataset, q1, q2, comment):
        percentiles = dataset.groupby('well')['htst'].agg(
            ['sum', 'mean', lambda x: x.quantile(q1), lambda x: x.quantile(q2)]).rename(
            columns={'sum':f'sum_{comment}',
                    'mean':f'avg_{comment}',
                    '<lambda_0>':f'q{int(q1*100)}_{comment}', 
                    '<lambda_1>':f'q{int(q2*100)}_{comment}'
                    }).reset_index()
        # percentiles['dataset'] = comment
        return percentiles
    htst_cube8 = groupby_quantiles(ntd_top_cube_bot8, 0.25, 0.75, 'cube')
    htst_phi8 = groupby_quantiles(dataset_logs[dataset_logs.htst>=1], 0.25, 0.75, 'logsbp')
    join_df = htst_cube8.set_index('well').join(htst_phi8.set_index('well')).reset_index().dropna()
    
    def scatterplot_display_sns():
        fig, ax = plt.subplots(1,2, figsize=(15,5))
        sns.scatterplot(data=join_df, x='avg_cube', y='avg_logsbp', alpha=0.5, color='red', label='avg', ax=ax[0])
        sns.scatterplot(data=join_df, x='q25_cube', y='q25_logsbp', alpha=0.5, color='green', label='q25', ax=ax[0])
        sns.scatterplot(data=join_df, x='q75_cube', y='q75_logsbp', alpha=0.5, color='blue', label='q75', ax=ax[0])
        max_val = join_df.q75_cube.max().round(1)
        sns.lineplot(x = [0, max_val], y = [0, max_val*1.25], color='black', linestyle='--', ax=ax[0])
        sns.lineplot(x = [0, max_val], y = [0, max_val], color='black', linestyle='--', ax=ax[0])
        sns.lineplot(x = [0, max_val], y = [0, max_val*0.75], color='black', linestyle='--', ax=ax[0])
        ax[0].set_title(f'Comparison of htst vsh_cutoff={vsh_cutoff}, max_avl={max_val}', fontsize=14)
        
        max_val_sum = join_df.sum_cube.max().round(1)
        sns.scatterplot(data=join_df, x='sum_cube', y='sum_logsbp', alpha=0.5, color='orange', label='sum', ax=ax[1], s=50)
        sns.lineplot(x = [0, max_val_sum], y = [0, max_val_sum*1.25], color='black', linestyle='--', ax=ax[1])
        sns.lineplot(x = [0, max_val_sum], y = [0, max_val_sum], color='black', linestyle='--', ax=ax[1])
        sns.lineplot(x = [0, max_val_sum], y = [0, max_val_sum*0.75], color='black', linestyle='--', ax=ax[1])
        ax[1].set_title(f'Comparison of htst_sum vsh_cutoff={vsh_cutoff}, max_avl={max_val_sum}', fontsize=14)
        plt.title(f'Comparison of htst vsh_cutoff={vsh_cutoff}, max_avl={max_val}', fontsize=14);
        plt.show()

    def scatterplot_display(join_df):
        max_val = join_df.q75_cube.max().round(1)
        max_val_sum = join_df.sum_cube.max().round(1)
        fig = make_subplots(rows=1, cols=2, subplot_titles=(f'Comparison of htst vsh_cutoff={vsh_cutoff}, max_avl={max_val}',
                                                            f'Comparison of htst_sum vsh_cutoff={vsh_cutoff}, max_avl={max_val_sum}'))
        avg = go.Scatter(x=join_df['avg_cube'], y=join_df['avg_logsbp'], mode='markers', 
                            marker=dict(size=8, color='red', opacity=0.5,
                            line=dict(color='black', width=0.5)),
                            customdata = join_df[['well', 'avg_cube', 'avg_logsbp']].round(1),
                            hovertemplate="".join(["well:%{customdata[0]}, %{customdata[1]}, %{customdata[2]}<extra></extra>"]), 
                            name='avg')
        q25 = go.Scatter(x=join_df['q25_cube'], y=join_df['q25_logsbp'], mode='markers', 
                            marker=dict(size=8, color='green', opacity=0.5,
                            line=dict(color='black', width=0.5)),
                            customdata = join_df[['well', 'q25_cube', 'q25_logsbp']].round(1),
                            hovertemplate="".join(["well:%{customdata[0]}, %{customdata[1]}, %{customdata[2]}<extra></extra>"]), 
                            name='q25')
        q75 = go.Scatter(x=join_df['q75_cube'], y=join_df['q75_logsbp'], mode='markers', 
                            marker=dict(size=8, color='blue', opacity=0.5,
                            line=dict(color='black', width=0.5)),
                            customdata = join_df[['well', 'q75_cube', 'q75_logsbp']].round(1),
                            hovertemplate="".join(["well:%{customdata[0]}, %{customdata[1]}, %{customdata[2]}<extra></extra>"]), 
                            name='q75')
        sum_htst = go.Scatter(x=join_df['sum_cube'], y=join_df['sum_logsbp'], mode='markers', 
                            marker=dict(size=8, color='orange', opacity=0.5,
                            line=dict(color='black', width=0.5)),
                            customdata = join_df[['well', 'sum_cube', 'sum_logsbp']].round(1),
                            hovertemplate="".join(["well:%{customdata[0]}, %{customdata[1]}, %{customdata[2]}<extra></extra>"]), 
                            name='sum_htst')

        line1 = go.Scatter(x = [0, max_val], y = [0, max_val*1.25], mode='lines', line=dict(color='gray', dash='dash'), name='1.25')
        line2 = go.Scatter(x = [0, max_val], y = [0, max_val], mode='lines', line=dict(color='black', dash='dot'), name='1 vs 1')
        line3 = go.Scatter(x = [0, max_val], y = [0, max_val*0.75], mode='lines', line=dict(color='gray', dash='dash'),  name='0.75')

        line1_sum = go.Scatter(x = [0, max_val_sum], y = [0, max_val_sum*1.25], mode='lines', line=dict(color='gray', dash='dash'), name='1.25')
        line2_sum = go.Scatter(x = [0, max_val_sum], y = [0, max_val_sum], mode='lines', line=dict(color='black', dash='dot'), name='1 vs 1')
        line3_sum = go.Scatter(x = [0, max_val_sum], y = [0, max_val_sum*0.75], mode='lines', line=dict(color='gray', dash='dash'),  name='0.75')
        fig.add_trace(avg, row=1, col=1)
        fig.add_trace(q25, row=1, col=1)
        fig.add_trace(q75, row=1, col=1)
        fig.add_trace(line1, row=1, col=1)
        fig.add_trace(line2, row=1, col=1)
        fig.add_trace(line3, row=1, col=1)
        fig.add_trace(sum_htst, row=1, col=2)
        fig.add_trace(line1_sum, row=1, col=2)
        fig.add_trace(line2_sum, row=1, col=2)
        fig.add_trace(line3_sum, row=1, col=2)
        fig.update_xaxes(title_text="cube", row=1, col=1)
        fig.update_yaxes(title_text="logs", row=1, col=1)
        fig.update_xaxes(title_text="cube", row=1, col=2)
        fig.update_yaxes(title_text="logs", row=1, col=2)
        fig.update_layout(
            autosize=False,
            width=1200,
            height=500,
            margin=dict(l=10, r=10, b=10, t=30),
            showlegend=True
        )
        fig.show()
    
    scatterplot_display(join_df)
    return ntd_top_cube_bot8
ntd_top_cube_bot8_05 = gr_cube_vsh_cutoff(df_cube8_interp_copy, ntd_top_phi_bot8_bp, 0.5)
ntd_top_cube_bot8_03 = gr_cube_vsh_cutoff(df_cube8_interp_copy, ntd_top_phi_bot8_bp, 0.3)
ntd_top_cube_bot8_035 = gr_cube_vsh_cutoff(df_cube8_interp_copy, ntd_top_phi_bot8_bp, 0.35)
ntd_top_cube_bot8_04 = gr_cube_vsh_cutoff(df_cube8_interp_copy, ntd_top_phi_bot8_bp, 0.4)

In [53]:
well = 'B04Z'
def well_gr_cube_display(dataset, formation, var_gr, var_phit, var_net, well):
    well_sel = dataset[(dataset.well == well) & (dataset.FORMATION_up == formation)]
    depth = well_sel['TST']
    grn = well_sel[var_gr]
    phit = well_sel[var_phit]
    # rdeep = well_sel['RDEEP']
    net = well_sel[var_net]*100
    phit_cutoff = [0.13 for i in range(len(depth))]
    fig = go.Figure()
    fig = make_subplots(rows=1, cols=2, subplot_titles=(var_gr, var_phit))
    fig.add_trace(go.Scatter(x=grn, y=depth, mode='lines', name='GR', 
                             line=dict(color='green')), row=1, col=1)
    
    fig.update_xaxes(range = [0,1])
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  
                             line=dict(color='rgba(245, 190, 20, 0.75)'), 
                             fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=1)
    fig.update_xaxes(range = [0, 1], row=1, col=1)

    fig.add_trace(go.Scatter(x=phit, y=depth, mode='lines', name='PHIT', 
                            line = dict(color='firebrick', width=2)), row=1, col=2)
    fig.add_trace(go.Scatter(x=phit_cutoff, y=depth, mode='lines', name='PHIT', 
                            line = dict(color='black', width=2, dash='dash')), row=1, col=2)
    fig.update_xaxes(range = [0.35, 0], row=1, col=2)
    fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  
                             marker=dict(color='rgba(245, 190, 20, 0.75)'), 
                             fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=2)
    # fig.add_trace(go.Scatter(x=rdeep, y=depth, mode='lines', name='RDEEP', 
    #                         line = dict(color='black', width=2, dash='dash')), row=1, col=3)
    # fig.add_trace(go.Scatter(x=net, y=depth, mode='lines', name='NET',  
    #                          marker=dict(color='rgba(245, 190, 20, 0.75)'), 
    #                          fill='tozerox', fillcolor='rgba(245, 190, 20, 0.25)'), row=1, col=3)
    fig.update_xaxes(range = [0, 2], type='log', row=1, col=3)
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(height=600, width=400,
                      title=dict(text=well, font=dict(size=24)),
                      showlegend=False,
                      margin=dict(l=10,r=10,b=10,t=50),
                      xaxis2=dict(  title='xaxis2 title',
                                    overlaying='x2',
                                    side='right'))
    fig.show()
# well_gr_cube_display(df_cube8_interp, 'Balakhany VIII','VSH_cube','PHIT_cube','NET', well)
# well_gr_cube_display(df_bal8_azr_v2, 'Balakhany VIII','VSH','PHIT','NET', well)

In [ ]:
test = df_cube8_interp[df_cube8_interp.FORMATION != 'Balakhany VIII']
test2 = df_bal8_azr_v2[df_bal8_azr_v2.FORMATION != 'Balakhany VIII']
print(test[test.well == 'C03']['NET'].sum(), test2[test2.well == 'C03']['NET'].sum())

In [ ]:
def gr_cube_htst_matrix(dataset_cube, dataset_logs, letters_list, rows, columns, comment, output_flag):
    """
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J']
    """
    for letter in letters_list:
        wells_letter = [wellname for wellname in dataset_cube.well.unique() if wellname.startswith(letter)]
        fig, ax = plt.subplots(rows,columns, figsize=(16,rows*2.5))
        counter = 0
        for j in range(0, rows):
            for i in range(0, columns):
                if counter < len(wells_letter):
                    wellname = wells_letter[counter]
                    welldata_cube = dataset_cube[dataset_cube.well==wellname]
                    welldata_logs = dataset_logs[dataset_logs.well==wellname]
                    depth_cube = welldata_cube['tst_index']
                    net_cube = welldata_cube['NET']+1
                    depth_logs = welldata_logs['tst_index']
                    net_logs = welldata_logs['NET']
                    net_cube_sum = (welldata_cube['NET'].sum()*0.1).round(0)
                    net_logs_sum = (welldata_logs['NET'].sum()*0.1).round(0)
                    ax[j,i].plot(net_cube, depth_cube, color='green', alpha=1, lw=0.5)
                    ax[j,i].fill_betweenx(depth_cube, 1, net_cube, color='green', alpha=0.5)
                    ax[j,i].text(1, depth_cube.iloc[0]+50, net_cube_sum, fontsize=12)
                    ax[j,i].plot(net_logs, depth_logs, color='orange', alpha=0.5, lw=1)
                    ax[j,i].fill_betweenx(depth_logs, net_logs, color='orange', alpha=0.5)
                    ax[j,i].text(0, depth_cube.iloc[0]+50, net_logs_sum, fontsize=12)
                    ax[j,i].set_xlim(0, 2)
                    ax[j,i].invert_yaxis()
                    ax[j,i].set_title(wellname)
                    ax[j,i].grid(False)
                    fig.tight_layout()
                    fig.suptitle(comment)
                    counter +=1
        if output_flag == 'print':
            plt.savefig('.\plots\\gr_cube8_' + str(letter) +'.pdf')
        else:
            pass
        plt.show()

vsh_cutoff = 0.4
df_cube8_interp['NET'] = 0
df_cube8_interp.loc[df_cube8_interp['VSH_cube'] <= vsh_cutoff, 'NET'] = 1

gr_cube_htst_matrix(df_cube8_interp[df_cube8_interp.FORMATION != 'Balakhany VIII'], 
                    df_bal8_azr_v2[df_bal8_azr_v2.FORMATION != 'Balakhany VIII'],
                    # ['C'], 2, 9, 
                    ['C','B','D'], 7, 9, 
                    'Logs-orange, cube-green',
                    'dontprint')

In [ ]:
def htst_distrib_map_customize(dataset, dataset_xy, percentiles, hist_flag, print_flag, path):
    def bins_calculation(dataset, percentiles):
        dataset_version = dataset[~(dataset.well.isin(['E20Y','E10Z', 'J13X']))]
        qunatiles = dataset_version['htst'].quantile(percentiles).reset_index()['htst']
        bins = [0, int(qunatiles.iloc[0].round(0)), int(qunatiles.iloc[1].round(0)), 100]
        dataset_version.loc[((dataset_version.htst > 0) & (dataset_version.htst <= bins[1])), 'htst_interv'] = f'{bins[0]}-{bins[1]}'
        dataset_version.loc[((dataset_version.htst > bins[1]) & (dataset_version.htst <= bins[2])), 'htst_interv'] = f'{bins[1]}-{bins[2]}'
        dataset_version.loc[((dataset_version.htst > bins[2]) & (dataset_version.htst <= bins[3])), 'htst_interv'] = f'{bins[2]}-{bins[3]}'
        return dataset_version, bins
    dataset_version, bins = bins_calculation(dataset, percentiles)
    print(bins)
    def display_histplot(dataset, bins, comment):
        fig, ax = plt.subplots(1, 2, figsize=(8, 2))
        custom_palette = {f'{bins[0]}-{bins[1]}': 'green', f'{bins[1]}-{bins[2]}': 'yellow', f'{bins[2]}-{bins[3]}': 'orange'}
        ax[0] = sns.histplot(data=dataset, x='htst', bins=50, kde=True, log_scale=False, ax=ax[0])
        ax[0].tick_params(axis='both', which='major', labelsize=8)
        ax[0].grid(True)
        ax[1] = sns.histplot(data=dataset, x='htst', bins=50, hue='htst_interv', kde=True, 
                            log_scale=False, ax=ax[1], palette=custom_palette)
        ax[1].tick_params(axis='both', which='major', labelsize=8)
        ax[1].grid(True)
        fig.suptitle(comment, fontsize=12) 
        plt.show();
    if hist_flag == 'hist':
        display_histplot(dataset_version, bins, 'Bal8 Azeri selected bins')
    else:
        pass

    phi8_range_test = dataset_version.groupby(['well','htst_interv']).agg({'htst':['sum','count']}).reset_index()
    phi8_range_test.columns = phi8_range_test.columns.map('_'.join)
    phi8_range_test = phi8_range_test.rename(columns={'well_':'well', 'htst_interv_':'htst_interv'})
    def ranged_data_transpose(dataset, var):
        df_lst = []
        for wellname in dataset.well.unique():
            data = dataset[dataset.well == wellname][['htst_interv', var]].T.reset_index()
            data.columns = data.iloc[0]
            data = data.iloc[1:]
            data['well'] = wellname
            df_lst.append(data)
        df = pd.concat(df_lst).reset_index(drop=True)
        df.insert(0, 'well', df.pop('well'))
        df_sum = dataset.groupby('well')['htst_sum'].sum()
        df = df.set_index('well').join(df_sum).reset_index()
        return df
    phit8_range_hcount_test = ranged_data_transpose(phi8_range_test, 'htst_count')

    xy8 = dataset_xy.groupby('well')[['TST','Xmean','Ymean']].agg({
                                                                    'Xmean': lambda x: x.iloc[0],
                                                                    'Ymean': lambda x: x.iloc[0],
                                                                    'TST': lambda x: x.iloc[-1] - x.iloc[0]
                                                                    }).dropna()
    phit8_range_hcount_xy_test = phit8_range_hcount_test.set_index('well').join(xy8).reset_index().dropna()
    phit8_range_hcount_xy_test = phit8_range_hcount_xy_test.rename(columns={'TST':'fmtst_gross'})
    phit8_range_hcount_xy_test['htst_sum_vv'] = phit8_range_hcount_xy_test['htst_sum'] / phit8_range_hcount_xy_test['fmtst_gross'].round(2)

    def piechart_map_version(dataset_map, bins, comment, print_flag, path):
        fig, ax = plt.subplots(figsize=(15,10))
        for ind, row in dataset_map.iterrows():
                ax.pie([row[f'{bins[0]}-{bins[1]}'], row[f'{bins[1]}-{bins[2]}'], row[f'{bins[2]}-{bins[3]}']], 
                        radius=row['htst_sum']*0.0025, center=(row['Xmean']/1000, row['Ymean']/1000), wedgeprops={"linewidth": 0.5, "edgecolor": "gray", "alpha":0.75},
                        colors=['green', 'yellow','orange'], frame=True)
                ax.scatter(x = dataset_map['Xmean']/1000, y = dataset_map['Ymean']/1000, s=0.5, c='black') 
                # ax.text(row['Xmean']/1000, row['Ymean']/1000, row['well'], fontsize=7, ha='left', va='bottom')
                if ind % 2 == 0:
                    ax.text(row['Xmean']/1000+0.1, row['Ymean']/1000+0.1, row['well'], fontsize=6, ha='left', va='bottom')
                else:
                    ax.text(row['Xmean']/1000-0.1, row['Ymean']/1000-0.1, row['well'], fontsize=6, ha='left', va='bottom')
        stats = (' orange = htst '+ f'{bins[2]}-{bins[3]}' 
                '\n lightgreen = htst '+ f'{bins[1]}-{bins[2]}' 
                '\n green = htst ' + f'{bins[0]}-{bins[1]}')
        bbox = dict(boxstyle='round', fc='white', ec='black', alpha=1)
        ax.text(0.8, 0.85, stats, fontsize=8, bbox=bbox, transform=ax.transAxes, horizontalalignment='left')
        ax.set_title(comment, fontsize=20)
        if print_flag == 'print':
            plt.savefig(path, format='pdf', bbox_inches='tight')
        plt.show();       
    piechart_map_version(phit8_range_hcount_xy_test, bins, f'Bal8 piechart test:htst_count, size:htst_sum, percentiles: {percentiles}', print_flag, path)
    
    return dataset_version, phit8_range_hcount_xy_test, bins

# ntd_top_phi_bot8_bp = ntd_top_phi_bot8_bp[ntd_top_phi_bot8_bp.htst >= 1]
percentiles = [0.4, 0.8]
dataset_version8_cube, vsh8_range_cube, bins_cube = htst_distrib_map_customize(ntd_top_cube_bot8_04, df_bal8_azr_vshclp2, percentiles, 'donthist', 'print', 
                                                     'C:\\jupyter\\SPP\plots\\bal8_htst_count_cube.pdf')

In [ ]:
def percenage_calculation(dataset, bins):
    dataset['bin_sum'] = (dataset[f'{bins[0]}-{bins[1]}'] + 
                          dataset[f'{bins[1]}-{bins[2]}'] + 
                          dataset[f'{bins[2]}-{bins[3]}'])
    dataset[f'{bins[0]}-{bins[1]}'+'%'] = dataset[f'{bins[0]}-{bins[1]}'] / dataset['bin_sum']
    dataset[f'{bins[1]}-{bins[2]}'+'%'] = dataset[f'{bins[1]}-{bins[2]}'] / dataset['bin_sum']
    dataset[f'{bins[2]}-{bins[3]}'+'%'] = dataset[f'{bins[2]}-{bins[3]}'] / dataset['bin_sum']
    return dataset
vsh8_range_cube = percenage_calculation(vsh8_range_cube, bins_cube)   

def facies_estimation(dataset, bins, htst_cutoff, bins_cutoff):
        dataset['geobody_td'] = 'proximal'
        well_axial = dataset[(dataset['htst_sum_vv'] >= htst_cutoff) & 
                                                (dataset[f'{bins[2]}-{bins[3]}'+'%'] >= bins_cutoff[1])].well.unique()
        well_distal = dataset[(dataset['htst_sum_vv'] < htst_cutoff) & 
                                                (dataset[f'{bins[0]}-{bins[1]}'+'%'] >= bins_cutoff[0])].well.unique()
        dataset.loc[dataset.well.isin(well_axial), 'geobody_td'] = 'axial'
        dataset.loc[dataset.well.isin(well_distal), 'geobody_td'] = 'distal'
        return dataset
vsh8_range_cube = facies_estimation(vsh8_range_cube, bins_cube, 0.3, [0.4, 0.3])

def gpd_buffer_map(dataset, bins, geobody, comment, print_flag, path):
    text_size = 6
    geometry = [Point(xy) for xy in zip(dataset['Xmean'], dataset['Ymean'])]
    gpd_facies_xy = gpd.GeoDataFrame(dataset, geometry=geometry).drop(['Xmean','Ymean'], axis=1).set_crs('EPSG:2499')

    gpd_facies_xy.loc[gpd_facies_xy.well.isin(['C21Z']), geobody] = 'axial'
    gpd_facies_xy.loc[gpd_facies_xy.well.isin(['C33Z', 'D05Z', 'D19Z','D23X']), geobody] = 'proximal'

    def gpd_buffer_calc(dataset, buff_radius):
        buffers = dataset.buffer(buff_radius)
        buffers = gpd.GeoDataFrame(geometry=buffers)
        dataset_buff = dataset.join(buffers, rsuffix='_buff')
        dataset_buff = gpd.GeoDataFrame(dataset_buff, geometry='geometry_buff').drop('geometry', axis=1).set_crs('EPSG:2499')
        return dataset_buff
    distal_facies_buff = gpd_buffer_calc(gpd_facies_xy[gpd_facies_xy[geobody] == 'distal'], 450)
    proximal_facies_buff = gpd_buffer_calc(gpd_facies_xy[gpd_facies_xy[geobody] == 'proximal'], 350)
    axial_facies_buff = gpd_buffer_calc(gpd_facies_xy[gpd_facies_xy[geobody] == 'axial'], 290)

    unified_axial = axial_facies_buff.unary_union
    unified_proximal = proximal_facies_buff.unary_union
    unified_distal = distal_facies_buff.unary_union

    diff_axial = gpd.GeoSeries(unified_axial).explode()
    diff_axial_ch = diff_axial.convex_hull
    diff_proximal = gpd.GeoSeries(unified_proximal).explode()
    diff_proximal_ch = diff_proximal.convex_hull
    unified_distal = gpd.GeoSeries(unified_distal).explode()
    unified_distal_ch = unified_distal.convex_hull

    fig, ax = plt.subplots(figsize=(15,10))
    radius_mult = 2
    # text_size = 7
    for ind, row in gpd_facies_xy.iterrows():
        ax.pie([row[f'{bins[0]}-{bins[1]}'], row[f'{bins[1]}-{bins[2]}'], row[f'{bins[2]}-{bins[3]}']], 
                radius=row['htst_sum']*radius_mult, center=(row['geometry'].x, row['geometry'].y), 
                wedgeprops={"linewidth": 0.5, "edgecolor": "gray", "alpha":1},
                colors=['green', 'yellow','orange'], frame=True)
    for ind, row in gpd_facies_xy.iterrows():
        if ind % 2 == 0:
            ax.text(row['geometry'].x, row['geometry'].y, row['well'], fontsize=text_size, ha='left', va='bottom')
        else:
            ax.text(row['geometry'].x, row['geometry'].y, row['well'], fontsize=text_size, ha='left', va='top')

    unified_distal_ch.plot(ax=ax, color='green', alpha=0.5, ec='gray', zorder=1)
    diff_proximal_ch.plot(ax=ax, color='yellow', alpha=0.5, ec='gray', zorder=2)
    diff_axial_ch.plot(ax=ax, color='orange', alpha=0.5, ec='gray', zorder=3)
    gpd_facies_xy.plot(ax=ax, color='black', markersize=2, zorder=4)

    stats = (   ' orange = axial' + 
                '\n yellow  = proximal' +
                '\n green  = distal')
    bbox = dict(boxstyle='round', fc='white', ec='black', alpha=1)
    ax.text(0.875, 0.9, stats, fontsize=10, bbox=bbox, transform=ax.transAxes, horizontalalignment='left')

    plt.title('Bal8 facies estimation ' + comment, fontsize=16)
    if print_flag == 'print':
        plt.savefig(path, format='pdf', bbox_inches='tight')
    plt.show();
    return gpd_facies_xy
gpd_facies_xy_cube = gpd_buffer_map(vsh8_range_cube, bins_cube, 'geobody_td','manual','print', 'C:\\jupyter\\SPP\plots\\bal8_facies_cube.pdf')

## Htst distribution Bal8 versions

In [ ]:
# 'C27X', 'C27Y','J09Z','J13X','C19Z'
def htst_distrib_map_customize(dataset, dataset_xy, percentiles, hist_flag, print_flag, path):
    def bins_calculation(dataset, percentiles):
        dataset_version = dataset[~(dataset.well.isin(['E20Y','E10Z', 'J13X']))]
        quantiles = dataset_version['htst'].quantile(percentiles).round(0).reset_index().rename(columns={'index':'quantile'})
        bins = [0, int(quantiles.iloc[0][1]), int(quantiles.iloc[1][1]), 100]
        dataset_version['htst_interv'] = pd.cut(dataset_version['htst'], bins=bins, 
                                                labels=[f'{bins[0]}-{bins[1]}', f'{bins[1]}-{bins[2]}', f'{bins[2]}-{bins[3]}'])
        return dataset_version, bins
    dataset_version, bins = bins_calculation(dataset, percentiles)

    def display_histplot(dataset, bins, comment):
        fig, ax = plt.subplots(1, 2, figsize=(8, 2))
        custom_palette = {f'{bins[0]}-{bins[1]}': 'green', f'{bins[1]}-{bins[2]}': 'yellow', f'{bins[2]}-{bins[3]}': 'orange'}
        ax[0] = sns.histplot(data=dataset, x='htst', bins=50, kde=True, log_scale=False, ax=ax[0])
        ax[0].tick_params(axis='both', which='major', labelsize=8)
        ax[0].grid(True)
        ax[1] = sns.histplot(data=dataset, x='htst', bins=50, hue='htst_interv', kde=True, 
                            log_scale=False, ax=ax[1], palette=custom_palette)
        ax[1].tick_params(axis='both', which='major', labelsize=8)
        ax[1].grid(True)
        fig.suptitle(comment, fontsize=12) 
        plt.show();
    if hist_flag == 'hist':
        display_histplot(dataset_version, bins, 'Bal8 Azeri selected bins')
    else:
        pass

    phi8_range_test = dataset_version.groupby(['well','htst_interv']).agg({'htst':['sum','count']}).reset_index()
    phi8_range_test.columns = phi8_range_test.columns.map('_'.join)
    phi8_range_test = phi8_range_test.rename(columns={'well_':'well', 'htst_interv_':'htst_interv'})

    def ranged_data_transpose(dataset, var):
        df_lst = []
        for wellname in dataset.well.unique():
            data = dataset[dataset.well == wellname][['htst_interv', var]].T.reset_index()
            data.columns = data.iloc[0]
            data = data.iloc[1:]
            data['well'] = wellname
            df_lst.append(data)
        df = pd.concat(df_lst).reset_index(drop=True)
        df.insert(0, 'well', df.pop('well'))
        df_sum = dataset.groupby('well')['htst_sum'].sum()
        df = df.set_index('well').join(df_sum).reset_index()
        return df
    phit8_range_hcount_test = ranged_data_transpose(phi8_range_test, 'htst_count')

    xy8 = dataset_xy.groupby('well')[['TST','Xmean','Ymean']].agg({
                                                                    'Xmean': lambda x: x.iloc[0],
                                                                    'Ymean': lambda x: x.iloc[0],
                                                                    'TST': lambda x: x.iloc[-1] - x.iloc[0]
                                                                    }).dropna()
    phit8_range_hcount_xy_test = phit8_range_hcount_test.set_index('well').join(xy8).reset_index().dropna()
    phit8_range_hcount_xy_test = phit8_range_hcount_xy_test.rename(columns={'TST':'fmtst_gross'})
    phit8_range_hcount_xy_test['htst_sum_vv'] = phit8_range_hcount_xy_test['htst_sum'] / phit8_range_hcount_xy_test['fmtst_gross'].round(2)

    def piechart_map_version(dataset_map, bins, comment, print_flag, path):
        fig, ax = plt.subplots(figsize=(15,10))
        for ind, row in dataset_map.iterrows():
                ax.pie([row[f'{bins[0]}-{bins[1]}'], row[f'{bins[1]}-{bins[2]}'], row[f'{bins[2]}-{bins[3]}']], 
                        radius=row['htst_sum']*0.0025, center=(row['Xmean']/1000, row['Ymean']/1000), wedgeprops={"linewidth": 0.5, "edgecolor": "gray", "alpha":0.75},
                        colors=['green', 'yellow','orange'], frame=True)
                ax.scatter(x = dataset_map['Xmean']/1000, y = dataset_map['Ymean']/1000, s=0.5, c='gray') 
                # ax.text(row['Xmean']/1000, row['Ymean']/1000, row['well'], fontsize=7, ha='left', va='bottom')
                if ind % 2 == 0:
                    ax.text(row['Xmean']/1000+0.1, row['Ymean']/1000+0.1, row['well'], fontsize=6, ha='left', va='bottom')
                else:
                    ax.text(row['Xmean']/1000-0.1, row['Ymean']/1000-0.1, row['well'], fontsize=6, ha='left', va='bottom')
        stats = (' orange = htst '+ f'{bins[2]}-{bins[3]}' 
                '\n lightgreen = htst '+ f'{bins[1]}-{bins[2]}' 
                '\n green = htst ' + f'{bins[0]}-{bins[1]}')
        bbox = dict(boxstyle='round', fc='white', ec='black', alpha=1)
        ax.text(0.8, 0.85, stats, fontsize=8, bbox=bbox, transform=ax.transAxes, horizontalalignment='left')
        ax.set_title(comment, fontsize=20)
        if print_flag == 'print':
            plt.savefig(path, format='pdf', bbox_inches='tight')
        plt.show();       
    piechart_map_version(phit8_range_hcount_xy_test, bins, f'Bal8 piechart test:htst_count, size:htst_sum, percentiles: {percentiles}', print_flag, path)

    return phit8_range_hcount_xy_test, bins
percentiles = [0.4, 0.8]
vsh8_range_hcount, bins = htst_distrib_map_customize(ntd_top_vsh_bot8, df_bal8_azr_vshclp2, percentiles, 'donthist', 'print', 
                                                     'C:\\jupyter\\SPP\plots\\bal8_htst_count_version.pdf')

In [ ]:
# phit8_range_hcount_xy_test.loc[phit8_range_hcount_xy_test.well.isin(['B18Y','B18X','B10','B12Z','B11','B22','B22Z','B39']), 'geobody_td'] = 'axial'
def percenage_calculation(dataset, bins):
    dataset['bin_sum'] = (dataset[f'{bins[0]}-{bins[1]}'] + 
                          dataset[f'{bins[1]}-{bins[2]}'] + 
                          dataset[f'{bins[2]}-{bins[3]}'])
    dataset[f'{bins[0]}-{bins[1]}'+'%'] = dataset[f'{bins[0]}-{bins[1]}'] / dataset['bin_sum']
    dataset[f'{bins[1]}-{bins[2]}'+'%'] = dataset[f'{bins[1]}-{bins[2]}'] / dataset['bin_sum']
    dataset[f'{bins[2]}-{bins[3]}'+'%'] = dataset[f'{bins[2]}-{bins[3]}'] / dataset['bin_sum']
    return dataset
vsh8_range_hcount = percenage_calculation(vsh8_range_hcount, bins)   

def facies_estimation(dataset, bins, htst_cutoff, bins_cutoff):
        dataset['geobody_td'] = 'proximal'
        well_axial = dataset[(dataset['htst_sum_vv'] >= htst_cutoff) & 
                                                (dataset[f'{bins[2]}-{bins[3]}'+'%'] >= bins_cutoff[1])].well.unique()
        well_distal = dataset[(dataset['htst_sum_vv'] < htst_cutoff) & 
                                                (dataset[f'{bins[0]}-{bins[1]}'+'%'] >= bins_cutoff[0])].well.unique()
        dataset.loc[dataset.well.isin(well_axial), 'geobody_td'] = 'axial'
        dataset.loc[dataset.well.isin(well_distal), 'geobody_td'] = 'distal'
        return dataset
vsh8_range_hcount = facies_estimation(vsh8_range_hcount, bins, 0.3, [0.4, 0.3])

def piechart_map_version_test(dataset_map, bins, text_flag, comment):
    fig, ax = plt.subplots(figsize=(18,11))
    radius_mult = 0.0025
    text_size = 7
    for ind, row in dataset_map.iterrows():
            ax.pie([row[f'{bins[0]}-{bins[1]}'], row[f'{bins[1]}-{bins[2]}'], row[f'{bins[2]}-{bins[3]}']], 
                    radius=row['htst_sum']*radius_mult, center=(row['Xmean']/1000, row['Ymean']/1000), 
                    wedgeprops={"linewidth": 0.5, "edgecolor": "gray", "alpha":0.75},
                    colors=['green', 'yellow','orange'], frame=True)
            if text_flag == 'wellnum':
                if ind % 2 == 0:
                        ax.text(row['Xmean']/1000+0.1, row['Ymean']/1000+0.1, row['well'], fontsize=text_size, ha='left', va='bottom')
                else:
                        ax.text(row['Xmean']/1000-0.1, row['Ymean']/1000-0.1, row['well'], fontsize=text_size, ha='left', va='bottom')
    for ind, row in dataset_map[dataset_map.geobody_td == 'axial'].iterrows():
            circle = Circle((row['Xmean']/1000, row['Ymean']/1000), 
                            radius=row['htst_sum']*radius_mult, fill=False, edgecolor='red', linewidth=2)
            ax.add_patch(circle)
    for ind, row in dataset_map[dataset_map.geobody_td == 'proximal'].iterrows():
            circle = Circle((row['Xmean']/1000, row['Ymean']/1000), 
                            radius=row['htst_sum']*radius_mult, fill=False, edgecolor='blue', linewidth=2)
            ax.add_patch(circle)
    for ind, row in dataset_map[dataset_map.geobody_td == 'distal'].iterrows():
            circle = Circle((row['Xmean']/1000, row['Ymean']/1000), 
                            radius=row['htst_sum']*radius_mult, fill=False, edgecolor='black', linewidth=2)
            ax.add_patch(circle)

    stats = (' orange = htst '+ f'{bins[2]}-{bins[3]}' 
            '\n lightgreen = htst '+ f'{bins[1]}-{bins[2]}' 
            '\n green = htst ' + f'{bins[0]}-{bins[1]}')
    bbox = dict(boxstyle='round', fc='white', ec='black', alpha=1)
    ax.text(0.875, 0.925, stats, fontsize=12, bbox=bbox, transform=ax.transAxes, horizontalalignment='left')
    ax.set_title(comment, fontsize=16)
    plt.grid(False)
    plt.show();       
# piechart_map_version_test(vsh8_range_hcount, bins, 'wellnum', f'Bal8 piechart test:htst_count, size:htst_sum, percentiles: {percentiles}')

print(vsh8_range_hcount.columns)
def gpd_buffer_map(dataset, bins, geobody, comment, print_flag, path):
    text_size = 6
    geometry = [Point(xy) for xy in zip(dataset['Xmean'], dataset['Ymean'])]
    gpd_facies_xy = gpd.GeoDataFrame(dataset, geometry=geometry).drop(['Xmean','Ymean'], axis=1).set_crs('EPSG:2499')

    gpd_facies_xy.loc[gpd_facies_xy.well.isin(['C21Z']), geobody] = 'axial'
    gpd_facies_xy.loc[gpd_facies_xy.well.isin(['C33Z', 'D05Z', 'D19Z','D23X']), geobody] = 'proximal'

    def gpd_buffer_calc(dataset, buff_radius):
        buffers = dataset.buffer(buff_radius)
        buffers = gpd.GeoDataFrame(geometry=buffers)
        dataset_buff = dataset.join(buffers, rsuffix='_buff')
        dataset_buff = gpd.GeoDataFrame(dataset_buff, geometry='geometry_buff').drop('geometry', axis=1).set_crs('EPSG:2499')
        return dataset_buff
    distal_facies_buff = gpd_buffer_calc(gpd_facies_xy[gpd_facies_xy[geobody] == 'distal'], 450)
    proximal_facies_buff = gpd_buffer_calc(gpd_facies_xy[gpd_facies_xy[geobody] == 'proximal'], 350)
    axial_facies_buff = gpd_buffer_calc(gpd_facies_xy[gpd_facies_xy[geobody] == 'axial'], 290)

    unified_axial = axial_facies_buff.unary_union
    unified_proximal = proximal_facies_buff.unary_union
    unified_distal = distal_facies_buff.unary_union

    diff_axial = gpd.GeoSeries(unified_axial).explode()
    diff_axial_ch = diff_axial.convex_hull
    diff_proximal = gpd.GeoSeries(unified_proximal).explode()
    diff_proximal_ch = diff_proximal.convex_hull
    unified_distal = gpd.GeoSeries(unified_distal).explode()
    unified_distal_ch = unified_distal.convex_hull

    fig, ax = plt.subplots(figsize=(15,10))
    radius_mult = 2
    # text_size = 7
    for ind, row in gpd_facies_xy.iterrows():
        ax.pie([row[f'{bins[0]}-{bins[1]}'], row[f'{bins[1]}-{bins[2]}'], row[f'{bins[2]}-{bins[3]}']], 
                radius=row['htst_sum']*radius_mult, center=(row['geometry'].x, row['geometry'].y), 
                wedgeprops={"linewidth": 0.5, "edgecolor": "gray", "alpha":1},
                colors=['green', 'yellow','orange'], frame=True)
    for ind, row in gpd_facies_xy.iterrows():
        if ind % 2 == 0:
            ax.text(row['geometry'].x, row['geometry'].y, row['well'], fontsize=text_size, ha='left', va='bottom')
        else:
            ax.text(row['geometry'].x, row['geometry'].y, row['well'], fontsize=text_size, ha='left', va='top')

    unified_distal_ch.plot(ax=ax, color='green', alpha=0.5, ec='gray', zorder=1)
    diff_proximal_ch.plot(ax=ax, color='yellow', alpha=0.5, ec='gray', zorder=2)
    diff_axial_ch.plot(ax=ax, color='orange', alpha=0.5, ec='gray', zorder=3)
    gpd_facies_xy.plot(ax=ax, color='black', markersize=2, zorder=4)

    stats = (   ' orange = axial' + 
                '\n yellow  = proximal' +
                '\n green  = distal')
    bbox = dict(boxstyle='round', fc='white', ec='black', alpha=1)
    ax.text(0.875, 0.9, stats, fontsize=10, bbox=bbox, transform=ax.transAxes, horizontalalignment='left')

    plt.title('Bal8 facies estimation ' + comment, fontsize=16)
    if print_flag == 'print':
        plt.savefig(path, format='pdf', bbox_inches='tight')
    plt.show();
    return gpd_facies_xy
gpd_facies_xy = gpd_buffer_map(vsh8_range_hcount, bins, 'geobody_td','manual','dontprint', 'C:\\jupyter\\SPP\plots\\bal8_facies.pdf')

In [ ]:
def current_time():
    current_datetime = pd.Timestamp.now()
    current_datetime_str = current_datetime.strftime('%d%m%y_%H%M')
    return current_datetime_str
time_str = current_time()
def final_dataset_creation(dataset_init, dataset_facies, bins, phit_cutoff):
    facies = dataset_facies[['well',
                             f'{bins[0]}-{bins[1]}',
                             f'{bins[1]}-{bins[2]}',
                             f'{bins[2]}-{bins[3]}',
                             'htst_sum', 'bin_sum',
                             f'{bins[0]}-{bins[1]}'+'%', 
                             f'{bins[1]}-{bins[2]}'+'%', 
                             f'{bins[2]}-{bins[3]}'+'%', 
                             'geobody_td']]
    dataset_init_facies = dataset_init.set_index('well').join(facies.set_index('well')).reset_index().dropna()
    dataset_init_facies = dataset_init_facies[dataset_init_facies.phit_avg >=phit_cutoff]
    dataset_init_facies['khtst'] = dataset_init_facies['htst']*dataset_init_facies['htst']
    khtst_sum = dataset_init_facies.groupby('well')['khtst'].sum().reset_index().rename(columns={'khtst':'khtst_sum'})
    result = dataset_init_facies.set_index('well').join(khtst_sum.set_index('well')).reset_index()
    return result
ntd_top_phi_bot8_facies_kh = final_dataset_creation(ntd_top_phi_bot8, gpd_facies_xy, bins, 0.13)
# ntd_top_vsh_bot8_facies_kh.to_csv('C:\\jupyter\\SPP\\inputoutput\\ntd_top_vsh_bot8_facies_kh_' + time_str + '.csv', index=False)
def kdeplot_display(data):
    khtst_sum_well = data.groupby('well')[['geobody_td','khtst_sum']].apply(lambda x: x.iloc[0]).reset_index()
    htst_counts = data.groupby(['geobody_td'])[[f'{bins[0]}-{bins[1]}',
                                                f'{bins[1]}-{bins[2]}',
                                                f'{bins[2]}-{bins[3]}']].sum().reset_index()
    htst_counts_barchart = pd.melt(htst_counts, ['geobody_td']).rename(columns={'variable':'bins', 'value':'count'})
    fig, ax = plt.subplots(1,4, figsize=(18, 4))

    custom_palette = {'distal': 'lightgreen', 'proximal':'yellow','axial': 'orange'}
    custom_palette2 = {'0-3': 'lightgreen', '3-8':'yellow','8-100': 'orange'}
    sns.kdeplot(data=data, x='phit_avg', hue='geobody_td', ax=ax[0], palette=custom_palette, bw_adjust=1)
    ax[0].grid(True, axis='x'), ax[0].set_xticks(np.arange(0.1, 0.35, 0.025)), ax[0].tick_params(axis='both', which='major', labelsize=8)
    sns.kdeplot(data=data, x='vsh_avg', hue='geobody_td', ax=ax[1], palette=custom_palette)
    sns.barplot(data=htst_counts_barchart, x='geobody_td', y='count', hue='bins', ax=ax[2], palette=custom_palette2)
    sns.kdeplot(data=khtst_sum_well, x='khtst_sum', hue='geobody_td', ax=ax[3], palette=custom_palette);
kdeplot_display(ntd_top_phi_bot8_facies_kh)

In [45]:
def quantile_display(dataset, q1, q2):
    def quant_calculation(dataset, wellname, q):
        well = dataset[dataset.well == wellname]
        well[f'htst_p{int(q*100)}'] = well['htst'].quantile([q]).iloc[0]
        well[f'phit_avg_p{int(q*100)}'] = well['phit_avg'].quantile([q]).iloc[0]
        well[f'vsh_avg_p{int(q*100)}'] = well['vsh_avg'].quantile([q]).iloc[0]
        well[f'perm_avg_p{int(q*100)}'] = well['perm_avg'].quantile([q]).iloc[0]
        return well
    df_lst1 = []
    for wellname in dataset.well.unique():
        well_quant = quant_calculation(dataset, wellname, q1)
        df_lst1.append(well_quant)
    facies_kh_quant_bal8_v1 = pd.concat(df_lst1)
    
    df_lst2 = []
    for wellname in dataset.well.unique():
        well_quant = quant_calculation(dataset, wellname, q2)
        df_lst2.append(well_quant)
    facies_kh_quant_bal8_v2 = pd.concat(df_lst2)

    fig, ax = plt.subplots(1,2, figsize=(15, 6))
    custom_palette = {'distal': 'lightgreen', 'proximal':'yellow','axial': 'orange'}
    sns.scatterplot(data=facies_kh_quant_bal8_v1, x=f'phit_avg_p{int(q1*100)}', y=f'htst_p{int(q1*100)}', 
                    size='khtst_sum', hue='geobody_td', palette=custom_palette, ax=ax[0])
    ax[0].set_xlim([0.13, 0.30]), ax[0].set_ylim([0, 20])
    sns.scatterplot(data=facies_kh_quant_bal8_v2, x=f'phit_avg_p{int(q2*100)}', y=f'htst_p{int(q2*100)}', 
                    size='khtst_sum', hue='geobody_td', palette=custom_palette, ax=ax[1])
    ax[1].set_xlim([0.13, 0.30]), ax[1].set_ylim([0, 20])
# quantile_display(ntd_top_phi_bot8_facies_kh, 0.1, 0.9)

In [ ]:
well_khtst_sum = ntd_top_phi_bot8_facies_kh.groupby('well')['khtst_sum'].apply(lambda x: x.iloc[0]).reset_index()
vsh8_range_hcount2 = vsh8_range_hcount.set_index('well').join(well_khtst_sum.set_index('well')).dropna(subset=['khtst_sum']).reset_index()

def htst_clustering(dataset, feature_list, normalize_flag, scaler, cluster_num):
    """
    MinMaxScaler(), StandardScaler()
    """
    data = dataset[feature_list]
    if normalize_flag == 'norm':
        normalized_data = scaler.fit_transform(data)
    else:
        normalized_data = data

    kmeans = KMeans(n_clusters=cluster_num, random_state=42)
    kmeans_labels = kmeans.fit_predict(normalized_data)
    kmeans_labels = pd.DataFrame(kmeans_labels, columns=['kmeans'])

    gmm = GaussianMixture(n_components=cluster_num, random_state=42)
    gmm.fit(normalized_data)
    gmm_labels = gmm.predict(normalized_data)
    gmm_labels = pd.DataFrame(gmm_labels, columns=['gmm'])

    agglomerative = AgglomerativeClustering(n_clusters=cluster_num)
    agglomerative_labels = agglomerative.fit_predict(normalized_data)
    agglomerative_labels = pd.DataFrame(agglomerative_labels, columns=['agglomer'])
    result = pd.concat([dataset, kmeans_labels, gmm_labels, agglomerative_labels], axis=1)
    return result
vsh8_range_hcount_clustered = htst_clustering(vsh8_range_hcount2, ['0-3%','3-8%', '8-100%', 'htst_sum_vv'], 'dontnorm', StandardScaler(), 3)
vsh8_range_hcount_clustered.columns[-3:]

In [ ]:
cluster_rename = {2: 'proximal', 1: 'axial', 0: 'distal'}
cluster_rename_color = {'axial': 'orange', 'proximal': 'yellow', 'distal': 'green'}
cluster_rename_agg = {2: 'proximal', 0: 'axial', 1: 'distal'}
def htst_clustering_display(dataset, clustering, cluster_rename, cluster_rename_color):
    dataset.loc[dataset[clustering] == 0, 'geobody_' + clustering] = cluster_rename[0]
    dataset.loc[dataset[clustering] == 1, 'geobody_' + clustering] = cluster_rename[1]
    dataset.loc[dataset[clustering] == 2, 'geobody_' + clustering] = cluster_rename[2]
    htst_counts_kmeans = dataset.groupby(['geobody_' + clustering])[[  f'{bins[0]}-{bins[1]}',
                                                                f'{bins[1]}-{bins[2]}',
                                                                f'{bins[2]}-{bins[3]}']].sum().reset_index()
    htst_counts_kmeans2 = pd.melt(htst_counts_kmeans, ['geobody_' + clustering]).rename(columns={'variable':'bins', 'value':'count'})
    custom_palette_bins = {f'{bins[0]}-{bins[1]}': 'lightgreen', f'{bins[1]}-{bins[2]}':'yellow', f'{bins[2]}-{bins[3]}': 'orange'}
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))
    sns.barplot(data=htst_counts_kmeans2, x='geobody_' + clustering, y='count', hue='bins', palette=custom_palette_bins, ax=ax[0])
    custom_palette = {cluster_rename[2]: cluster_rename_color[cluster_rename[2]], 
                      cluster_rename[1]:cluster_rename_color[cluster_rename[1]], 
                      cluster_rename[0]: cluster_rename_color[cluster_rename[0]]}
    sns.kdeplot(data=dataset, x='khtst_sum', hue='geobody_' + clustering, palette=custom_palette, ax=ax[1]);
    return dataset
def kdeplot_display_v2(data):
    khtst_sum_well = data.groupby('well')[['geobody_td','khtst_sum']].apply(lambda x: x.iloc[0]).reset_index()
    htst_counts = data.groupby(['geobody_td'])[[f'{bins[0]}-{bins[1]}',
                                                f'{bins[1]}-{bins[2]}',
                                                f'{bins[2]}-{bins[3]}']].sum().reset_index()
    htst_counts_barchart = pd.melt(htst_counts, ['geobody_td']).rename(columns={'variable':'bins', 'value':'count'})
    fig, ax = plt.subplots(1,2, figsize=(12, 4))

    custom_palette = {'distal': 'green', 'proximal':'yellow','axial': 'orange'}
    custom_palette2 = {'0-3': 'lightgreen', '3-8':'yellow','8-100': 'orange'}
    sns.barplot(data=htst_counts_barchart, x='geobody_td', y='count', hue='bins', ax=ax[0], palette=custom_palette2)
    sns.kdeplot(data=khtst_sum_well, x='khtst_sum', hue='geobody_td', ax=ax[1], palette=custom_palette);
vsh8_clustered_td = kdeplot_display_v2(ntd_top_phi_bot8_facies_kh)
vsh8_clustered_kmeans = htst_clustering_display(vsh8_range_hcount_clustered, 'kmeans' ,cluster_rename, cluster_rename_color)
vsh8_clustered_gmm = htst_clustering_display(vsh8_range_hcount_clustered, 'gmm' ,cluster_rename, cluster_rename_color)
vsh8_clustered_agg = htst_clustering_display(vsh8_range_hcount_clustered, 'agglomer', cluster_rename_agg, cluster_rename_color)

In [ ]:
sns.pairplot(data=vsh8_range_hcount_clustered[['0-3','3-8', '8-100', 'htst_sum_vv', 'khtst_sum','geobody_kmeans']], 
             hue='geobody_kmeans', palette=cluster_rename_color, height=2, aspect=1, plot_kws={'alpha':0.5, 'ec':'gray'});

In [ ]:
gpd_facies_xy_cluster = gpd_buffer_map(vsh8_clustered_kmeans, bins, 'geobody_kmeans', 'kmeans','dontprint', 'C:\\jupyter\\SPP\plots\\bal8_kmeans.pdf')
gpd_facies_xy_cluster = gpd_buffer_map(vsh8_clustered_gmm, bins, 'geobody_gmm', 'gmm','dontprint', 'C:\\jupyter\\SPP\plots\\bal8_gmm.pdf')
gpd_facies_xy_cluster = gpd_buffer_map(vsh8_clustered_agg, bins, 'geobody_agglomer', 'agglomer','dontprint', 'C:\\jupyter\\SPP\plots\\bal8_agg.pdf')

In [ ]:
def gr_correlation_panel(dataset, dataset_cluster, cluster, well_list):
    N = len(well_list)
    fig, ax = plt.subplots(1, N, figsize=(N*2, 5))
    data_gr = dataset
    cluster_rename_color = {'axial': 'orange', 'proximal': 'yellow', 'distal': 'green'}
    for i in range(N):
        well = data_gr[data_gr.well == well_list[i]]
        cluster_name = dataset_cluster[dataset_cluster.well == well_list[i]][cluster].iloc[0]
        color = cluster_rename_color[cluster_name]
        ax[i].plot(well['GR_N'], well['TST'], color='green')
        ax[i].plot(well['NET_clp2']*150, well['TST'], color='orange', alpha=1, lw=0.25)
        ax[i].invert_yaxis()
        ax[i].fill_betweenx(well['TST'],well['NET_clp2']*150, color='orange', alpha=0.33)
        ax[i].set_xlim([0, 150])
        ax[i].set_title(well_list[i], color=color, weight='bold')
gr_correlation_panel(df_bal8_azr_vshclp2, gpd_facies_xy_cluster, 'geobody_kmeans', ['B01Y','B12','B06','B10', 'B11','B14Z', 'B16Y', 'B16Z'])
gr_correlation_panel(df_bal8_azr_vshclp2, gpd_facies_xy_cluster, 'geobody_td', ['B01Y','B12','B06','B10', 'B11','B14Z', 'B16Y', 'B16Z'])

In [ ]:
gr_correlation_panel(df_bal8_azr_vshclp2, gpd_facies_xy_cluster, 'geobody_kmeans', ['D03','D04','D06','D01Z','D09', 'D09Y'])
gr_correlation_panel(df_bal8_azr_vshclp2, gpd_facies_xy_cluster, 'geobody_td', ['D03','D04','D06','D01Z','D09', 'D09Y'])

## Htst distribution Bal8 vsh

In [ ]:
ntd_top_vsh_bot8 = ntd_top_vsh_bot8[~(ntd_top_vsh_bot8.well.isin(['E20Y','E10Z','J13X']))]
quantiles = ntd_top_vsh_bot8['htst'].quantile([0.25, 0.5, 0.75]).round(0).reset_index().rename(columns={'index':'quantile'})
bins = [0, int(quantiles.iloc[0][1]), int(quantiles.iloc[1][1]), int(quantiles.iloc[2][1]), 100]

def htst_distrib_histo(dataset, bins, comment):
        dataset['htst_interv'] = pd.cut(dataset['htst'], bins=bins, 
                                        labels=[f'{bins[0]}-{bins[1]}', 
                                                f'{bins[1]}-{bins[2]}', 
                                                f'{bins[2]}-{bins[3]}', 
                                                f'{bins[3]}-{bins[4]}'])
        custom_palette = {f'{bins[0]}-{bins[1]}': 'green', 
                        f'{bins[1]}-{bins[2]}': 'lightgreen', 
                        f'{bins[2]}-{bins[3]}': 'yellow',
                        f'{bins[3]}-{bins[4]}': 'orange'}
        fig, ax = plt.subplots(1, 2, figsize=(16, 4))
        ax[0] = sns.histplot(data=dataset, x='htst', bins=75, kde=True, log_scale=False, ax=ax[0])
        ax[0].grid(True)

        ax[1] = sns.histplot(data=dataset, x='htst', bins=75, hue='htst_interv', kde=True, 
                        log_scale=False, ax=ax[1], palette = custom_palette)
        ax[1].grid(True)
        fig.suptitle(comment)
        plt.show();
        return dataset
ntd_dist_vsh8 = htst_distrib_histo(ntd_top_vsh_bot8, bins, 'Bal8 Azeri vsh')

In [76]:
# well_display_logs(df_bal8_azr_vshclp2, 'B28', 'Balakhany VIII','NET_clp2','vshclp2', 'TST', 10, 'dontprint', 'C:\jupyter\SPP\plots\logs_vsh_bal8\\')

In [ ]:
def htst_sum_count_calc(dataset):
    htst_range = dataset.groupby(['well','htst_interv']).agg({'htst':['sum','count']}).reset_index()
    htst_range.columns = htst_range.columns.map('_'.join)
    htst_range = htst_range.rename(columns={'well_':'well', 'htst_interv_':'htst_interv'})
    return htst_range
vsh8_range = htst_sum_count_calc(ntd_dist_vsh8)

def ranged_data_transpose(dataset, var):
    df_lst = []
    for wellname in dataset.well.unique():
        data = dataset[dataset.well == wellname][['htst_interv',var]].T.reset_index()
        data.columns = data.iloc[0]
        data = data.iloc[1:]
        data['well'] = wellname
        df_lst.append(data)
    df = pd.concat(df_lst).reset_index(drop=True)
    df.insert(0, 'well', df.pop('well'))
    df_sum = dataset.groupby('well')['htst_sum'].sum()
    df = df.set_index('well').join(df_sum).reset_index()
    return df
vsh8_range_hcount = ranged_data_transpose(vsh8_range, 'htst_count')

def convert_linestringz_polygon_run():
    def convert_linestringz_polygon(dataset):
        geom = [x for x in dataset.geometry]
        df_lst = []
        for i in range(len(geom)):
            all_coords = mapping(geom[i])['coordinates']
            lats = [x[1] for x in all_coords]
            lons = [x[0] for x in all_coords]
            polyg = Polygon(zip(lons, lats))
            df = gpd.GeoDataFrame(index=[0], crs='EPSG:2499', geometry=[polyg])
            df_lst.append(df)
        result = pd.concat(df_lst).reset_index(drop=True)
        return result 
    bal8_20_3 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_20_base_3.shp').set_crs('EPSG:2499')
    bal8_20_3_polygon = convert_linestringz_polygon(bal8_20_3)
    bal8_20_1 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_20_base_1.shp').set_crs('EPSG:2499')
    bal8_20_1_polygon = convert_linestringz_polygon(bal8_20_1)
    bal8_20_0 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_20_base_0.shp').set_crs('EPSG:2499')
    bal8_20_0_polygon = convert_linestringz_polygon(bal8_20_0)
    bal8_30_3 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_30_base_3.shp').set_crs('EPSG:2499')
    bal8_30_3_polygon = convert_linestringz_polygon(bal8_30_3)
    bal8_30_1 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_30_base_1.shp').set_crs('EPSG:2499')
    bal8_30_1_polygon = convert_linestringz_polygon(bal8_30_1)
    bal8_30_0 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_30_base_0.shp').set_crs('EPSG:2499')
    bal8_30_0_polygon = convert_linestringz_polygon(bal8_30_0)
    bal8_1510_3 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_1510_base_3.shp').set_crs('EPSG:2499')
    bal8_1510_3_polygon = convert_linestringz_polygon(bal8_1510_3)
    bal8_1510_1 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_1510_base_1.shp').set_crs('EPSG:2499')
    bal8_1510_1_polygon = convert_linestringz_polygon(bal8_1510_1)
    bal8_1510_0 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_1510_base_0.shp').set_crs('EPSG:2499')
    bal8_1510_0_polygon = convert_linestringz_polygon(bal8_1510_0)
    result = {'bal8_20':[bal8_20_0_polygon, bal8_20_1_polygon, bal8_20_3_polygon],
              'bal8_30':[bal8_30_0_polygon, bal8_30_1_polygon, bal8_30_3_polygon],
              'bal8_1510':[bal8_1510_0_polygon, bal8_1510_1_polygon, bal8_1510_3_polygon]}
    return result
polygons_dict= convert_linestringz_polygon_run()

xy8 = df_interp[df_interp.FORMATION_up == 'Balakhany VIII'].groupby('well')[['Xmean','Ymean']].apply(lambda x: x.iloc[0]).dropna()
geometry = [Point(xy) for xy in zip(xy8['Xmean'], xy8['Ymean'])]
bdl8_xy_gpd = gpd.GeoDataFrame(xy8, geometry=geometry).drop(['Xmean','Ymean'], axis=1)
vsh8_range_hcount_xy = vsh8_range_hcount.set_index('well').join(bdl8_xy_gpd).reset_index()
vsh8_range_hcount_xy = vsh8_range_hcount_xy[~vsh8_range_hcount_xy.geometry.isna()]

bal8_20_list = [polygons_dict['bal8_20']] 
bal8_30_list = [polygons_dict['bal8_30']]
bal8_1510_list = [polygons_dict['bal8_1510']]
empty_list = []

def piechart_map(dataset_map, dataset_polygon, comment, print_flag, path):
    fig, ax = plt.subplots(figsize=(15,10), dpi = 300, layout='tight')
    if len(dataset_polygon) != 0:
        dataset_polygon[2].plot(ax=ax, color='orange', label='bal8_1510_3', alpha=0.5)
        dataset_polygon[1].plot(ax=ax, color='yellow', label='bal8_1510_1', alpha=0.5)
        dataset_polygon[0].plot(ax=ax, color='green', label='bal8_1510_0', alpha=0.5)
    else:
        pass
    for ind, row in dataset_map.iterrows():
            ax.pie([row[f'{bins[0]}-{bins[1]}'], row[f'{bins[1]}-{bins[2]}'], row[f'{bins[2]}-{bins[3]}'], row[f'{bins[3]}-{bins[4]}']], 
                    radius=row['htst_sum']*2, center=(row['geometry'].x, row['geometry'].y), wedgeprops={"linewidth": 0.1, "edgecolor": "black", "alpha":0.75},
                    colors=['green', 'lightgreen', 'yellow','orange'], frame=True)
            ax.scatter(x = row['geometry'].x, y = row['geometry'].y, s=0.5, c='black')
            if ind % 2 == 0:
                ax.text(row['geometry'].x, row['geometry'].y, row['well'], fontsize=4, ha='left', va='bottom')
            else:
                ax.text(row['geometry'].x, row['geometry'].y, row['well'], fontsize=4, ha='left', va='top')
                #  pass
    stats = (' orange = htst 7-100 \n yellow = htst 4-7 \n lightgreen = htst 2-4 \n green = htst 0-2')
    bbox = dict(boxstyle='round', fc='white', ec='black', alpha=1)
    ax.text(0.8, 0.85, stats, fontsize=10, bbox=bbox, transform=ax.transAxes, horizontalalignment='left')
    ax.set_title(comment, fontsize=20)
    if print_flag == 'print':
        plt.savefig(path, format='pdf', bbox_inches='tight')
    plt.show();       
piechart_map(vsh8_range_hcount_xy, empty_list, 'Bal8 piechart:htst_count, size:htst_sum polygon:', 'dontprint', 
             'C:\\jupyter\\SPP\plots\\bal8_htst_count_vsh_empty_new_wells.pdf')
# piechart_map(vsh8_range_hcount_xy, bal8_20_list[0], 'Bal8 piechart:htst_count, size:htst_sum polygon:bal8_20', 'print', 
#              'C:\\jupyter\\SPP\plots\\bal8_htst_count_vsh_20.pdf')
# piechart_map(vsh8_range_hcount_xy, bal8_30_list[0], 'Bal8 piechart:htst_count, size:htst_sum polygon:bal8_30', 'print', 
#              'C:\\jupyter\\SPP\plots\\bal8_htst_count_vsh_30.pdf')
# piechart_map(vsh8_range_hcount_xy, bal8_1510_list[0], 'Bal8 piechart:htst_count, size:htst_sum polygon:bal8_1510', 'print', 
#              'C:\\jupyter\\SPP\plots\\bal8_htst_count_vsh_1510.pdf')

In [ ]:
vsh8_range_hsum = ranged_data_transpose(vsh8_range, 'htst_sum')

xy8 = df_interp[df_interp.FORMATION_up == 'Balakhany VIII'].groupby('well')[['Xmean','Ymean']].apply(lambda x: x.iloc[0])
vsh8_range_hsum_xy = vsh8_range_hsum.set_index('well').join(xy8).reset_index()
for col in vsh8_range_hsum_xy.columns:
    try:
        vsh8_range_hsum_xy[col] = pd.to_numeric(vsh8_range_hsum_xy[col])
    except:
        vsh8_range_hsum_xy[col] = vsh8_range_hsum_xy[col].astype('string')
vsh8_range_hsum_xy[f'{bins[3]}-{bins[4]}'] = vsh8_range_hsum_xy[f'{bins[3]}-{bins[4]}'].round(0).astype('int')

def bubble_map(dataset_map, comment):
    fig, ax = plt.subplots(figsize=(15,10))
    ax.scatter(x = dataset_map['Xmean']/1000, y = dataset_map['Ymean']/1000, s=dataset_map[f'{bins[3]}-{bins[4]}'].values*5, c='orange', ec='gray', alpha=0.75)
    ax.scatter(x = dataset_map['Xmean']/1000, y = dataset_map['Ymean']/1000, s=1, c='orange', ec='black', alpha=0.75)
    for ind, row in dataset_map.iterrows():
            ax.text(row['Xmean']/1000, row['Ymean']/1000, row['well'], fontsize=7, ha='left', va='bottom')
    stats = (f' orange = htst {bins[3]}-{bins[4]}')
    bbox = dict(boxstyle='round', fc='white', ec='black', alpha=1)
    ax.text(0.8, 0.95, stats, fontsize=12, bbox=bbox, transform=ax.transAxes, horizontalalignment='left')
    ax.set_title(comment, fontsize=18)
    ax.grid()
    # plt.savefig('C:\\jupyter\\SPP\\plots\\bal8_htst_sum.pdf', format='pdf', bbox_inches='tight')
    plt.show()
bubble_map(vsh8_range_hsum_xy, 'Bal8 htst_sum for interval ' + f'{bins[3]}-{bins[4]}')

## Htst distribution Bal8 phi

In [ ]:
ntd_top_phi_bot8 = ntd_top_phi_bot8[~(ntd_top_phi_bot8.well.isin(['E20Y','E10Z','J13X']))]
quantiles = ntd_top_phi_bot8['htst'].quantile([0.25, 0.5, 0.75]).round(0).reset_index().rename(columns={'index':'quantile'})
bins = [0, int(quantiles.iloc[0][1]), int(quantiles.iloc[1][1]), int(quantiles.iloc[2][1]), 100]

def htst_distrib_histo(dataset, bins, comment):
        dataset['htst_interv'] = pd.cut(dataset['htst'], bins=bins, 
                                        labels=[f'{bins[0]}-{bins[1]}', 
                                                f'{bins[1]}-{bins[2]}', 
                                                f'{bins[2]}-{bins[3]}', 
                                                f'{bins[3]}-{bins[4]}'])
        custom_palette = {f'{bins[0]}-{bins[1]}': 'green', 
                          f'{bins[1]}-{bins[2]}': 'lightgreen', 
                          f'{bins[2]}-{bins[3]}': 'yellow',
                          f'{bins[3]}-{bins[4]}': 'orange'}
        fig, ax = plt.subplots(1, 2, figsize=(16, 4))
        ax[0] = sns.histplot(data=dataset, x='htst', bins=75, kde=True, log_scale=False, ax=ax[0])
        ax[0].grid(True)

        ax[1] = sns.histplot(data=dataset, x='htst', bins=75, hue='htst_interv', kde=True, 
                        log_scale=False, ax=ax[1], palette = custom_palette)
        ax[1].grid(True)
        fig.suptitle(comment)
        plt.show()
        return dataset
ntd_dist_phi8 = htst_distrib_histo(ntd_top_phi_bot8, bins, 'Bal8 Azeri phi')

In [ ]:
def well_collecting_clusters_top_phi_bot(dataset, clustering, fm):
    df_lst = []
    for wellname in dataset.well.unique()[:]:
        data = dataset[dataset.well == wellname]
        well_lst = []
        phit_lst = []
        htst_lst = []
        bot_lst = []
        ntg_lst = []
        vsh_lst = []
        cluster_lst = []
        for ind, row in data.iterrows():
            well_lst.append(wellname)
            well_lst.append(wellname)

            phit_lst.append(0)
            phit_lst.append(row['phit_avg'])

            cluster_lst.append(np.nan)
            cluster_lst.append(row[clustering])

            htst_lst.append(row['top_htst'])
            htst_lst.append(row['htst'])
            
            bot_lst.append(row['bot_htst'])

            ntg_lst.append(0)
            ntg_lst.append(row['ntg'])

            vsh_lst.append(0)
            vsh_lst.append(row['vsh_avg'])

        phit_lst.append(0)
        cluster_lst.append(np.nan)
        htst_lst.append(data['bot_htst'].iloc[-1])
        well_lst.append(wellname)
        well_collect_cluster_short = pd.DataFrame(zip(well_lst, phit_lst, htst_lst, ntg_lst, vsh_lst, cluster_lst ), columns=[  'well','phit', 'htst', 
                                                                                                                                'ntg', 'vsh', 'cluster'])
        well_last_row = pd.DataFrame({'well':[well_lst[-1]], 'phit':[0], 'htst': [bot_lst[-1]], 'ntg':[0], 'vsh':[0], 'cluster':[cluster_lst[-1]]})
        well_collect_cluster = pd.concat([well_collect_cluster_short, well_last_row]).reset_index(drop=True)
        well_collect_cluster['depth'] = well_collect_cluster['htst'].cumsum()
        df_lst.append(well_collect_cluster)
    result = pd.concat(df_lst)
    result['FORMATION_up'] = fm
    return result
ntd_phit8_clusters = well_collecting_clusters_top_phi_bot(ntd_dist_phi8, 'htst_interv', 'Balakhany VIII')
def coloring_clusters_matrix(dataset, letters_list, rows, columns, clustering, output_flag):
    """
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J']
    """
    def clusters_rectangle(data, k, color):
        # cluster_xy = data['depth'].iloc[k-2]
        cluster_xy = data['depth'].iloc[k-1]
        # cluster_h = data['depth'].iloc[k+1] - data['depth'].iloc[k-2]
        cluster_h = data['depth'].iloc[k] - data['depth'].iloc[k-1]
        rectangle = patches.Rectangle((0, cluster_xy) , 1, cluster_h, edgecolor=color, facecolor=color, alpha=0.75)
        ax[j,i].add_patch(rectangle)
    for letter in letters_list:
        wells_letter = [wellname for wellname in dataset.well.unique() if wellname.startswith(letter)]
        fig, ax = plt.subplots(rows,columns, figsize=(16,rows*2.5))
        counter = 0
        for j in range(0, rows):
            for i in range(0, columns):
                if counter < len(wells_letter):
                    wellname = wells_letter[counter]
                    welldata = dataset[dataset.well==wellname]
                    df_top = pd.DataFrame({'well':[wellname], 'phit':[0], 'htst':[0], 'cluster':welldata['cluster'].iloc[0],'depth':[0]})
                    welldata = pd.concat([df_top, welldata]).reset_index().drop('index', axis=1)
                    ax[j,i].plot(welldata['phit'], welldata['depth'], drawstyle='steps-post', color='black', alpha=1, lw=0.75)
                    ax[j,i].plot(welldata['vsh']*0.35, welldata['depth'], drawstyle='steps-post', color='green', alpha=1, lw=1)
                    ax[j,i].set_xlim(0, 0.35)
                    ax[j,i].set_ylim(0, 150)
                    ax[j,i].invert_yaxis()
                    ax[j,i].set_title(wellname)
                    ax[j,i].xaxis.set_ticks(np.arange(0, 0.35, 0.1))
                    # ax[j,i].tick_params(axis='both', which='major', labelsize=10)
                    ax[j,i].grid()
                    # for k in range(len(welldata)):
                    #     if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == 0:
                    #         clusters_rectangle(welldata, k, 'blue')
                    #     if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == 1:
                    #         clusters_rectangle(welldata, k, 'green')
                    #     if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == 2:
                    #         clusters_rectangle(welldata, k, 'red')
                    # labels=['0-2', '2-4', '4-7', '7-100'])
                    for k in range(len(welldata)):
                        if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == f"{bins[0]}-{bins[1]}":
                            clusters_rectangle(welldata, k, 'green')
                        if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == f"{bins[1]}-{bins[2]}":
                            clusters_rectangle(welldata, k, 'lightgreen')
                        if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == f"{bins[2]}-{bins[3]}":
                            clusters_rectangle(welldata, k, 'yellow')
                        if welldata['phit'].iloc[k] > 0 and welldata['cluster'].iloc[k] == f"{bins[3]}-{bins[4]}":
                            clusters_rectangle(welldata, k, 'orange')
                    fig.suptitle(clustering)
                    fig.tight_layout()
                    counter +=1
        if output_flag == 'print':
            plt.savefig('.\plots\\' + clustering + '_' + str(letter) +'.pdf')
        else:
            pass
        plt.show()
coloring_clusters_matrix(ntd_phit8_clusters, ['C','B','D'], 4, 9, 'tst_interval bal8', 'dontprint')

In [ ]:
def htst_sum_count_calc(dataset):
    htst_range = dataset.groupby(['well','htst_interv']).agg({'htst':['sum','count'], 'phit_avg':['mean'], 'perm_avg':['mean']}).reset_index()
    htst_range.columns = htst_range.columns.map('_'.join)
    htst_range = htst_range.rename(columns={'well_':'well', 'htst_interv_':'htst_interv'})
    return htst_range
phit8_range = htst_sum_count_calc(ntd_dist_phi8)

def ranged_data_transpose(dataset, var):
    df_lst = []
    for wellname in dataset.well.unique():
        data = dataset[dataset.well == wellname][['htst_interv',var]].T.reset_index()
        data.columns = data.iloc[0]
        data = data.iloc[1:]
        data['well'] = wellname
        df_lst.append(data)
    df = pd.concat(df_lst).reset_index(drop=True)
    df.insert(0, 'well', df.pop('well'))
    df_sum = dataset.groupby('well')['htst_sum'].sum()
    df = df.set_index('well').join(df_sum).reset_index()
    return df
phit8_range_hcount = ranged_data_transpose(phit8_range, 'htst_count')

def ranged_data_transpose_selected_var(dataset, var, prefix):
    df_lst = []
    for wellname in dataset.well.unique():
        data = dataset[dataset.well == wellname][['htst_interv',var]].T.reset_index()
        data.columns = prefix + data.iloc[0]
        data = data.iloc[1:]
        data['well'] = wellname
        df_lst.append(data)
    df = pd.concat(df_lst).reset_index(drop=True)
    df.insert(0, 'well', df.pop('well'))
    df.drop(df.columns[1], axis=1, inplace=True)
    return df
phit8_range_phit_avg = ranged_data_transpose_selected_var(phit8_range, 'phit_avg_mean', 'phit_')
phit8_range_hcount = phit8_range_hcount.merge(phit8_range_phit_avg, on='well')

phit8_range_perm_avg = ranged_data_transpose_selected_var(phit8_range, 'perm_avg_mean', 'perm_')
phit8_range_hcount = phit8_range_hcount.merge(phit8_range_perm_avg, on='well')

def convert_linestringz_polygon_run():
    def convert_linestringz_polygon(dataset):
        geom = [x for x in dataset.geometry]
        df_lst = []
        for i in range(len(geom)):
            all_coords = mapping(geom[i])['coordinates']
            lats = [x[1] for x in all_coords]
            lons = [x[0] for x in all_coords]
            polyg = Polygon(zip(lons, lats))
            df = gpd.GeoDataFrame(index=[0], crs='EPSG:2499', geometry=[polyg])
            df_lst.append(df)
        result = pd.concat(df_lst).reset_index(drop=True)
        return result 
    bal8_20_3 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_20_base_3.shp').set_crs('EPSG:2499')
    bal8_20_3_polygon = convert_linestringz_polygon(bal8_20_3)
    bal8_20_1 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_20_base_1.shp').set_crs('EPSG:2499')
    bal8_20_1_polygon = convert_linestringz_polygon(bal8_20_1)
    bal8_20_0 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_20_base_0.shp').set_crs('EPSG:2499')
    bal8_20_0_polygon = convert_linestringz_polygon(bal8_20_0)
    bal8_30_3 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_30_base_3.shp').set_crs('EPSG:2499')
    bal8_30_3_polygon = convert_linestringz_polygon(bal8_30_3)
    bal8_30_1 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_30_base_1.shp').set_crs('EPSG:2499')
    bal8_30_1_polygon = convert_linestringz_polygon(bal8_30_1)
    bal8_30_0 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_30_base_0.shp').set_crs('EPSG:2499')
    bal8_30_0_polygon = convert_linestringz_polygon(bal8_30_0)
    bal8_1510_3 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_1510_base_3.shp').set_crs('EPSG:2499')
    bal8_1510_3_polygon = convert_linestringz_polygon(bal8_1510_3)
    bal8_1510_1 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_1510_base_1.shp').set_crs('EPSG:2499')
    bal8_1510_1_polygon = convert_linestringz_polygon(bal8_1510_1)
    bal8_1510_0 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_1510_base_0.shp').set_crs('EPSG:2499')
    bal8_1510_0_polygon = convert_linestringz_polygon(bal8_1510_0)
    result = {'bal8_20':[bal8_20_0_polygon, bal8_20_1_polygon, bal8_20_3_polygon],
              'bal8_30':[bal8_30_0_polygon, bal8_30_1_polygon, bal8_30_3_polygon],
              'bal8_1510':[bal8_1510_0_polygon, bal8_1510_1_polygon, bal8_1510_3_polygon]}
    return result
polygons_dict= convert_linestringz_polygon_run()

xy8 = df_interp[df_interp.FORMATION_up == 'Balakhany VIII'].groupby('well')[['Xmean','Ymean']].apply(lambda x: x.iloc[0]).dropna()
geometry = [Point(xy) for xy in zip(xy8['Xmean'], xy8['Ymean'])]
bdl8_xy_gpd = gpd.GeoDataFrame(xy8, geometry=geometry).drop(['Xmean','Ymean'], axis=1)
phit8_range_hcount_xy = phit8_range_hcount.set_index('well').join(bdl8_xy_gpd).reset_index()
phit8_range_hcount_xy = phit8_range_hcount_xy[~phit8_range_hcount_xy.geometry.isna()]

# def mixmax_var(dataset, var):
#     dataset[var] = (dataset[var] - dataset[var].min()) / (dataset[var].max() - dataset[var].min())
#     return dataset
# phit8_range_hcount_xy = mixmax_var(phit8_range_hcount_xy, f'{bins[3]}-{bins[4]}')

bal8_20_list = [polygons_dict['bal8_20']] 
bal8_30_list = [polygons_dict['bal8_30']]
bal8_1510_list = [polygons_dict['bal8_1510']]

def piechart_map(dataset_map, dataset_polygon, comment, print_flag, path):
    fig, ax = plt.subplots(figsize=(15,10), dpi = 300, layout='tight')
    if len(dataset_polygon) != 0:
        dataset_polygon[2].plot(ax=ax, color='orange', label='bal8_1510_3', alpha=0.5)
        dataset_polygon[1].plot(ax=ax, color='yellow', label='bal8_1510_1', alpha=0.5)
        dataset_polygon[0].plot(ax=ax, color='green', label='bal8_1510_0', alpha=0.5)
    else:
        pass
    for ind, row in dataset_map.iterrows():
            # print(row['well'], row['htst_sum'])
            ax.pie([row[f'{bins[0]}-{bins[1]}'], row[f'{bins[1]}-{bins[2]}'], row[f'{bins[2]}-{bins[3]}'], row[f'{bins[3]}-{bins[4]}']], 
                    radius=row['htst_sum']*2.5, center=(row['geometry'].x, row['geometry'].y), 
                    wedgeprops={"linewidth": 0.1, "edgecolor": "black", "alpha":0.66},
                    #  wedgeprops={"linewidth": 0.1, "edgecolor": "black", "alpha":row[f'{bins[3]}-{bins[4]}']},
                    colors=['green', 'lightgreen', 'yellow','orange'], frame=True)
            ax.scatter(x = row['geometry'].x, y = row['geometry'].y, s=0.5, c='black')
            if ind % 2 == 0:
                ax.text(row['geometry'].x, row['geometry'].y, row['well'], fontsize=4, ha='left', va='bottom')
            else:
                ax.text(row['geometry'].x, row['geometry'].y, row['well'], fontsize=4, ha='left', va='top')
                #  pass
    stats = (f" orange = htst {bins[3]}-{bins[4]} \n yellow = htst {bins[2]}-{bins[3]} \n lightgreen = htst {bins[1]}-{bins[2]} \n green = htst {bins[0]}-{bins[1]}")
    bbox = dict(boxstyle='round', fc='white', ec='black', alpha=1)
    ax.text(0.8, 0.85, stats, fontsize=10, bbox=bbox, transform=ax.transAxes, horizontalalignment='left')
    ax.set_title(comment, fontsize=20)
    if print_flag == 'print':
        plt.savefig(path, format='pdf', bbox_inches='tight')
    plt.show();       
piechart_map(phit8_range_hcount_xy, empty_list, 'Bal8 piechart:htst_count, size:htst_sum polygon:', 'dontprint', 
             'C:\\jupyter\\SPP\plots\\bal8_htst_count_phi_empty2.pdf')
# piechart_map(phit8_range_hcount_xy, bal8_20_list[0], 'Bal8 piechart:htst_count, size:htst_sum polygon:bal8_20', 'dontprint', 
#              'C:\\jupyter\\SPP\plots\\bal8_htst_count_phi_20.pdf')
# piechart_map(phit8_range_hcount_xy, bal8_30_list[0], 'Bal8 piechart:htst_count, size:htst_sum polygon:bal8_30', 'dontprint', 
#              'C:\\jupyter\\SPP\plots\\bal8_htst_count_phi_30.pdf')
# piechart_map(phit8_range_hcount_xy, bal8_1510_list[0], 'Bal8 piechart:htst_count, size:htst_sum polygon:bal8_1510', 'dontprint', 
#              'C:\\jupyter\\SPP\plots\\bal8_htst_count_phi_1510.pdf')

In [ ]:
phit8_range_hsum = ranged_data_transpose(phit8_range, 'htst_sum')

xy8 = df_interp[df_interp.FORMATION_up == 'Balakhany VIII'].groupby('well')[['Xmean','Ymean']].apply(lambda x: x.iloc[0])
phit8_range_hsum_xy = phit8_range_hsum.set_index('well').join(xy8).reset_index()
for col in phit8_range_hsum_xy.columns:
    try:
        phit8_range_hsum_xy[col] = pd.to_numeric(phit8_range_hsum_xy[col])
    except:
        phit8_range_hsum_xy[col] = phit8_range_hsum_xy[col].astype('string')
phit8_range_hsum_xy[f'{bins[3]}-{bins[4]}'] = phit8_range_hsum_xy[f'{bins[3]}-{bins[4]}'].round(0).astype('int')

def bubble_map(dataset_map, comment):
    fig, ax = plt.subplots(figsize=(15,10))
    ax.scatter(x = dataset_map['Xmean']/1000, y = dataset_map['Ymean']/1000, s=dataset_map[f'{bins[3]}-{bins[4]}'].values*5, c='orange', ec='gray', alpha=0.75)
    ax.scatter(x = dataset_map['Xmean']/1000, y = dataset_map['Ymean']/1000, s=1, c='orange', ec='black', alpha=0.75)
    for ind, row in dataset_map.iterrows():
            ax.text(row['Xmean']/1000, row['Ymean']/1000, row['well'], fontsize=7, ha='left', va='bottom')
    stats = (f' orange = htst {bins[3]}-{bins[4]}')
    bbox = dict(boxstyle='round', fc='white', ec='black', alpha=1)
    ax.text(0.8, 0.95, stats, fontsize=12, bbox=bbox, transform=ax.transAxes, horizontalalignment='left')
    ax.set_title(comment, fontsize=18)
    ax.grid()
    # plt.savefig('C:\\jupyter\\SPP\\plots\\bal8_htst_sum.pdf', format='pdf', bbox_inches='tight')
    plt.show()
bubble_map(phit8_range_hsum_xy, f'Bal8 htst_sum for interval {bins[3]}-{bins[4]}')

In [ ]:
def distrib_perm_per_htst_bins(data, variable, max_x, step, comment):
    fig, ax = plt.subplots(1, 4, figsize=(18, 4))
    sns.histplot(data=data[data.htst_interv == f'{bins[0]}-{bins[1]}'], x=variable, bins=30, ax=ax[0], kde=True,  color='green')
    ax[0].grid(True, axis='x'), ax[0].set_xticks(np.arange(0, max_x, step)), ax[0].tick_params(axis='both', which='major', labelsize=8)
    ax[0].set_title(f'{bins[0]}-{bins[1]}'), ax[0].set_xlim([0, max_x])
    sns.histplot(data=data[data.htst_interv == f'{bins[1]}-{bins[2]}'], x=variable, bins=30, ax=ax[1], kde=True,  color='lightgreen')
    ax[1].grid(True, axis='x'), ax[1].set_xticks(np.arange(0, max_x, step)), ax[1].tick_params(axis='both', which='major', labelsize=8)
    ax[1].set_title(f'{bins[1]}-{bins[2]}'), ax[1].set_xlim([0, max_x])
    sns.histplot(data=data[data.htst_interv == f'{bins[2]}-{bins[3]}'], x=variable, bins=30, ax=ax[2], kde=True,  color='yellow')
    ax[2].grid(True, axis='x'), ax[2].set_xticks(np.arange(0, max_x, step)), ax[2].tick_params(axis='both', which='major', labelsize=8)
    ax[2].set_title(f'{bins[2]}-{bins[3]}'), ax[2].set_xlim([0, max_x])
    sns.histplot(data=data[data.htst_interv == f'{bins[3]}-{bins[4]}'], x=variable, bins=30, ax=ax[3], kde=True,  color='orange')
    ax[3].grid(True, axis='x'), ax[3].set_xticks(np.arange(0, max_x, step)), ax[3].tick_params(axis='both', which='major', labelsize=8)
    ax[3].set_title(f'{bins[3]}-{bins[4]}'), ax[3].set_xlim([0, max_x])
    fig.suptitle(comment)
    plt.show();
distrib_perm_per_htst_bins(ntd_dist_phi8, 'perm_avg', 610, 100,'Bal8 Azeri distrib perm_avg per htst interval')
distrib_perm_per_htst_bins(ntd_dist_phi8, 'phit_avg', 0.35, 0.05, 'Bal8 Azeri distrib phit_avg per htst interval')
distrib_perm_per_htst_bins(ntd_dist_phi8, 'vsh_avg', 1, 0.1, 'Bal8 Azeri distrib vsh_avg per htst interval')

## Htst distribution Bal10 phi

In [ ]:
quantiles = ntd_top_phi_bot10['htst'].quantile([0.25, 0.5, 0.75]).round(0).reset_index().rename(columns={'index':'quantile'})
bins = [0, int(quantiles.iloc[0][1]), int(quantiles.iloc[1][1]), int(quantiles.iloc[2][1]), 100]

def htst_distrib_histo(dataset, bins, comment):
        dataset['htst_interv'] = pd.cut(dataset['htst'], bins=bins, 
                                        labels=[f'{bins[0]}-{bins[1]}', 
                                                f'{bins[1]}-{bins[2]}', 
                                                f'{bins[2]}-{bins[3]}', 
                                                f'{bins[3]}-{bins[4]}'])
        custom_palette = {f'{bins[0]}-{bins[1]}': 'green', 
                        f'{bins[1]}-{bins[2]}': 'lightgreen', 
                        f'{bins[2]}-{bins[3]}': 'yellow',
                        f'{bins[3]}-{bins[4]}': 'orange'}
        fig, ax = plt.subplots(1, 2, figsize=(16, 4))
        ax[0] = sns.histplot(data=dataset, x='htst', bins=75, kde=True, log_scale=False, ax=ax[0])
        ax[0].grid(True)

        ax[1] = sns.histplot(data=dataset, x='htst', bins=75, hue='htst_interv', kde=True, 
                        log_scale=False, ax=ax[1], palette = custom_palette)
        ax[1].grid(True)
        fig.suptitle(comment)
        plt.show()
        return dataset
ntd_dist_phi10 = htst_distrib_histo(ntd_top_phi_bot10, bins, 'Bal10 Azeri')

In [ ]:
def htst_sum_count_calc(dataset):
    htst_range = dataset.groupby(['well','htst_interv']).agg({'htst':['sum','count']}).reset_index()
    htst_range.columns = htst_range.columns.map('_'.join)
    htst_range = htst_range.rename(columns={'well_':'well', 'htst_interv_':'htst_interv'})
    return htst_range
phi10_range = htst_sum_count_calc(ntd_dist_phi10)

def ranged_data_transpose(dataset, var):
    df_lst = []
    for wellname in dataset.well.unique():
        data = dataset[dataset.well == wellname][['htst_interv',var]].T.reset_index()
        data.columns = data.iloc[0]
        data = data.iloc[1:]
        data['well'] = wellname
        df_lst.append(data)
    df = pd.concat(df_lst).reset_index(drop=True)
    df.insert(0, 'well', df.pop('well'))
    df_sum = dataset.groupby('well')['htst_sum'].sum()
    df = df.set_index('well').join(df_sum).reset_index()
    return df
phit10_range_hcount = ranged_data_transpose(phi10_range, 'htst_count')

xy10 = df_bal10_vshclp2.groupby('well')[['Xmean','Ymean']].apply(lambda x: x.iloc[0])
phit10_range_hcount_xy = phit10_range_hcount.set_index('well').join(xy10).reset_index()
phit10_range_hcount_xy = phit10_range_hcount_xy.dropna()

def piechart_map(dataset_map, comment, print_flag):
    fig, ax = plt.subplots(figsize=(15,10))
    for ind, row in dataset_map.iterrows():
            ax.pie([row[f'{bins[0]}-{bins[1]}'], row[f'{bins[1]}-{bins[2]}'], row[f'{bins[2]}-{bins[3]}'], row[f'{bins[3]}-{bins[4]}']], 
                    radius=row['htst_sum']*0.007, center=(row['Xmean']/1000, row['Ymean']/1000), wedgeprops={"linewidth": 0.5, "edgecolor": "gray", "alpha":0.75},
                    colors=['green', 'lightgreen', 'yellow','orange'], frame=True)
            ax.scatter(x = dataset_map['Xmean']/1000, y = dataset_map['Ymean']/1000, s=0.5, c='gray') 
            # ax.text(row['Xmean']/1000, row['Ymean']/1000, row['well'], fontsize=7, ha='left', va='bottom')
            if ind % 2 == 0:
                ax.text(row['Xmean']/1000+0.1, row['Ymean']/1000+0.1, row['well'], fontsize=4, ha='left', va='bottom')
            else:
                ax.text(row['Xmean']/1000-0.1, row['Ymean']/1000-0.1, row['well'], fontsize=4, ha='left', va='bottom')
    stats = (' orange = htst 7-100 \n yellow = htst 4-7 \n lightgreen = htst 2-4 \n green = htst 0-2')
    bbox = dict(boxstyle='round', fc='white', ec='black', alpha=1)
    ax.text(0.8, 0.85, stats, fontsize=8, bbox=bbox, transform=ax.transAxes, horizontalalignment='left')
    ax.set_title(comment, fontsize=20)
    if print_flag == 'print':
        plt.savefig('C:\\jupyter\\SPP\plots\\bal10_htst_count.pdf', format='pdf', bbox_inches='tight')
    plt.show();       
piechart_map(phit10_range_hcount_xy, 'Bal10 piechart:htst_count, size:htst_sum', 'dontprint')

# Additional things

## Phit vs Htst xplots Bal8 & Bal10

In [ ]:
def get_bins_limits(dataset):
    quantiles = dataset['htst'].quantile([0.25, 0.5, 0.75]).round(0).reset_index().rename(columns={'index':'quantile'})
    bins = [0, int(quantiles.iloc[0][1]), int(quantiles.iloc[1][1]), int(quantiles.iloc[2][1]), 100]
    return bins
bins8 = get_bins_limits(ntd_dist_phi8)

def htst_xplots_properties(dataset, bins, comment):
    fig, ax = plt.subplots(1, 3, figsize=(18, 5))
    custom_palette = {  f'{bins[0]}-{bins[1]}': '#1f77b4', 
                        f'{bins[1]}-{bins[2]}': '#ff7f0e', 
                        f'{bins[2]}-{bins[3]}': '#2ca02c',
                        f'{bins[3]}-{bins[4]}': '#d62728'}
    sns.scatterplot(data=dataset, x='phit_avg', y='htst', hue='htst_interv', alpha=0.5, ec='gray', palette = custom_palette, ax=ax[0])
    ax[0].grid()
    sns.scatterplot(data=dataset, x='vsh_avg', y='htst', hue='htst_interv',  alpha=0.5, ec='gray', palette = custom_palette, ax=ax[1])
    ax[1].grid()
    sns.scatterplot(data=dataset, x='perm_avg', y='htst', hue='htst_interv',  alpha=0.5, ec='gray', palette = custom_palette, ax=ax[2])
    ax[2].set_xscale('log')
    ax[2].grid()
    fig.suptitle(comment, fontsize=18)
    plt.show();
htst_xplots_properties(ntd_dist_phi8, bins8, 'Bal8 Azeri')

In [ ]:
ntd_dist_phi8[(ntd_dist_phi8.phit_avg < 0.16) & (ntd_dist_phi8.htst>5)]

In [ ]:
bins10 = get_bins_limits(ntd_dist_phi10)
htst_xplots_properties(ntd_dist_phi10, bins10, 'Bal10 Azeri')

In [ ]:
ntd_dist_phi10[(ntd_dist_phi10.phit_avg < 0.13) & (ntd_dist_phi10.htst>5) & (ntd_dist_phi10.htst<10)]

In [73]:
# well_display_logs(df_bal10_vshclp2, 'A20', 'Balakhany X', 'NET_clp2', 'outliers detect', 'TST', 10, 'dontprint','path')

## Vsh maps

In [74]:
def well_display_logs(dataset, wellname, fmname, net_var, comments, ref_depth, depth_step=10, print=0):
    """
    dataset = df_bal or something else
    net_var = NET or FLUIDS_int
    comments = put what you want
    ref_depth = MD or TST
    depth_step = step for ticks on the diagramm
    print = 1 if we want to print the plot
    well_display_logs(df_bal8_azr_vshclp2_flag, 'C01', 'Balakhany VIII', 'NET_clp2', 'df_interp', 'TST')
    """
    data = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname)]
    depth = data[ref_depth]
    grn = data['GR_N']
    vsh = data['VSH']
    rhob = data['RHOB'] 
    npss = data['NPSS']
    rdeep = data['RDEEP']
    phit = data['PHIT'] 
    net = data[net_var]
    perm = data['LPERM']
    # perm1 = data['PERM_DS']
    # perm2 = data['PERM_US']
    fig, ax = plt.subplots(1,4, figsize=(7,7), sharey=True)
    well_bal_tops = dataset[(dataset.well == wellname)].groupby('FORMATION')[ref_depth].apply(lambda x: x.iloc[0]).reset_index()
    ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[0].plot(grn, depth, color='lightgreen', lw=3, zorder=10)
    ax[0].invert_yaxis() 
    ax[0].set_xlim(0, 150) 
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
                    xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    twin0 = ax[0].twiny()
    twin0.plot(vsh, depth, color='black', alpha=0.5, zorder=5)
    twin0.xaxis.set_ticks(np.arange(0, 1.5, 0.5))
    twin0.set_xlim(0, 1.5)
    twin0.grid(axis='x')
    ax[1].plot(rhob, depth, color='red', alpha=0.75) 
    ax[1].invert_yaxis() 
    ax[1].xaxis.set_ticks(np.arange(1.65, 2.65, 0.3))
    ax[1].grid(axis='x')
    ax[1].set_xlim(1.65, 2.65)
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[1].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
        xmin=0, xmax=150, linewidth=2, color='black', lw=2, alpha=0.33)
        ax[1].text(1.67, well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0]+0.5*depth_step, i, fontsize = 8, color ="black")
    twin1 = ax[1].twiny()
    twin1.plot(npss, depth, color='blue', alpha=0.75)
    twin1.set_xlim(0.6, 0)
    ax[2].plot(phit, depth, color='green', linestyle='dashed')
    ax[2].set_xlim(0.3, 0)
    ax[2].xaxis.set_ticks(np.arange(0, 0.3, 0.1))
    ax[2].grid(axis='x')
    ax[2].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[2].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    ax[2].vlines(0.13, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed')
    twin2 = ax[2].twiny()
    twin2.plot(net, depth, color='orange', linewidth=0.5)
    twin2.fill_betweenx(depth,net, color='orange', alpha=0.33)
    twin2.set_xlim(0, 1)
    twin2.set_ylim(min(depth), max(depth))
    # twin2.grid(axis='x')
    ax[3].plot(perm, depth, color='purple', alpha=0.75)
    # ax[3].plot(perm1, depth, color='purple', alpha=0.5)
    # ax[3].plot(perm2, depth, color='purple', alpha=0.5)
    ax[3].set_xscale('log')
    ax[3].set_xlim(0.1, 1000)
    ax[3].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[3].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    # fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(round(max(kh.dropna()),0)) + ' ' + str(comments), fontsize=14)
    fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(comments), fontsize=14)
    fig.tight_layout()
    if print == 1:
        path = 'C:\\jupyter\\SPP\\inputoutput\\wellplots\\'
        fig.savefig(path + fmname.replace(' ','') + '_' + wellname + '.png')
    else:
        pass
    plt.show()
# well_display_logs(df_bal8_azr_vshclp2, 'C18X', 'Balakhany VIII', 'NET', 'df_interp', 'TST')

In [75]:
def vsh_wavg_for_map(dataset, dataset_xy):
    data = dataset.groupby('well')['htst'].sum().reset_index().rename(columns={'htst':'htst_sum'})
    data_merge = pd.merge(ntd_top_vsh_bot8, data, on='well', how='left')
    data_merge['vsh_htst'] = data_merge['vsh_avg']*data_merge['htst']
    data3 = data_merge.groupby('well')['vsh_htst'].sum().reset_index().rename(columns={'vsh_htst':'vsh_htst_sum'})
    data_merge2 = pd.merge(data_merge, data3, on='well', how='left')
    data_merge2['vsh_wavg'] = data_merge2['vsh_htst_sum']/data_merge2['htst_sum']
    xy = dataset_xy.groupby('well')[['Xmean','Ymean']].apply(lambda x: x.iloc[0])
    result = data_merge2.merge(xy, on='well', how='left')
    result = result.groupby('well').apply(lambda x: x.iloc[0]).reset_index(drop=True)[['well','vsh_wavg', 'htst_sum','Xmean','Ymean']]
    result = result[~(result.well == 'C18X')]
    result['vsh_wavg'] = result['vsh_wavg'].round(2)
    return result
vsh_wavg8 = vsh_wavg_for_map(ntd_top_vsh_bot8, df_interp)
def vsh_total_for_map(dataset):
    data = dataset.groupby('well').agg({'VSH':'mean', 'NET_clp2':'sum','Xmean':(lambda x: x.iloc[0]), 'Ymean':(lambda x: x.iloc[0])}).reset_index()
    data['NET_clp2'] = data['NET_clp2']*0.1
    data = data.rename(columns={'NET_clp2':'htst_sum'})
    data['VSH'] = data['VSH'].round(2)
    result = data[~(data.well == 'C18X')]
    return result
vsh_total8 = vsh_total_for_map(df_bal8_azr_vshclp2)

def display_vshavg_map(dataset, var, coeff, comment, print_flag, path):
    wells = go.Scatter( x=dataset.Xmean, y=dataset.Ymean, 
                        mode='markers',
                        marker=dict(color=dataset[var], size=dataset.htst_sum*coeff, colorscale='viridis_r',  showscale=True,),
                        customdata = dataset[['well', var, 'htst_sum']],
                        hovertemplate="".join(["well:%{customdata[0]},vsh:%{customdata[1]}, htst_sum:%{customdata[2]}<extra></extra>"]))
    wells_centers = go.Scatter( x=dataset.Xmean, y=dataset.Ymean, 
                                mode='markers',
                                marker=dict(symbol='diamond', color='black', size=1),
                                customdata = dataset[['well']],
                                hovertemplate="".join(["well:%{customdata[0]}<extra></extra>"]))
    fig = go.Figure()  
    fig.add_trace(wells)
    fig.add_trace(wells_centers)
    fig.update_layout(  title_text= (comment),
                        autosize=True, margin=dict(l=10,r=10,b=10,t=50), showlegend=False)
    if print_flag == 'print':
        fig.write_html(path)
    fig.show()
# display_vshavg_map(vsh_wavg8, 'vsh_wavg', 0.4, 'Map of vsh_wavg Bal8', 'dontprint',r'C:\jupyter\SPP\plots\vsh_wavg_bal8.html')
# display_vshavg_map(vsh_total8, 'VSH', 0.4, 'Map of vsh_total Bal8', 'dontprint' ,r'C:\jupyter\SPP\plots\vsh_total_bal8.html')

## Ternary plot

In [102]:
# well_display_logs(df_bal8_azr_vshclp2, 'D36', 'Balakhany VIII', 'NET_clp2', 'df_interp', 'TST', 10, 'dontprint')

In [ ]:
def wavg_for_ternary(dataset, dataset_xy):
    data = dataset.groupby('well')['htst'].sum().reset_index().rename(columns={'htst':'htst_sum'})
    data_merge = pd.merge(ntd_top_vsh_bot8, data, on='well', how='left')
    data_merge['vsh_htst'] = data_merge['vsh_avg']*data_merge['htst']
    data_merge['phit_htst'] = data_merge['phit_avg']*data_merge['htst']
    data_merge['perm_htst'] = data_merge['perm_avg']*data_merge['htst']
    data3 = data_merge.groupby('well')[['vsh_htst', 'phit_htst','perm_htst' ]].sum().reset_index().rename(
        columns={'vsh_htst':'vsh_htst_sum', 'phit_htst':'phit_htst_sum', 'perm_htst':'perm_htst_sum'})
    data_merge2 = pd.merge(data_merge, data3, on='well', how='left')
    data_merge2['vsh_wavg'] = data_merge2['vsh_htst_sum']/data_merge2['htst_sum']
    data_merge2['phit_wavg'] = data_merge2['phit_htst_sum']/data_merge2['htst_sum']
    data_merge2['perm_wavg'] = data_merge2['perm_htst_sum']/data_merge2['htst_sum']
    xy = dataset_xy.groupby('well')[['Xmean','Ymean']].apply(lambda x: x.iloc[0])
    result = data_merge2.merge(xy, on='well', how='left')
    result = result.groupby('well').apply(lambda x: x.iloc[0]).reset_index(drop=True)[['well','phit_wavg', 'vsh_wavg', 'perm_wavg', 'htst_sum','Xmean','Ymean']]
    result = result[~(result.well == 'C18X')]
    result['vsh_wavg'] = result['vsh_wavg'].round(3)
    result['phit_wavg'] = result['phit_wavg'].round(3)
    result['perm_wavg'] = result['perm_wavg'].round(0)
    result = result[result.phit_wavg != 0]
    return result
# var_wavg8 = wavg_for_ternary(ntd_top_vsh_bot8, df_interp)

def display_ternary(dataset, var_list):
    data = dataset[var_list]
    data[var_list[1]] = (data[var_list[1]] - data[var_list[1]].min()) / (data[var_list[1]].max() - data[var_list[1]].min())
    data[var_list[2]] = (data[var_list[2]] - data[var_list[2]].min()) / (data[var_list[2]].max() - data[var_list[2]].min())
    data[var_list[3]] = (data[var_list[3]] - data[var_list[3]].min()) / (data[var_list[3]].max() - data[var_list[3]].min())
    fig = go.Figure(go.Scatterternary({
        'mode': 'markers',
        'a': data[var_list[1]],
        'b': data[var_list[2]],
        'c': data[var_list[3]],
        'text': data[var_list[0]],
        'marker': {
            'symbol': 0,
            'color': '#2ca02c',
            'size': 10,
            'line': { 'width': 1 },
            'opacity': 0.7
        },
        'hovertemplate':
        "<b>well</b>: %{text}<br>" +
        "<b>phit_avg</b>: %{a}<br>" +
        "<b>vsh_avg</b>: %{b}<br>" +
        "<b>htst</b>: %{c}<extra></extra>"
    }))
    fig.update_layout({
        'ternary': {
            'sum': 1,
            'aaxis': {'title': var_list[1], 'min': 0.01, 'linewidth':1, 'ticks':'outside' },
            'baxis': {'title': var_list[2], 'min': 0.01, 'linewidth':1, 'ticks':'outside' },
            'caxis': {'title': var_list[3], 'min': 0.01, 'linewidth':1, 'ticks':'outside' }
        },
        'showlegend': False,
        'width': 800,
        'height': 800,
        'title': 'Bal8 Azeri ternary plot',
        'title_font': {'size': 24}
    })
    fig.show()
display_ternary(ntd_top_vsh_bot8_bp[ntd_top_vsh_bot8_bp.htst >=1], ['well','phit_avg','vsh_avg','htst'])

## Map of Htst cluster

In [ ]:
def data_clustering(dataset, feature_list, scaler, cluster_num):
    """
    MinMaxScaler(), StandardScaler()
    """
    data = dataset[feature_list]
    normalized_data = scaler.fit_transform(data)

    kmeans = KMeans(n_clusters=cluster_num, random_state=42)
    kmeans_labels = kmeans.fit_predict(normalized_data)
    kmeans_labels = pd.DataFrame(kmeans_labels, columns=['kmeans'])

    gmm = GaussianMixture(n_components=cluster_num, random_state=42)
    gmm.fit(normalized_data)
    gmm_labels = gmm.predict(normalized_data)
    gmm_labels = pd.DataFrame(gmm_labels, columns=['gmm'])

    agglomerative = AgglomerativeClustering(n_clusters=cluster_num)
    agglomerative_labels = agglomerative.fit_predict(normalized_data)
    agglomerative_labels = pd.DataFrame(agglomerative_labels, columns=['agglomer'])
    result = pd.concat([dataset, kmeans_labels, gmm_labels, agglomerative_labels], axis=1)
    return result
data_phi_clustered8 = data_clustering(ntd_top_phi_bot8, ['phit_avg','htst'], StandardScaler(), 3)

# 'kmeans', 'gmm', 'agglomer'
cluster_selected = 'kmeans'
def histo_clustering(dataset, clustering, comment):
    data = dataset[dataset.phit_avg !=0]
    fig, ax = plt.subplots(1, 4, figsize=(18, 4))
    custom_palette = {2: 'orange', 0: 'green', 1: 'yellow'}
    sns.histplot(data=data, x='phit_avg', hue=clustering, bins=30, ax=ax[0], kde=True,  palette=custom_palette)
    ax[0].grid(True, axis='x'), ax[0].set_xticks(np.arange(0.12, 0.32, 0.02)), ax[0].tick_params(axis='both', which='major', labelsize=8)
    sns.histplot(data=data[data.htst < 30], x='htst', hue=clustering, bins=30, ax=ax[1], kde=True,  palette=custom_palette)
    ax[1].grid(True, axis='x'), ax[1].set_xticks(np.arange(0, 30, 3)), ax[1].tick_params(axis='both', which='major', labelsize=8)
    sns.histplot(data=data, x='ntg', hue=clustering, bins=30, ax=ax[2], kde=True,  palette=custom_palette)
    ax[2].grid(True, axis='x'), ax[2].set_xticks(np.arange(0, 1, 0.1)), ax[2].tick_params(axis='both', which='major', labelsize=8)
    sns.histplot(data=data, x='vsh_avg', hue=clustering, bins=30, ax=ax[3], kde=True,  palette=custom_palette)
    ax[3].grid(True, axis='x'), ax[3].set_xticks(np.arange(0, 0.6, 0.1)), ax[3].tick_params(axis='both', which='major', labelsize=8)
    fig.suptitle(comment)
    plt.show()
histo_clustering(data_phi_clustered8, cluster_selected, cluster_selected + ' Bal VIII')

def well_collecting_clusters_top_phi_bot(dataset, clustering, fm):
    df_lst = []
    for wellname in dataset.well.unique()[:]:
        data = dataset[dataset.well == wellname]
        well_lst = []
        phit_lst = []
        htst_lst = []
        bot_lst = []
        ntg_lst = []
        vsh_lst = []
        cluster_lst = []
        for ind, row in data.iterrows():
            well_lst.append(wellname)
            well_lst.append(wellname)

            phit_lst.append(0)
            phit_lst.append(row['phit_avg'])

            cluster_lst.append(np.nan)
            cluster_lst.append(row[clustering])

            htst_lst.append(row['top_htst'])
            htst_lst.append(row['htst'])
            
            bot_lst.append(row['bot_htst'])

            ntg_lst.append(0)
            ntg_lst.append(row['ntg'])

            vsh_lst.append(0)
            vsh_lst.append(row['vsh_avg'])

        phit_lst.append(0)
        cluster_lst.append(np.nan)
        htst_lst.append(data['bot_htst'].iloc[-1])
        well_lst.append(wellname)
        well_collect_cluster_short = pd.DataFrame(zip(well_lst, phit_lst, htst_lst, ntg_lst, vsh_lst, cluster_lst ), columns=[  'well','phit', 'htst', 
                                                                                                                                'ntg', 'vsh', 'cluster'])
        well_last_row = pd.DataFrame({'well':[well_lst[-1]], 'phit':[0], 'htst': [bot_lst[-1]], 'ntg':[0], 'vsh':[0], 'cluster':[cluster_lst[-1]]})
        well_collect_cluster = pd.concat([well_collect_cluster_short, well_last_row]).reset_index(drop=True)
        well_collect_cluster['depth'] = well_collect_cluster['htst'].cumsum()
        df_lst.append(well_collect_cluster)
    result = pd.concat(df_lst)
    result['FORMATION_up'] = fm
    return result
ntd_phit8_clusters = well_collecting_clusters_top_phi_bot(data_phi_clustered8, cluster_selected, 'Balakhany VIII')

In [ ]:
def cluster_transpose(dataset, wellname):
    result = dataset[dataset.well==wellname]
    result.loc[result.cluster == 0, 'cluster_0'] = result.htst
    result.loc[result.cluster == 1, 'cluster_1'] = result.htst
    result.loc[result.cluster == 2, 'cluster_2'] = result.htst
    result.fillna(0)
    result = result.groupby('well').sum().reset_index()
    result = result[['well', 'cluster_0', 'cluster_1','cluster_2']]
    return result
df_lst = []
for wellname in ntd_phit8_clusters.well.unique():
    df = cluster_transpose(ntd_phit8_clusters, wellname)
    df_lst.append(df)
data_transpose = pd.concat(df_lst).reset_index(drop=True)

xy8 = df_bal8_azr_vshclp2.groupby('well')[['Xmean','Ymean']].apply(lambda x: x.iloc[0])
data_transpose_xy = data_transpose.set_index('well').join(xy8).reset_index()
data_transpose_xy['htst_sum'] = data_transpose_xy['cluster_0']+data_transpose_xy['cluster_1']+data_transpose_xy['cluster_2']
data_transpose_xy = data_transpose_xy.dropna()

def piechart_map(dataset_map, comment, print_flag, path):
    fig, ax = plt.subplots(figsize=(15,10), dpi = 300, layout='tight')
    for ind, row in dataset_map.iterrows():
            ax.pie([row['cluster_0'], row['cluster_1'], row['cluster_2']], 
                    radius=row['htst_sum']*0.003, center=(row['Xmean']/1000, row['Ymean']/1000), 
                    wedgeprops={"linewidth": 0.5, "edgecolor": "gray", "alpha":0.75},
                    colors=['yellow', 'green', 'orange'], frame=True)
            ax.scatter(x = dataset_map['Xmean']/1000, y = dataset_map['Ymean']/1000, s=0.5, c='gray') 
            if ind % 2 == 0:
                ax.text(row['Xmean']/1000+0.1, row['Ymean']/1000+0.1, row['well'], fontsize=6, ha='left', va='bottom')
            else:
                ax.text(row['Xmean']/1000-0.1, row['Ymean']/1000-0.1, row['well'], fontsize=6, ha='left', va='bottom')
    ax.set_title(comment, fontsize=14)
    if print_flag == 'print':
        plt.savefig(path, format='pdf', bbox_inches='tight')
    plt.show() 
piechart_map(data_transpose_xy, 'Bal8 clustering kmeans', 'dontprint', 'C:\\jupyter\\SPP\plots\\bal8_clustering_kmeans.pdf')

In [ ]:
ntd_phit8_clusters = ntd_phit8_clusters[ntd_phit8_clusters.cluster==1]
ntd_phit8_clusters_sum = ntd_phit8_clusters.groupby('well')['htst'].sum().reset_index()
xy8 = df_bal8_azr_vshclp2.groupby('well')[['Xmean','Ymean']].apply(lambda x: x.iloc[0])

phit8_cluster_hsum_xy = ntd_phit8_clusters_sum.set_index('well').join(xy8).reset_index()
for col in phit8_cluster_hsum_xy.columns:
    try:
        phit8_cluster_hsum_xy[col] = pd.to_numeric(phit8_cluster_hsum_xy[col])
    except:
        phit8_cluster_hsum_xy[col] = phit8_cluster_hsum_xy[col].astype('string')

def bubble_map(dataset_map, comment):
    fig, ax = plt.subplots(figsize=(15,10))
    ax.scatter(x = dataset_map['Xmean']/1000, y = dataset_map['Ymean']/1000, s=dataset_map['htst'].values*10, 
               c='green', ec='gray', alpha=0.5)
    ax.scatter(x = dataset_map['Xmean']/1000, y = dataset_map['Ymean']/1000, s=1, c='black')
    for ind, row in dataset_map.iterrows():
            ax.text(row['Xmean']/1000, row['Ymean']/1000, row['well'], fontsize=7, ha='left', va='bottom')
    stats = (' green = htst kmeans')
    bbox = dict(boxstyle='round', fc='white', ec='black', alpha=1)
    ax.text(0.8, 0.95, stats, fontsize=12, bbox=bbox, transform=ax.transAxes, horizontalalignment='left')
    ax.set_title(comment, fontsize=18)
    ax.grid()
    # plt.savefig('C:\\jupyter\\SPP\\plots\\bal8_htst_agglomer2_sum.pdf', format='pdf', bbox_inches='tight')
    plt.show()
bubble_map(phit8_cluster_hsum_xy, 'Bal8 htst kmeans1 (green)')

## P50 diagrams

In [ ]:
ntd_top_phi_bot8 = ntd_dataframe_calculation_bp(df_bal8_azr_v2_flag, 'Balakhany VIII', 'NET')
data_phi_clustered8 = ntd_top_phi_bot8[ntd_top_phi_bot8.htst>=1].copy()

In [89]:
def p50_calculation(dataset, wellname):
    well = dataset[dataset.well == wellname]
    well['htst_p50'] = well['htst'].quantile([0.5]).iloc[0]
    well['phit_avg_p50'] = well['phit_avg'].quantile([0.5]).iloc[0]
    well['vsh_avg_p50'] = well['vsh_avg'].quantile([0.5]).iloc[0]
    well['perm_avg_p50'] = well['perm_avg'].quantile([0.5]).iloc[0]
    return well
df_lst = []
for wellname in data_phi_clustered8.well.unique():
    well_p50 = p50_calculation(data_phi_clustered8, wellname)
    df_lst.append(well_p50)
data_phi_p50_bal8 = pd.concat(df_lst)

quantiles = data_phi_clustered8['htst'].quantile([0.25, 0.5, 0.75]).round(0).reset_index().rename(columns={'index':'quantile'})
bins = [0, int(quantiles.iloc[0][1]), int(quantiles.iloc[1][1]), int(quantiles.iloc[2][1]), 100]
quantiles_p50 = data_phi_p50_bal8['htst_p50'].quantile([0.25, 0.5, 0.75]).round(0).reset_index().rename(columns={'index':'quantile'})
bins_p50 = [0, int(quantiles_p50.iloc[0][1]), int(quantiles_p50.iloc[1][1]), int(quantiles_p50.iloc[2][1]), 100]

def p50_cut_by_bins(dataset, bins_p50, bins):
    dataset['htst_interv_50'] = pd.cut(dataset['htst_p50'], bins=bins_p50, 
                                       labels=[ f'{bins_p50[0]}-{bins_p50[1]}', 
                                                f'{bins_p50[1]}-{bins_p50[2]}', 
                                                f'{bins_p50[2]}-{bins_p50[3]}', 
                                                f'{bins_p50[3]}-{bins_p50[4]}'])
    dataset_gb = dataset.groupby('well')[
                            ['FORMATION_up','htst_p50', 'phit_avg_p50', 'vsh_avg_p50', 'perm_avg_p50','htst_interv_50']].apply(
                            lambda x: x.iloc[0]).reset_index()

    dataset['htst_interv'] = pd.cut(dataset['htst'], bins=bins, 
                                        labels=[f'{bins[0]}-{bins[1]}', 
                                                f'{bins[1]}-{bins[2]}', 
                                                f'{bins[2]}-{bins[3]}', 
                                                f'{bins[3]}-{bins[4]}'])
    return dataset, dataset_gb
data_phi_p50_bal8, data_phi_p50_bal8_gb = p50_cut_by_bins(data_phi_p50_bal8, bins_p50, bins)

In [ ]:
def htst_xplots_p50_properties(dataset, bins_p50, comment):
    fig, ax = plt.subplots(1, 3, figsize=(18, 5))
    custom_palette = {  f'{bins_p50[0]}-{bins_p50[1]}': '#1f77b4', 
                        f'{bins_p50[1]}-{bins_p50[2]}': '#ff7f0e', 
                        f'{bins_p50[2]}-{bins_p50[3]}': '#2ca02c',
                        f'{bins_p50[3]}-{bins_p50[4]}': '#d62728'}
    sns.scatterplot(data=dataset, x='phit_avg_p50', y='htst_p50', hue='htst_interv_50', alpha=0.5, ec='gray', ax=ax[0], s=50,
                    palette = custom_palette)
    ax[0].grid(True), ax[0].set_xticks(np.arange(0.12, 0.28, 0.02)), 
    ax[0].tick_params(axis='both', which='major', labelsize=8), ax[0].set_ylim(0, 25)
    sns.scatterplot(data=dataset, x='vsh_avg_p50', y='htst_p50', hue='htst_interv_50',  alpha=0.5, ec='gray', ax=ax[1], s=50,
                    palette = custom_palette)
    ax[1].grid(True), ax[1].set_xticks(np.arange(0, 0.65, 0.1)), 
    ax[1].tick_params(axis='both', which='major', labelsize=8), ax[1].set_ylim(0, 25)
    sns.scatterplot(data=dataset, x='perm_avg_p50', y='htst_p50', hue='htst_interv_50',  alpha=0.5, ec='gray', ax=ax[2], s=50,
                    palette = custom_palette)
    ax[2].set_xscale('log'), ax[2].grid(True), ax[2].set_xlim(0.2, 1000), ax[2].set_ylim(0, 25)
    fig.suptitle(comment, fontsize=18)
    plt.show();
htst_xplots_p50_properties(data_phi_p50_bal8_gb, bins_p50, 'Bal8 Azeri properties p50')

def htst_xplots_properties(dataset, bins, comment):
    fig, ax = plt.subplots(1, 3, figsize=(18, 5))
    custom_palette = {  f'{bins[0]}-{bins[1]}': '#1f77b4', 
                        f'{bins[1]}-{bins[2]}': '#ff7f0e', 
                        f'{bins[2]}-{bins[3]}': '#2ca02c',
                        f'{bins[3]}-{bins[4]}': '#d62728'}
    sns.scatterplot(data=dataset, x='phit_avg', y='htst', hue='htst_interv', alpha=0.5, ec='gray', ax=ax[0], palette = custom_palette)
    ax[0].grid(True)
    sns.scatterplot(data=dataset, x='vsh_avg', y='htst', hue='htst_interv',  alpha=0.5, ec='gray', ax=ax[1], palette = custom_palette)
    ax[1].grid(True)
    sns.scatterplot(data=dataset, x='perm_avg', y='htst', hue='htst_interv',  alpha=0.5, ec='gray', ax=ax[2], palette = custom_palette)
    ax[2].set_xscale('log')
    ax[2].grid(True)
    fig.suptitle(comment, fontsize=18)
    plt.show();
htst_xplots_properties(data_phi_p50_bal8, bins, 'Bal8 Azeri properties')

In [ ]:
def plot_3d_var_p50(df, print_flag):
    fig = go.Figure(data=[go.Scatter3d(
        x=df['htst_p50'],
        y=df['phit_avg_p50'],
        z=df['vsh_avg_p50'],
        mode='markers',
        marker=dict(size=8,
                    color=df['perm_avg_p50'],                # set color to an array/list of desired values
                    colorscale='Viridis',   # choose a colorscale
                    opacity=0.8),
        customdata = df[['well','htst_p50', 'phit_avg_p50', 'vsh_avg_p50','perm_avg_p50']],
        hovertemplate="".join(["well: %{customdata[0]} "+
                                "htst: %{customdata[1]} " + 
                                "phit: %{customdata[2]} " +
                                "vsh: %{customdata[3]} " +
                                "perm: %{customdata[4]} " + 
                                "<extra></extra>"]))])
    fig.update_layout(autosize=False,
                      width=1000,  
                      height=750,
                      title='Variables p50 for Bal8 Azeri',
                      scene=dict(xaxis_title='htst_p50', yaxis_title='phit_avg_p50', zaxis_title='vsh_avg_p50'),
                      margin=dict(l=10, r=10, b=10, t=50))
    if print_flag == 'print':
        fig.write_html('C:\\jupyter\\SPP\\plots\\Bal8_Azeri_3d_p50.html')
    fig.show()
var_p50_3d = data_phi_p50_bal8_gb.groupby('well')[['well','htst_p50', 'phit_avg_p50', 'vsh_avg_p50', 'perm_avg_p50']].apply(lambda x: x.iloc[0])
var_p50_3d = var_p50_3d.round({'htst_p50':0, 'phit_avg_p50':2, 'vsh_avg_p50':2, 'perm_avg_p50':0})
plot_3d_var_p50(var_p50_3d, 'print')

# Outside

## Compaparison cuttoff 1m (clean) vs 0.5m (orig)

In [ ]:
df_1m = pd.read_csv('C:\jupyter\SPP\inputoutput\df_bal8_azr_vshclp2.csv')
df_05m = pd.read_csv('C:\jupyter\SPP\inputoutput\df_bal8_vshclp2_cutoff05.csv')

ntd_1m_bal8 = ntd_dataframe_calculation(df_1m, 'Balakhany VIII', 'NET_clp2')
ntd_05m_bal8 = ntd_dataframe_calculation(df_05m, 'Balakhany VIII', 'NET_clp2')

In [ ]:
ntd_05m_bal8 = ntd_05m_bal8[~(ntd_05m_bal8.well == 'B28')]
quantiles_1m = ntd_1m_bal8['htst'].quantile([0.25, 0.5, 0.75]).round(0).reset_index().rename(columns={'index':'quantile'})
bins_1m = [0, int(quantiles.iloc[0][1]), int(quantiles.iloc[1][1]), int(quantiles.iloc[2][1]), 100]
quantiles_05m = ntd_05m_bal8['htst'].quantile([0.25, 0.5, 0.75]).round(0).reset_index().rename(columns={'index':'quantile'})
bins_05m = [0, int(quantiles.iloc[0][1]), int(quantiles.iloc[1][1]), int(quantiles.iloc[2][1]), 100]
print(ntd_1m_bal8.shape, ntd_05m_bal8.shape)
print(bins_05m, bins_1m)

In [ ]:
def htst_distrib_histo(dataset, bins, comment):
        dataset['htst_interv'] = pd.cut(dataset['htst'], bins=bins, 
                                        labels=[f'{bins[0]}-{bins[1]}', 
                                                f'{bins[1]}-{bins[2]}', 
                                                f'{bins[2]}-{bins[3]}', 
                                                f'{bins[3]}-{bins[4]}'])
        custom_palette = {f'{bins[0]}-{bins[1]}': 'green', 
                          f'{bins[1]}-{bins[2]}': 'lightgreen', 
                          f'{bins[2]}-{bins[3]}': 'yellow',
                        f'{bins[3]}-{bins[4]}': 'orange'}
        fig, ax = plt.subplots(1, 2, figsize=(16, 4))
        ax[0] = sns.histplot(data=dataset, x='htst', bins=75, kde=True, log_scale=False, ax=ax[0])
        ax[0].grid(True, axis='x', color='black', alpha=0.5), ax[0].set_xticks(np.arange(0, 30, 1)), ax[0].tick_params(axis='both', which='major', labelsize=8)

        ax[1] = sns.histplot(data=dataset, x='htst', bins=75, hue='htst_interv', kde=True, 
                        log_scale=False, ax=ax[1], palette = custom_palette)
        ax[1].grid(True, axis='x', color='black', alpha=0.5), ax[1].set_xticks(np.arange(0, 30, 1)), ax[1].tick_params(axis='both', which='major', labelsize=8)
        fig.suptitle(comment)
        plt.show();
        return dataset
ntd_dist_1m = htst_distrib_histo(ntd_1m_bal8, bins_1m, 'Bal8 Azeri 1m')
ntd_dist_05m = htst_distrib_histo(ntd_05m_bal8, bins_05m, 'Bal8 Azeri 0.5m')

In [ ]:
ntd_05m_bal8 = ntd_05m_bal8[~(ntd_05m_bal8.well == 'B28')]
def htst_sum_count_calc(dataset):
    htst_range = dataset.groupby(['well','htst_interv']).agg({'htst':['sum','count']}).reset_index()
    htst_range.columns = htst_range.columns.map('_'.join)
    htst_range = htst_range.rename(columns={'well_':'well', 'htst_interv_':'htst_interv'})
    return htst_range
range_05m = htst_sum_count_calc(ntd_05m_bal8)

def ranged_data_transpose(dataset, var):
    df_lst = []
    for wellname in dataset.well.unique():
        data = dataset[dataset.well == wellname][['htst_interv',var]].T.reset_index()
        data.columns = data.iloc[0]
        data = data.iloc[1:]
        data['well'] = wellname
        df_lst.append(data)
    df = pd.concat(df_lst).reset_index(drop=True)
    df.insert(0, 'well', df.pop('well'))
    df_sum = dataset.groupby('well')['htst_sum'].sum()
    df = df.set_index('well').join(df_sum).reset_index()
    return df
range_05m_hcount = ranged_data_transpose(range_05m, 'htst_count')

def convert_linestringz_polygon_run():
    def convert_linestringz_polygon(dataset):
        geom = [x for x in dataset.geometry]
        df_lst = []
        for i in range(len(geom)):
            all_coords = mapping(geom[i])['coordinates']
            lats = [x[1] for x in all_coords]
            lons = [x[0] for x in all_coords]
            polyg = Polygon(zip(lons, lats))
            df = gpd.GeoDataFrame(index=[0], crs='EPSG:2499', geometry=[polyg])
            df_lst.append(df)
        result = pd.concat(df_lst).reset_index(drop=True)
        return result 
    bal8_20_3 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_20_base_3.shp').set_crs('EPSG:2499')
    bal8_20_3_polygon = convert_linestringz_polygon(bal8_20_3)
    bal8_20_1 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_20_base_1.shp').set_crs('EPSG:2499')
    bal8_20_1_polygon = convert_linestringz_polygon(bal8_20_1)
    bal8_20_0 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_20_base_0.shp').set_crs('EPSG:2499')
    bal8_20_0_polygon = convert_linestringz_polygon(bal8_20_0)
    bal8_30_3 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_30_base_3.shp').set_crs('EPSG:2499')
    bal8_30_3_polygon = convert_linestringz_polygon(bal8_30_3)
    bal8_30_1 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_30_base_1.shp').set_crs('EPSG:2499')
    bal8_30_1_polygon = convert_linestringz_polygon(bal8_30_1)
    bal8_30_0 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_30_base_0.shp').set_crs('EPSG:2499')
    bal8_30_0_polygon = convert_linestringz_polygon(bal8_30_0)
    bal8_1510_3 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_1510_base_3.shp').set_crs('EPSG:2499')
    bal8_1510_3_polygon = convert_linestringz_polygon(bal8_1510_3)
    bal8_1510_1 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_1510_base_1.shp').set_crs('EPSG:2499')
    bal8_1510_1_polygon = convert_linestringz_polygon(bal8_1510_1)
    bal8_1510_0 = gpd.read_file(r'C:\jupyter\SPP\input\surfaces\petrel\BalakhanyVIII_1510_base_0.shp').set_crs('EPSG:2499')
    bal8_1510_0_polygon = convert_linestringz_polygon(bal8_1510_0)
    result = {'bal8_20':[bal8_20_0_polygon, bal8_20_1_polygon, bal8_20_3_polygon],
              'bal8_30':[bal8_30_0_polygon, bal8_30_1_polygon, bal8_30_3_polygon],
              'bal8_1510':[bal8_1510_0_polygon, bal8_1510_1_polygon, bal8_1510_3_polygon]}
    return result
polygons_dict= convert_linestringz_polygon_run()

xy8 = df_interp[df_interp.FORMATION_up == 'Balakhany VIII'].groupby('well')[['Xmean','Ymean']].apply(lambda x: x.iloc[0])
geometry = [Point(xy) for xy in zip(xy8['Xmean'], xy8['Ymean'])]
bdl8_xy_gpd = gpd.GeoDataFrame(xy8, geometry=geometry).drop(['Xmean','Ymean'], axis=1)
range_05m_hcount_xy = range_05m_hcount.set_index('well').join(bdl8_xy_gpd).reset_index()

bal8_20_list = [polygons_dict['bal8_20']] 
bal8_30_list = [polygons_dict['bal8_30']]
bal8_1510_list = [polygons_dict['bal8_1510']]
empty_list = []

def piechart_map(dataset_map, dataset_polygon, comment, print_flag, path):
    fig, ax = plt.subplots(figsize=(15,10), dpi = 300, layout='tight')
    if len(dataset_polygon) != 0:
        dataset_polygon[2].plot(ax=ax, color='orange', label='bal8_1510_3', alpha=0.5)
        dataset_polygon[1].plot(ax=ax, color='yellow', label='bal8_1510_1', alpha=0.5)
        dataset_polygon[0].plot(ax=ax, color='green', label='bal8_1510_0', alpha=0.5)
    else:
        pass
    for ind, row in dataset_map.iterrows():
            ax.pie([row[f'{bins[0]}-{bins[1]}'], row[f'{bins[1]}-{bins[2]}'], row[f'{bins[2]}-{bins[3]}'], row[f'{bins[3]}-{bins[4]}']], 
                    radius=row['htst_sum']*2, center=(row['geometry'].x, row['geometry'].y), wedgeprops={"linewidth": 0.1, "edgecolor": "black", "alpha":0.75},
                    colors=['green', 'lightgreen', 'yellow','orange'], frame=True)
            ax.scatter(x = row['geometry'].x, y = row['geometry'].y, s=0.5, c='black')
            if ind % 2 == 0:
                ax.text(row['geometry'].x, row['geometry'].y, row['well'], fontsize=4, ha='left', va='bottom')
            else:
                ax.text(row['geometry'].x, row['geometry'].y, row['well'], fontsize=4, ha='left', va='top')
                #  pass
    stats = (' orange = htst 7-100 \n yellow = htst 4-7 \n lightgreen = htst 2-4 \n green = htst 0-2')
    bbox = dict(boxstyle='round', fc='white', ec='black', alpha=1)
    ax.text(0.8, 0.85, stats, fontsize=10, bbox=bbox, transform=ax.transAxes, horizontalalignment='left')
    ax.set_title(comment, fontsize=20)
    if print_flag == 'print':
        plt.savefig(path, format='pdf', bbox_inches='tight')
    plt.show();       
piechart_map(range_05m_hcount_xy, empty_list, 'Bal8 piechart:htst_count, size:htst_sum polygon:', 'print', 
             'C:\\jupyter\\SPP\plots\\bal8_htst_05m_empty.pdf')